In [2]:
using Pkg
Pkg.activate(".")

In [3]:
using JuMP
using DataFrames
using CSV
using Gurobi

In [4]:
include("./parameters.jl")
include("./get_functions.jl")

loadcsv = CSV.File("./LOAD.csv");

In [4]:
loadcsv[1][2]

0.037894737

In [5]:
loaddis = Dict{Int64,Float64}()

for i = 1:24
    loaddis[loadcsv[i][1]] = loadcsv[i][2]
end

In [8]:
loadraw1 = [985.0197922,
985.7248887,
1001.58956,
1036.139287,
1139.788471,
1286.801089,
1347.086838,
1305.486145,
1232.508659,
1184.20955,
1153.185304,
1129.212024,
1107.706581,
1093.957199,
1087.611331,
1091.489362,
1170.460168,
1280.102672,
1266.000742,
1245.200396,
1204.657348,
1117.577932,
1031.203612,
991.0131123];

loadraw2 = [970.9178624,
968.4500247,
977.9688273,
1011.46091,
1097.83523,
1221.227115,
1274.109352,
1224.752598,
1174.690747,
1150.364918,
1128.506927,
1120.750866,
1112.642256,
1101.713261,
1099.245423,
1101.360713,
1163.056655,
1265.295646,
1256.129391,
1222.989857,
1176.10094,
1094.662296,
1009.345621,
963.8668976]

windraw1 = [713.2,
712.8,
708.4,
710.7,
701.4,
682.5,
614.7,
517.7,
426.6,
274.2,
93,
6.3,
3.8,
1.1,
0,
0,
0.9,
276.3,
272.9,
345.6,
411.7,
376.6,
561.3,
568.4];

windraw2 = [630,
672.3,
661.3,
654,
654.4,
666.4,
686.7,
689.3,
683.8,
698.6,
713.5,
710.3,
702.2,
699.2,
686.9,
695.4,
712.3,
712.4,
712.2,
711.2,
711.9,
696.7,
689.7,
666.6];


In [9]:
load = Dict()

load[1] = 0.01*loadraw1;
load[2] = 0.01*loadraw2;

wind = Dict()

wind[1] = 0.01*windraw1;
wind[2] = 0.01*windraw2;

In [10]:
nscen = 2

2

In [12]:
ptdfdf = DataFrame(CSV.File("./ptdfsmall.csv"));

In [13]:
ptdfdict = Dict()

for i = 1:38
    br = ptdfdf[i,1]
    ptdfdict[br] = Dict()
    for j = 2:25
        bus = parse(Int64,names(ptdfdf)[j])
        ptdfdict[br][bus] = ptdfdf[i,j]
    end
end

# If you mess up start here

In [12]:
exmodel = JuMP.Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [13]:
submodel = JuMP.read_from_file("./new_store_exp_Z1_newptdf_newobj.mps")

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2329 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [14]:
#add first stage variables to extensive form

for bus in buses        
    subnamePR = JuMP.name(get_PR_variable(submodel, bus))
    subnameER = JuMP.name(get_ER_variable(submodel, bus))
    
    JuMP.@variable(exmodel, base_name = string(subnamePR,"_0"))
    JuMP.@variable(exmodel, base_name = string(subnameER,"_0"))
end

In [15]:
#add ER_lb constraint
outercount = 0
for bus in buses
    outercount += 1
    subcon = get_ER_lb(submodel, bus)
    subconname = JuMP.name(subcon)
    exconname = string(subconname, "_0")
    
    terms = JuMP.constraint_object(subcon).func.terms
    value = JuMP.constraint_object(subcon).set.lower
    
    innercount = 0
    for (subvar, coeff) in terms
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
        if innercount == 0
            JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                        base_name = exconname)
            innercount += 1
        else
            excon = JuMP.constraint_by_name(exmodel, exconname)
            JuMP.set_normalized_coefficient(excon, exvar, coeff)
        end
    end
end

In [16]:
#add PR_lb constraint
outercount = 0
for bus in buses
    outercount += 1
    subcon = get_PR_lb(submodel, bus)
    subconname = JuMP.name(subcon)
    exconname = string(subconname, "_0")
    
    terms = JuMP.constraint_object(subcon).func.terms
    value = JuMP.constraint_object(subcon).set.lower
    
    innercount = 0
    for (subvar, coeff) in terms
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
        if innercount == 0
            JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                        base_name = exconname)
            innercount += 1
        else
            excon = JuMP.constraint_by_name(exmodel, exconname)
            JuMP.set_normalized_coefficient(excon, exvar, coeff)
        end
    end
end

In [17]:
#add ER_ub constraint
outercount = 0
for bus in buses
    outercount += 1
    subcon = get_ER_ub(submodel, bus)
    subconname = JuMP.name(subcon)
    exconname = string(subconname, "_0")
    
    terms = JuMP.constraint_object(subcon).func.terms
    value = JuMP.constraint_object(subcon).set.upper
    
    innercount = 0
    for (subvar, coeff) in terms
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
        if innercount == 0
            JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                        base_name = exconname)
            innercount += 1
        else
            excon = JuMP.constraint_by_name(exmodel, exconname)
            JuMP.set_normalized_coefficient(excon, exvar, coeff)
        end
    end
end

In [18]:
#add expansion budget
subcon = JuMP.constraint_by_name(submodel, "expansion_budget")
subconname = JuMP.name(subcon)
exconname = string(subconname, "_0")

terms = JuMP.constraint_object(subcon).func.terms
value = JuMP.constraint_object(subcon).set.upper

innercount = 0
for (subvar, coeff) in terms
    exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
    if innercount == 0
        JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                    base_name = exconname)
        innercount += 1
    else
        excon = JuMP.constraint_by_name(exmodel, exconname)
        JuMP.set_normalized_coefficient(excon, exvar, coeff)
    end
end

In [19]:
for (F, S) in list_of_constraint_types(exmodel)
    for con in all_constraints(exmodel, F, S)
        println(con)
    end
end

ER_lb_{101}_0 : -0.2 PR_{101}_0 + ER_{101}_0 >= 0.0
ER_lb_{102}_0 : -0.2 PR_{102}_0 + ER_{102}_0 >= 0.0
ER_lb_{103}_0 : -0.2 PR_{103}_0 + ER_{103}_0 >= 0.0
ER_lb_{104}_0 : -0.2 PR_{104}_0 + ER_{104}_0 >= 0.0
ER_lb_{105}_0 : -0.2 PR_{105}_0 + ER_{105}_0 >= 0.0
ER_lb_{106}_0 : -0.2 PR_{106}_0 + ER_{106}_0 >= 0.0
ER_lb_{107}_0 : -0.2 PR_{107}_0 + ER_{107}_0 >= 0.0
ER_lb_{108}_0 : -0.2 PR_{108}_0 + ER_{108}_0 >= 0.0
ER_lb_{109}_0 : -0.2 PR_{109}_0 + ER_{109}_0 >= 0.0
ER_lb_{110}_0 : -0.2 PR_{110}_0 + ER_{110}_0 >= 0.0
ER_lb_{111}_0 : -0.2 PR_{111}_0 + ER_{111}_0 >= 0.0
ER_lb_{112}_0 : -0.2 PR_{112}_0 + ER_{112}_0 >= 0.0
ER_lb_{113}_0 : -0.2 PR_{113}_0 + ER_{113}_0 >= 0.0
ER_lb_{114}_0 : -0.2 PR_{114}_0 + ER_{114}_0 >= 0.0
ER_lb_{115}_0 : -0.2 PR_{115}_0 + ER_{115}_0 >= 0.0
ER_lb_{116}_0 : -0.2 PR_{116}_0 + ER_{116}_0 >= 0.0
ER_lb_{117}_0 : -0.2 PR_{117}_0 + ER_{117}_0 >= 0.0
ER_lb_{118}_0 : -0.2 PR_{118}_0 + ER_{118}_0 >= 0.0
ER_lb_{119}_0 : -0.2 PR_{119}_0 + ER_{119}_0 >= 0.0
ER_lb_{120}_

In [20]:
# initialize objective function
JuMP.@objective(exmodel, Min, 0)

0

# start scenario loop

In [21]:
scen = 2

2

In [22]:
#adjust load balance values (remember to change load)

for bus in buses
    lf = loaddis[bus]
    for ts in timesteps
        con = get_load_balance(submodel, bus, ts)
        oldval = JuMP.constraint_object(con).set.value
        lval = load[scen][ts]
        JuMP.set_normalized_rhs(con, lf*lval)
        newval = JuMP.constraint_object(con).set.value
        println("$(name(con)), $(oldval), $(newval)")
    end
end

Dbal_{101,1}, 0.5673409942736843, 0.3679267704425019
Dbal_{101,2}, 0.5647604960842106, 0.36699158983650004
Dbal_{101,3}, 0.5695528501894737, 0.3705987150473192
Dbal_{101,4}, 0.5846671978105263, 0.3832904517023067
Dbal_{101,5}, 0.6377517352421054, 0.41602177310184507
Dbal_{101,6}, 0.7341517807578948, 0.4627808034019375
Dbal_{101,7}, 0.8012447378526317, 0.4828203880328042
Dbal_{101,8}, 0.7984799181473685, 0.46411677591276723
Dbal_{101,9}, 0.7306496757473685, 0.4451459691389854
Dbal_{101,10}, 0.6823574920421053, 0.43592776021636565
Dbal_{101,11}, 0.6403322338105263, 0.42764473201343195
Dbal_{101,12}, 0.6156331776, 0.42470559309592243
Dbal_{101,13}, 0.5984912959578947, 0.42163285666206674
Dbal_{101,14}, 0.582639663031579, 0.4174913427500736
Dbal_{101,15}, 0.5743452046736842, 0.41655616203038753
Dbal_{101,16}, 0.5726863125473685, 0.4173577456126748
Dbal_{101,17}, 0.5907498010105263, 0.44073726057324736
Dbal_{101,18}, 0.6668745025263159, 0.479480457324151
Dbal_{101,19}, 0.7094527249263158, 0

Dbal_{115,5}, 1.871919445108772, 1.221100938109061
Dbal_{115,6}, 2.1548714305578947, 1.3583473503311805
Dbal_{115,7}, 2.3518016842526315, 1.4171672419191064
Dbal_{115,8}, 2.343686426414035, 1.3622686770302586
Dbal_{115,9}, 2.144592103814035, 1.306585846356683
Dbal_{115,10}, 2.0028456016421052, 1.2795286962484826
Dbal_{115,11}, 1.879493686277193, 1.2552164747184087
Dbal_{115,12}, 1.8069973823999999, 1.2465895577600863
Dbal_{115,13}, 1.7566827853578946, 1.2375705073532595
Dbal_{115,14}, 1.710155307231579, 1.2254143971443627
Dbal_{115,15}, 1.6858095359403507, 1.2226694685662363
Dbal_{115,16}, 1.6809403803473684, 1.225022264808139
Dbal_{115,17}, 1.733960064077193, 1.2936454703630587
Dbal_{115,18}, 1.9574001601929825, 1.4073639268398321
Dbal_{115,19}, 2.0823751277929823, 1.3971684783120537
Dbal_{115,20}, 2.037470703014035, 1.36030801423686
Dbal_{115,21}, 2.0293554440631576, 1.3081543768138582
Dbal_{115,22}, 1.881657755182456, 1.217571744858487
Dbal_{115,23}, 1.7139424283719298, 1.1226756538

In [23]:
# change ptdf constraint (remember to run load changes FIRST)
for ts in timesteps
    for br in branches
        ptdfcon = get_ptdf_con(submodel,br,ts)
        
        valold = JuMP.constraint_object(ptdfcon).set.value
        valnew = 0.0
        for bus in buses
            buscon = get_load_balance(submodel,bus,ts)
            
            loadcon = copy(JuMP.constraint_object(buscon).func)
            loadval = copy(JuMP.constraint_object(buscon).set.value)

            valnew -= ptdfdict[br][bus]*loadval

        end 

        JuMP.set_normalized_rhs(ptdfcon, valnew)
        println("$(JuMP.name(ptdfcon)), $(valold), $(valnew)")
        
    end
end


PTDF_{A1,1}, -0.0844087470228698, -0.05473998601458146
PTDF_{A2,1}, -0.029270684761423808, -0.018982356490104314
PTDF_{A3,1}, -0.45366156006768504, -0.2942044263673138
PTDF_{A4,1}, -0.3309481669262001, -0.21462346685525954
PTDF_{A5,1}, -0.2630168422008444, -0.1705692672104523
PTDF_{A6,1}, -0.9401927183051911, -0.6097251438959558
PTDF_{A7,1}, -0.03464628962058087, -0.02246849625958435
PTDF_{A8,1}, -0.7196818111507614, -0.46672143541970057
PTDF_{A9,1}, -0.8266357322290885, -0.5360822125275951
PTDF_{A10,1}, -0.9774462423973358, -0.6338844553043383
PTDF_{A11,1}, -0.6566446692982456, -0.4258411665269429
PTDF_{A12-1,1}, -0.7979335638348837, -0.5174685431077118
PTDF_{A13-2,1}, -0.757001013063362, -0.4909233408592313
PTDF_{A14,1}, -1.448522977044826, -0.9393828136703034
PTDF_{A15,1}, -1.9285876574030425, -1.2507099484589592
PTDF_{A16,1}, -1.5491447214089016, -1.0046371038245137
PTDF_{A17,1}, -2.0363039524453863, -1.32056513142126
PTDF_{A18,1}, -5.538422436127683, -3.5917268314320165
PTDF_{A19,

PTDF_{A28,5}, -2.7572896153603335, -1.7986505460344395
PTDF_{A29,5}, 1.4245548514291568, 0.9292735700139854
PTDF_{A30,5}, 0.07456511859954833, 0.04864073432340184
PTDF_{A31-1,5}, -0.2709231648178767, -0.17673011022735124
PTDF_{A31-2,5}, -0.2709231648178767, -0.17673011022735124
PTDF_{A32-1,5}, -1.9130571432089962, -1.2479361107895441
PTDF_{A32-2,5}, -1.9130571432089962, -1.2479361107895441
PTDF_{A33-1,5}, -2.29098409742654, -1.4944675324968422
PTDF_{A33-2,5}, -2.29098409742654, -1.4944675324968422
PTDF_{A34,5}, -0.07456511859954833, -0.04864073432340184
PTDF_{A1,6}, -0.10922678347563385, -0.06885232807488169
PTDF_{A2,6}, -0.03787691275352472, -0.023876137570493262
PTDF_{A3,6}, -0.5870480813949955, -0.37005233578117414
PTDF_{A4,6}, -0.42825423958386516, -0.26995486167187716
PTDF_{A5,6}, -0.3403496045940933, -0.2145431886361439
PTDF_{A6,6}, -1.2166301490041613, -0.7669164480941967
PTDF_{A7,6}, -0.04483306420359458, -0.028261027969610157
PTDF_{A8,6}, -0.9312841634364968, -0.58704540740176

PTDF_{A33-2,10}, -2.4512205559658202, -1.5659754519577518
PTDF_{A34,10}, -0.07978036673172623, -0.05096811691076807
PTDF_{A1,11}, -0.09526835197850772, -0.06362479854746805
PTDF_{A2,11}, -0.0330365038797191, -0.022063370774654503
PTDF_{A3,11}, -0.5120273752190294, -0.3419565608658999
PTDF_{A4,11}, -0.37352629396127207, -0.24945886611274617
PTDF_{A5,11}, -0.29685526657890826, -0.19825425716702672
PTDF_{A6,11}, -1.0611531858287202, -0.7086892466390551
PTDF_{A7,11}, -0.03910370702965852, -0.026115343686784948
PTDF_{A8,11}, -0.8122724541647807, -0.5424746966222006
PTDF_{A9,11}, -0.9329865289278013, -0.6230933776246048
PTDF_{A10,11}, -1.1031995610069785, -0.7367698406127993
PTDF_{A11,11}, -0.7411252706140349, -0.4949591771228862
PTDF_{A12-1,11}, -0.9005916838724108, -0.6014585352855186
PTDF_{A13-2,11}, -0.854392956941624, -0.5706047980373676
PTDF_{A14,11}, -1.6348826595977397, -1.0918534444419836
PTDF_{A15,11}, -2.176710047799875, -1.4537118897110817
PTDF_{A16,11}, -1.7484498916308238, -1.1

PTDF_{A1,16}, -0.08520402115689879, -0.062094305153273334
PTDF_{A2,16}, -0.0295464644560302, -0.02153263678421077
PTDF_{A3,16}, -0.4579358244899172, -0.33373080189369153
PTDF_{A4,16}, -0.3340662622201557, -0.2434581375379592
PTDF_{A5,16}, -0.26549490873771214, -0.1934852545471011
PTDF_{A6,16}, -0.9490509347370485, -0.6916417394519218
PTDF_{A7,16}, -0.034972716666913706, -0.02548713956021406
PTDF_{A8,16}, -0.7264624393359452, -0.5294254774709817
PTDF_{A9,16}, -0.8344240484793908, -0.608104877539855
PTDF_{A10,16}, -0.9866554504640278, -0.7190468552384959
PTDF_{A11,16}, -0.6628313802631578, -0.48305294294569734
PTDF_{A12-1,16}, -0.8054514567828679, -0.5869904609473386
PTDF_{A13-2,16}, -0.7641332516802899, -0.5568789097983589
PTDF_{A14,16}, -1.4621705301340604, -1.065588929311135
PTDF_{A15,16}, -1.9467582372687113, -1.4187428761336922
PTDF_{A16,16}, -1.563740302675736, -1.139610135752737
PTDF_{A17,16}, -2.0554894032371385, -1.4979831055017343
PTDF_{A18,16}, -5.590603806686171, -4.074275463

PTDF_{A13-2,20}, -0.9262072181142471, -0.6183780211348532
PTDF_{A14,20}, -1.7722993943086829, -1.183267604243123
PTDF_{A15,20}, -2.359668980922792, -1.575422226997953
PTDF_{A16,20}, -1.8954122889032534, -1.2654633677313594
PTDF_{A17,20}, -2.4914622127085893, -1.6634133779801732
PTDF_{A18,20}, -6.776380412687756, -4.524219456729304
PTDF_{A19,20}, 3.108668729951443, 2.07548848507237
PTDF_{A20,20}, -5.717646734985713, -3.81736075066244
PTDF_{A21,20}, 0.8665155404159391, 0.5785251450578001
PTDF_{A22,20}, 4.12067874284173, 2.7511523569430607
PTDF_{A23,20}, 1.8617623691163556, 1.2429971491802199
PTDF_{A24,20}, -2.588468428492209, -1.728179133004968
PTDF_{A25-1,20}, 0.25430362168958176, 0.1697846516363566
PTDF_{A25-2,20}, 0.25430362168958176, 0.1697846516363566
PTDF_{A26,20}, 0.04239048954188493, 0.028301820464595943
PTDF_{A27,20}, 1.6317011028351902, 1.0893977930498604
PTDF_{A28,20}, -3.0011423972868005, -2.0036990196494524
PTDF_{A29,20}, 1.5505414948316663, 1.0352119511643687
PTDF_{A30,20},

PTDF_{A10,24}, -0.9466430310547943, -0.6292810823984469
PTDF_{A11,24}, -0.6359511890350876, -0.4227486381145494
PTDF_{A12-1,24}, -0.7727875096193294, -0.5137106016547234
PTDF_{A13-2,24}, -0.7331449060157583, -0.48735817501982615
PTDF_{A14,24}, -1.402874267723479, -0.9325608615676388
PTDF_{A15,24}, -1.867810204253395, -1.2416270876289037
PTDF_{A16,24}, -1.5003250214766533, -0.9973412643615023
PTDF_{A17,24}, -1.9721319312291752, -1.3109749707924894
PTDF_{A18,24}, -5.363884758857689, -3.565643121943922
PTDF_{A19,24}, 2.4606854700340413, 1.6357409962650482
PTDF_{A20,24}, -4.525837734980104, -3.0085512506102594
PTDF_{A21,24}, 0.6858955987547444, 0.4559491916950965
PTDF_{A22,24}, 3.2617480954129077, 2.1682474894897146
PTDF_{A23,24}, 1.4736892246515845, 0.9796351123016573
PTDF_{A24,24}, -2.048917786017076, -1.3620183764341363
PTDF_{A25-1,24}, 0.20129556450949393, 0.1338112532137147
PTDF_{A25-2,24}, 0.20129556450949393, 0.1338112532137147
PTDF_{A26,24}, 0.033554447496557775, 0.0223053263536935

In [24]:
#adjust wind ub values (remember to change wind)

bus = 122
for ts in timesteps
    con = get_wind_ub(submodel, bus, ts)
    oldval = JuMP.constraint_object(con).set.upper
    wval = wind[scen][ts]
    JuMP.set_normalized_rhs(con, wval)
    newval = JuMP.constraint_object(con).set.upper
    println("$(name(con)), $(oldval), $(newval)")
end


Wub_{122,1}, 2.784, 6.3
Wub_{122,2}, 3.1239999999999997, 6.723
Wub_{122,3}, 4.065, 6.6129999999999995
Wub_{122,4}, 3.9850000000000003, 6.54
Wub_{122,5}, 1.6689999999999998, 6.544
Wub_{122,6}, 1.339, 6.664
Wub_{122,7}, 2.798, 6.867000000000001
Wub_{122,8}, 3.43, 6.893
Wub_{122,9}, 3.969, 6.838
Wub_{122,10}, 3.761, 6.986000000000001
Wub_{122,11}, 2.5660000000000003, 7.135
Wub_{122,12}, 2.167, 7.103
Wub_{122,13}, 2.091, 7.022
Wub_{122,14}, 2.322, 6.992000000000001
Wub_{122,15}, 1.671, 6.869
Wub_{122,16}, 1.262, 6.954
Wub_{122,17}, 2.1059999999999994, 7.122999999999999
Wub_{122,18}, 2.434, 7.124
Wub_{122,19}, 1.956, 7.122000000000001
Wub_{122,20}, 1.641, 7.112000000000001
Wub_{122,21}, 2.356, 7.119
Wub_{122,22}, 2.154, 6.9670000000000005
Wub_{122,23}, 2.3569999999999998, 6.897
Wub_{122,24}, 3.125, 6.666


#### add all vars and constraints to extensive form

In [25]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 48
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [26]:
# add thermal vars to extensive form
for ts in timesteps
    
    for gen in gens
        
        name = JuMP.name(get_thermal_variable(submodel, gen, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end

exmodel

A JuMP Model
Minimization problem with:
Variables: 624
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [27]:
#add branch vars to extensive form

for ts in timesteps
    
    for br in branches
        name = JuMP.name(get_line_variable(submodel, br, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end  

exmodel

A JuMP Model
Minimization problem with:
Variables: 1536
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [28]:
#add wind vars to extensive form

for ts in timesteps
    
    bus = 122
    name = JuMP.name(get_wind_variable(submodel, bus, ts))

    JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
end  

exmodel

A JuMP Model
Minimization problem with:
Variables: 1560
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [29]:
# add charging vars to extensive form

for ts in timesteps
    
    for bus in buses
        
        name = JuMP.name(get_charging_variable(submodel, bus, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end

exmodel

A JuMP Model
Minimization problem with:
Variables: 2136
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [30]:
# add discharging vars to extensive form

for ts in timesteps
    
    for bus in buses
        
        name = JuMP.name(get_discharging_variable(submodel, bus, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end

exmodel

A JuMP Model
Minimization problem with:
Variables: 2712
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [31]:
# add storage vars to extensive form

for ts in timesteps
    
    for bus in buses
        
        name = JuMP.name(get_stored_variable(submodel, bus, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end

exmodel

A JuMP Model
Minimization problem with:
Variables: 3288
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [32]:
# add lossofload vars to extensive form

for ts in timesteps
    
    for bus in buses
        
        name = JuMP.name(get_lossofload_variable(submodel, bus, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end

exmodel

A JuMP Model
Minimization problem with:
Variables: 3864
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [33]:
# add overload vars to extensive form

for ts in timesteps
    
    for bus in buses
        
        name = JuMP.name(get_overload_variable(submodel, bus, ts))
        
        JuMP.@variable(exmodel, base_name = string(name, "_$(scen)"))
    end
end

exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [34]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [35]:
submodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2329 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [36]:
# add storage balance constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_storage_balance(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.value
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar == value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

Sbal_{101,1}_2 : -0.92195 Pch_{101_STORAGE_1,1}_2 + 1.084657519388253 Pdis_{101_STORAGE_1,1}_2 + Est_{101_STORAGE_1,1}_2 - Est_{101_STORAGE_1,24}_2 == 0.0
Sbal_{102,1}_2 : -0.92195 Pch_{102_STORAGE_1,1}_2 + 1.084657519388253 Pdis_{102_STORAGE_1,1}_2 + Est_{102_STORAGE_1,1}_2 - Est_{102_STORAGE_1,24}_2 == 0.0
Sbal_{103,1}_2 : -0.92195 Pch_{103_STORAGE_1,1}_2 + 1.084657519388253 Pdis_{103_STORAGE_1,1}_2 + Est_{103_STORAGE_1,1}_2 - Est_{103_STORAGE_1,24}_2 == 0.0
Sbal_{104,1}_2 : -0.92195 Pch_{104_STORAGE_1,1}_2 + 1.084657519388253 Pdis_{104_STORAGE_1,1}_2 + Est_{104_STORAGE_1,1}_2 - Est_{104_STORAGE_1,24}_2 == 0.0
Sbal_{105,1}_2 : -0.92195 Pch_{105_STORAGE_1,1}_2 + 1.084657519388253 Pdis_{105_STORAGE_1,1}_2 + Est_{105_STORAGE_1,1}_2 - Est_{105_STORAGE_1,24}_2 == 0.0
Sbal_{106,1}_2 : -0.92195 Pch_{106_STORAGE_1,1}_2 + 1.084657519388253 Pdis_{106_STORAGE_1,1}_2 + Est_{106_STORAGE_1,1}_2 - Est_{106_STORAGE_1,24}_2 == 0.0
Sbal_{107,1}_2 : -0.92195 Pch_{107_STORAGE_1,1}_2 + 1.084657519388253 

Sbal_{120,3}_2 : -Est_{120_STORAGE_1,2}_2 - 0.92195 Pch_{120_STORAGE_1,3}_2 + 1.084657519388253 Pdis_{120_STORAGE_1,3}_2 + Est_{120_STORAGE_1,3}_2 == 0.0
Sbal_{121,3}_2 : -Est_{121_STORAGE_1,2}_2 - 0.92195 Pch_{121_STORAGE_1,3}_2 + 1.084657519388253 Pdis_{121_STORAGE_1,3}_2 + Est_{121_STORAGE_1,3}_2 == 0.0
Sbal_{122,3}_2 : -Est_{122_STORAGE_1,2}_2 - 0.92195 Pch_{122_STORAGE_1,3}_2 + 1.084657519388253 Pdis_{122_STORAGE_1,3}_2 + Est_{122_STORAGE_1,3}_2 == 0.0
Sbal_{123,3}_2 : -Est_{123_STORAGE_1,2}_2 - 0.92195 Pch_{123_STORAGE_1,3}_2 + 1.084657519388253 Pdis_{123_STORAGE_1,3}_2 + Est_{123_STORAGE_1,3}_2 == 0.0
Sbal_{124,3}_2 : -Est_{124_STORAGE_1,2}_2 - 0.92195 Pch_{124_STORAGE_1,3}_2 + 1.084657519388253 Pdis_{124_STORAGE_1,3}_2 + Est_{124_STORAGE_1,3}_2 == 0.0
Sbal_{101,4}_2 : -Est_{101_STORAGE_1,3}_2 - 0.92195 Pch_{101_STORAGE_1,4}_2 + 1.084657519388253 Pdis_{101_STORAGE_1,4}_2 + Est_{101_STORAGE_1,4}_2 == 0.0
Sbal_{102,4}_2 : -Est_{102_STORAGE_1,3}_2 - 0.92195 Pch_{102_STORAGE_1,4}_2 

Sbal_{115,6}_2 : -Est_{115_STORAGE_1,5}_2 - 0.92195 Pch_{115_STORAGE_1,6}_2 + 1.084657519388253 Pdis_{115_STORAGE_1,6}_2 + Est_{115_STORAGE_1,6}_2 == 0.0
Sbal_{116,6}_2 : -Est_{116_STORAGE_1,5}_2 - 0.92195 Pch_{116_STORAGE_1,6}_2 + 1.084657519388253 Pdis_{116_STORAGE_1,6}_2 + Est_{116_STORAGE_1,6}_2 == 0.0
Sbal_{117,6}_2 : -Est_{117_STORAGE_1,5}_2 - 0.92195 Pch_{117_STORAGE_1,6}_2 + 1.084657519388253 Pdis_{117_STORAGE_1,6}_2 + Est_{117_STORAGE_1,6}_2 == 0.0
Sbal_{118,6}_2 : -Est_{118_STORAGE_1,5}_2 - 0.92195 Pch_{118_STORAGE_1,6}_2 + 1.084657519388253 Pdis_{118_STORAGE_1,6}_2 + Est_{118_STORAGE_1,6}_2 == 0.0
Sbal_{119,6}_2 : -Est_{119_STORAGE_1,5}_2 - 0.92195 Pch_{119_STORAGE_1,6}_2 + 1.084657519388253 Pdis_{119_STORAGE_1,6}_2 + Est_{119_STORAGE_1,6}_2 == 0.0
Sbal_{120,6}_2 : -Est_{120_STORAGE_1,5}_2 - 0.92195 Pch_{120_STORAGE_1,6}_2 + 1.084657519388253 Pdis_{120_STORAGE_1,6}_2 + Est_{120_STORAGE_1,6}_2 == 0.0
Sbal_{121,6}_2 : -Est_{121_STORAGE_1,5}_2 - 0.92195 Pch_{121_STORAGE_1,6}_2 

Sbal_{118,9}_2 : -Est_{118_STORAGE_1,8}_2 - 0.92195 Pch_{118_STORAGE_1,9}_2 + 1.084657519388253 Pdis_{118_STORAGE_1,9}_2 + Est_{118_STORAGE_1,9}_2 == 0.0
Sbal_{119,9}_2 : -Est_{119_STORAGE_1,8}_2 - 0.92195 Pch_{119_STORAGE_1,9}_2 + 1.084657519388253 Pdis_{119_STORAGE_1,9}_2 + Est_{119_STORAGE_1,9}_2 == 0.0
Sbal_{120,9}_2 : -Est_{120_STORAGE_1,8}_2 - 0.92195 Pch_{120_STORAGE_1,9}_2 + 1.084657519388253 Pdis_{120_STORAGE_1,9}_2 + Est_{120_STORAGE_1,9}_2 == 0.0
Sbal_{121,9}_2 : -Est_{121_STORAGE_1,8}_2 - 0.92195 Pch_{121_STORAGE_1,9}_2 + 1.084657519388253 Pdis_{121_STORAGE_1,9}_2 + Est_{121_STORAGE_1,9}_2 == 0.0
Sbal_{122,9}_2 : -Est_{122_STORAGE_1,8}_2 - 0.92195 Pch_{122_STORAGE_1,9}_2 + 1.084657519388253 Pdis_{122_STORAGE_1,9}_2 + Est_{122_STORAGE_1,9}_2 == 0.0
Sbal_{123,9}_2 : -Est_{123_STORAGE_1,8}_2 - 0.92195 Pch_{123_STORAGE_1,9}_2 + 1.084657519388253 Pdis_{123_STORAGE_1,9}_2 + Est_{123_STORAGE_1,9}_2 == 0.0
Sbal_{124,9}_2 : -Est_{124_STORAGE_1,8}_2 - 0.92195 Pch_{124_STORAGE_1,9}_2 

Sbal_{109,14}_2 : -Est_{109_STORAGE_1,13}_2 - 0.92195 Pch_{109_STORAGE_1,14}_2 + 1.084657519388253 Pdis_{109_STORAGE_1,14}_2 + Est_{109_STORAGE_1,14}_2 == 0.0
Sbal_{110,14}_2 : -Est_{110_STORAGE_1,13}_2 - 0.92195 Pch_{110_STORAGE_1,14}_2 + 1.084657519388253 Pdis_{110_STORAGE_1,14}_2 + Est_{110_STORAGE_1,14}_2 == 0.0
Sbal_{111,14}_2 : -Est_{111_STORAGE_1,13}_2 - 0.92195 Pch_{111_STORAGE_1,14}_2 + 1.084657519388253 Pdis_{111_STORAGE_1,14}_2 + Est_{111_STORAGE_1,14}_2 == 0.0
Sbal_{112,14}_2 : -Est_{112_STORAGE_1,13}_2 - 0.92195 Pch_{112_STORAGE_1,14}_2 + 1.084657519388253 Pdis_{112_STORAGE_1,14}_2 + Est_{112_STORAGE_1,14}_2 == 0.0
Sbal_{113,14}_2 : -Est_{113_STORAGE_1,13}_2 - 0.92195 Pch_{113_STORAGE_1,14}_2 + 1.084657519388253 Pdis_{113_STORAGE_1,14}_2 + Est_{113_STORAGE_1,14}_2 == 0.0
Sbal_{114,14}_2 : -Est_{114_STORAGE_1,13}_2 - 0.92195 Pch_{114_STORAGE_1,14}_2 + 1.084657519388253 Pdis_{114_STORAGE_1,14}_2 + Est_{114_STORAGE_1,14}_2 == 0.0
Sbal_{115,14}_2 : -Est_{115_STORAGE_1,13}_2 - 

Sbal_{108,18}_2 : -Est_{108_STORAGE_1,17}_2 - 0.92195 Pch_{108_STORAGE_1,18}_2 + 1.084657519388253 Pdis_{108_STORAGE_1,18}_2 + Est_{108_STORAGE_1,18}_2 == 0.0
Sbal_{109,18}_2 : -Est_{109_STORAGE_1,17}_2 - 0.92195 Pch_{109_STORAGE_1,18}_2 + 1.084657519388253 Pdis_{109_STORAGE_1,18}_2 + Est_{109_STORAGE_1,18}_2 == 0.0
Sbal_{110,18}_2 : -Est_{110_STORAGE_1,17}_2 - 0.92195 Pch_{110_STORAGE_1,18}_2 + 1.084657519388253 Pdis_{110_STORAGE_1,18}_2 + Est_{110_STORAGE_1,18}_2 == 0.0
Sbal_{111,18}_2 : -Est_{111_STORAGE_1,17}_2 - 0.92195 Pch_{111_STORAGE_1,18}_2 + 1.084657519388253 Pdis_{111_STORAGE_1,18}_2 + Est_{111_STORAGE_1,18}_2 == 0.0
Sbal_{112,18}_2 : -Est_{112_STORAGE_1,17}_2 - 0.92195 Pch_{112_STORAGE_1,18}_2 + 1.084657519388253 Pdis_{112_STORAGE_1,18}_2 + Est_{112_STORAGE_1,18}_2 == 0.0
Sbal_{113,18}_2 : -Est_{113_STORAGE_1,17}_2 - 0.92195 Pch_{113_STORAGE_1,18}_2 + 1.084657519388253 Pdis_{113_STORAGE_1,18}_2 + Est_{113_STORAGE_1,18}_2 == 0.0
Sbal_{114,18}_2 : -Est_{114_STORAGE_1,17}_2 - 

Sbal_{104,22}_2 : -Est_{104_STORAGE_1,21}_2 - 0.92195 Pch_{104_STORAGE_1,22}_2 + 1.084657519388253 Pdis_{104_STORAGE_1,22}_2 + Est_{104_STORAGE_1,22}_2 == 0.0
Sbal_{105,22}_2 : -Est_{105_STORAGE_1,21}_2 - 0.92195 Pch_{105_STORAGE_1,22}_2 + 1.084657519388253 Pdis_{105_STORAGE_1,22}_2 + Est_{105_STORAGE_1,22}_2 == 0.0
Sbal_{106,22}_2 : -Est_{106_STORAGE_1,21}_2 - 0.92195 Pch_{106_STORAGE_1,22}_2 + 1.084657519388253 Pdis_{106_STORAGE_1,22}_2 + Est_{106_STORAGE_1,22}_2 == 0.0
Sbal_{107,22}_2 : -Est_{107_STORAGE_1,21}_2 - 0.92195 Pch_{107_STORAGE_1,22}_2 + 1.084657519388253 Pdis_{107_STORAGE_1,22}_2 + Est_{107_STORAGE_1,22}_2 == 0.0
Sbal_{108,22}_2 : -Est_{108_STORAGE_1,21}_2 - 0.92195 Pch_{108_STORAGE_1,22}_2 + 1.084657519388253 Pdis_{108_STORAGE_1,22}_2 + Est_{108_STORAGE_1,22}_2 == 0.0
Sbal_{109,22}_2 : -Est_{109_STORAGE_1,21}_2 - 0.92195 Pch_{109_STORAGE_1,22}_2 + 1.084657519388253 Pdis_{109_STORAGE_1,22}_2 + Est_{109_STORAGE_1,22}_2 == 0.0
Sbal_{110,22}_2 : -Est_{110_STORAGE_1,21}_2 - 

Sbal_{121,24}_2 : -Est_{121_STORAGE_1,23}_2 - 0.92195 Pch_{121_STORAGE_1,24}_2 + 1.084657519388253 Pdis_{121_STORAGE_1,24}_2 + Est_{121_STORAGE_1,24}_2 == 0.0
Sbal_{122,24}_2 : -Est_{122_STORAGE_1,23}_2 - 0.92195 Pch_{122_STORAGE_1,24}_2 + 1.084657519388253 Pdis_{122_STORAGE_1,24}_2 + Est_{122_STORAGE_1,24}_2 == 0.0
Sbal_{123,24}_2 : -Est_{123_STORAGE_1,23}_2 - 0.92195 Pch_{123_STORAGE_1,24}_2 + 1.084657519388253 Pdis_{123_STORAGE_1,24}_2 + Est_{123_STORAGE_1,24}_2 == 0.0
Sbal_{124,24}_2 : -Est_{124_STORAGE_1,23}_2 - 0.92195 Pch_{124_STORAGE_1,24}_2 + 1.084657519388253 Pdis_{124_STORAGE_1,24}_2 + Est_{124_STORAGE_1,24}_2 == 0.0


In [37]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 576 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [38]:
# add load balance constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_load_balance(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.value
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar == value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

Dbal_{101,1}_2 : Pth_{101_STEAM_3,1}_2 + Pth_{101_STEAM_4,1}_2 + Pth_{101_CT_1,1}_2 + Pth_{101_CT_2,1}_2 - Fbr_{A3,1}_2 - Fbr_{A1,1}_2 - Fbr_{A2,1}_2 - Pch_{101_STORAGE_1,1}_2 + Pdis_{101_STORAGE_1,1}_2 + LOL_{101,1}_2 - OL_{101,1}_2 == 0.3679267704425019
Dbal_{102,1}_2 : Pth_{102_STEAM_3,1}_2 + Pth_{102_CT_2,1}_2 + Pth_{102_CT_1,1}_2 + Pth_{102_STEAM_4,1}_2 + Fbr_{A1,1}_2 - Fbr_{A5,1}_2 - Fbr_{A4,1}_2 - Pch_{102_STORAGE_1,1}_2 + Pdis_{102_STORAGE_1,1}_2 + LOL_{102,1}_2 - OL_{102,1}_2 == 0.33045274887555887
Dbal_{103,1}_2 : Fbr_{A2,1}_2 - Fbr_{A7,1}_2 - Fbr_{A6,1}_2 - Pch_{103_STORAGE_1,1}_2 + Pdis_{103_STORAGE_1,1}_2 + LOL_{103,1}_2 - OL_{103,1}_2 == 0.6132112840708365
Dbal_{104,1}_2 : Fbr_{A4,1}_2 - Fbr_{A8,1}_2 - Pch_{104_STORAGE_1,1}_2 + Pdis_{104_STORAGE_1,1}_2 + LOL_{104,1}_2 - OL_{104,1}_2 == 0.25209796856444106
Dbal_{105,1}_2 : Fbr_{A3,1}_2 - Fbr_{A9,1}_2 - Pch_{105_STORAGE_1,1}_2 + Pdis_{105_STORAGE_1,1}_2 + LOL_{105,1}_2 - OL_{105,1}_2 == 0.24187778616028136
Dbal_{106,1}_2 : 

Dbal_{121,3}_2 : Pth_{121_NUCLEAR_1,3}_2 + Fbr_{A25-1,3}_2 + Fbr_{A25-2,3}_2 + Fbr_{A31-1,3}_2 + Fbr_{A31-2,3}_2 - Fbr_{A34,3}_2 - Pch_{121_STORAGE_1,3}_2 + Pdis_{121_STORAGE_1,3}_2 + LOL_{121,3}_2 - OL_{121,3}_2 == 0.0
Dbal_{122,3}_2 : Pre_{122_WIND_1,3}_2 + Fbr_{A30,3}_2 + Fbr_{A34,3}_2 - Pch_{122_STORAGE_1,3}_2 + Pdis_{122_STORAGE_1,3}_2 + LOL_{122,3}_2 - OL_{122,3}_2 == 0.0
Dbal_{123,3}_2 : Pth_{123_STEAM_2,3}_2 + Pth_{123_CT_1,3}_2 + Pth_{123_STEAM_3,3}_2 + Pth_{123_CT_5,3}_2 + Pth_{123_CT_4,3}_2 + Fbr_{A21,3}_2 + Fbr_{A22,3}_2 + Fbr_{A33-2,3}_2 + Fbr_{A33-1,3}_2 - Pch_{123_STORAGE_1,3}_2 + Pdis_{123_STORAGE_1,3}_2 + LOL_{123,3}_2 - OL_{123,3}_2 == 0.0
Dbal_{124,3}_2 : Fbr_{A7,3}_2 + Fbr_{A26,3}_2 - Pch_{124_STORAGE_1,3}_2 + Pdis_{124_STORAGE_1,3}_2 + LOL_{124,3}_2 - OL_{124,3}_2 == 0.0
Dbal_{101,4}_2 : Pth_{101_STEAM_3,4}_2 + Pth_{101_STEAM_4,4}_2 + Pth_{101_CT_1,4}_2 + Pth_{101_CT_2,4}_2 - Fbr_{A3,4}_2 - Fbr_{A1,4}_2 - Fbr_{A2,4}_2 - Pch_{101_STORAGE_1,4}_2 + Pdis_{101_STORAGE_1

Dbal_{107,7}_2 : Pth_{107_CC_1,7}_2 - Fbr_{A11,7}_2 - Pch_{107_STORAGE_1,7}_2 + Pdis_{107_STORAGE_1,7}_2 + LOL_{107,7}_2 - OL_{107,7}_2 == 0.5588198896633745
Dbal_{108,7}_2 : Fbr_{A11,7}_2 - Fbr_{A12-1,7}_2 - Fbr_{A13-2,7}_2 - Pch_{108_STORAGE_1,7}_2 + Pdis_{108_STORAGE_1,7}_2 + LOL_{108,7}_2 - OL_{108,7}_2 == 0.7644656112
Dbal_{109,7}_2 : Fbr_{A6,7}_2 + Fbr_{A8,7}_2 + Fbr_{A12-1,7}_2 - Fbr_{A14,7}_2 - Fbr_{A15,7}_2 - Pch_{109_STORAGE_1,7}_2 + Pdis_{109_STORAGE_1,7}_2 + LOL_{109,7}_2 - OL_{109,7}_2 == 0.7823478506251617
Dbal_{110,7}_2 : Fbr_{A9,7}_2 + Fbr_{A10,7}_2 + Fbr_{A13-2,7}_2 - Fbr_{A17,7}_2 - Fbr_{A16,7}_2 - Pch_{110_STORAGE_1,7}_2 + Pdis_{110_STORAGE_1,7}_2 + LOL_{110,7}_2 - OL_{110,7}_2 == 0.8717590350098765
Dbal_{111,7}_2 : Fbr_{A14,7}_2 + Fbr_{A16,7}_2 - Fbr_{A18,7}_2 - Fbr_{A19,7}_2 - Pch_{111_STORAGE_1,7}_2 + Pdis_{111_STORAGE_1,7}_2 + LOL_{111,7}_2 - OL_{111,7}_2 == 0.0
Dbal_{112,7}_2 : Fbr_{A15,7}_2 + Fbr_{A17,7}_2 - Fbr_{A21,7}_2 - Fbr_{A20,7}_2 - Pch_{112_STORAGE_1,7}

Dbal_{115,10}_2 : Pth_{115_STEAM_3,10}_2 + Pth_{115_STEAM_2,10}_2 + Pth_{115_STEAM_1,10}_2 - Fbr_{A25-1,10}_2 - Fbr_{A26,10}_2 - Fbr_{A25-2,10}_2 - Fbr_{A24,10}_2 - Pch_{115_STORAGE_1,10}_2 + Pdis_{115_STORAGE_1,10}_2 + LOL_{115,10}_2 - OL_{115,10}_2 == 1.2795286962484826
Dbal_{116,10}_2 : Pth_{116_STEAM_1,10}_2 + Fbr_{A23,10}_2 + Fbr_{A24,10}_2 - Fbr_{A28,10}_2 - Fbr_{A27,10}_2 - Pch_{116_STORAGE_1,10}_2 + Pdis_{116_STORAGE_1,10}_2 + LOL_{116,10}_2 - OL_{116,10}_2 == 0.40363680990242046
Dbal_{117,10}_2 : Fbr_{A27,10}_2 - Fbr_{A29,10}_2 - Fbr_{A30,10}_2 - Pch_{117_STORAGE_1,10}_2 + Pdis_{117_STORAGE_1,10}_2 + LOL_{117,10}_2 - OL_{117,10}_2 == 0.0
Dbal_{118,10}_2 : Pth_{118_CC_1,10}_2 + Fbr_{A29,10}_2 - Fbr_{A31-1,10}_2 - Fbr_{A31-2,10}_2 - Pch_{118_STORAGE_1,10}_2 + Pdis_{118_STORAGE_1,10}_2 + LOL_{118,10}_2 - OL_{118,10}_2 == 1.344110585372724
Dbal_{119,10}_2 : Fbr_{A28,10}_2 - Fbr_{A32-2,10}_2 - Fbr_{A32-1,10}_2 - Pch_{119_STORAGE_1,10}_2 + Pdis_{119_STORAGE_1,10}_2 + LOL_{119,10}_2 

Dbal_{121,13}_2 : Pth_{121_NUCLEAR_1,13}_2 + Fbr_{A25-1,13}_2 + Fbr_{A25-2,13}_2 + Fbr_{A31-1,13}_2 + Fbr_{A31-2,13}_2 - Fbr_{A34,13}_2 - Pch_{121_STORAGE_1,13}_2 + Pdis_{121_STORAGE_1,13}_2 + LOL_{121,13}_2 - OL_{121,13}_2 == 0.0
Dbal_{122,13}_2 : Pre_{122_WIND_1,13}_2 + Fbr_{A30,13}_2 + Fbr_{A34,13}_2 - Pch_{122_STORAGE_1,13}_2 + Pdis_{122_STORAGE_1,13}_2 + LOL_{122,13}_2 - OL_{122,13}_2 == 0.0
Dbal_{123,13}_2 : Pth_{123_STEAM_2,13}_2 + Pth_{123_CT_1,13}_2 + Pth_{123_STEAM_3,13}_2 + Pth_{123_CT_5,13}_2 + Pth_{123_CT_4,13}_2 + Fbr_{A21,13}_2 + Fbr_{A22,13}_2 + Fbr_{A33-2,13}_2 + Fbr_{A33-1,13}_2 - Pch_{123_STORAGE_1,13}_2 + Pdis_{123_STORAGE_1,13}_2 + LOL_{123,13}_2 - OL_{123,13}_2 == 0.0
Dbal_{124,13}_2 : Fbr_{A7,13}_2 + Fbr_{A26,13}_2 - Pch_{124_STORAGE_1,13}_2 + Pdis_{124_STORAGE_1,13}_2 + LOL_{124,13}_2 - OL_{124,13}_2 == 0.0
Dbal_{101,14}_2 : Pth_{101_STEAM_3,14}_2 + Pth_{101_STEAM_4,14}_2 + Pth_{101_CT_1,14}_2 + Pth_{101_CT_2,14}_2 - Fbr_{A3,14}_2 - Fbr_{A1,14}_2 - Fbr_{A2,14}_2

Dbal_{102,17}_2 : Pth_{102_STEAM_3,17}_2 + Pth_{102_CT_2,17}_2 + Pth_{102_CT_1,17}_2 + Pth_{102_STEAM_4,17}_2 + Fbr_{A1,17}_2 - Fbr_{A5,17}_2 - Fbr_{A4,17}_2 - Pch_{102_STORAGE_1,17}_2 + Pdis_{102_STORAGE_1,17}_2 + LOL_{102,17}_2 - OL_{102,17}_2 == 0.3958473560191064
Dbal_{103,17}_2 : Fbr_{A2,17}_2 - Fbr_{A7,17}_2 - Fbr_{A6,17}_2 - Pch_{103_STORAGE_1,17}_2 + Pdis_{103_STORAGE_1,17}_2 + LOL_{103,17}_2 - OL_{103,17}_2 == 0.7345621009554124
Dbal_{104,17}_2 : Fbr_{A4,17}_2 - Fbr_{A8,17}_2 - Pch_{104_STORAGE_1,17}_2 + Pdis_{104_STORAGE_1,17}_2 + LOL_{104,17}_2 - OL_{104,17}_2 == 0.3019866369808936
Dbal_{105,17}_2 : Fbr_{A3,17}_2 - Fbr_{A9,17}_2 - Pch_{105_STORAGE_1,17}_2 + Pdis_{105_STORAGE_1,17}_2 + LOL_{105,17}_2 - OL_{105,17}_2 == 0.2897439420828006
Dbal_{106,17}_2 : Fbr_{A5,17}_2 - Fbr_{A10,17}_2 - Pch_{106_STORAGE_1,17}_2 + Pdis_{106_STORAGE_1,17}_2 + LOL_{106,17}_2 - OL_{106,17}_2 == 0.5550024711082819
Dbal_{107,17}_2 : Pth_{107_CC_1,17}_2 - Fbr_{A11,17}_2 - Pch_{107_STORAGE_1,17}_2 +

Dbal_{124,19}_2 : Fbr_{A7,19}_2 + Fbr_{A26,19}_2 - Pch_{124_STORAGE_1,19}_2 + Pdis_{124_STORAGE_1,19}_2 + LOL_{124,19}_2 - OL_{124,19}_2 == 0.0
Dbal_{101,20}_2 : Pth_{101_STEAM_3,20}_2 + Pth_{101_STEAM_4,20}_2 + Pth_{101_CT_1,20}_2 + Pth_{101_CT_2,20}_2 - Fbr_{A3,20}_2 - Fbr_{A1,20}_2 - Fbr_{A2,20}_2 - Pch_{101_STORAGE_1,20}_2 + Pdis_{101_STORAGE_1,20}_2 + LOL_{101,20}_2 - OL_{101,20}_2 == 0.46344878984682614
Dbal_{102,20}_2 : Pth_{102_STEAM_3,20}_2 + Pth_{102_CT_2,20}_2 + Pth_{102_CT_1,20}_2 + Pth_{102_STEAM_4,20}_2 + Fbr_{A1,20}_2 - Fbr_{A5,20}_2 - Fbr_{A4,20}_2 - Pch_{102_STORAGE_1,20}_2 + Pdis_{102_STORAGE_1,20}_2 + LOL_{102,20}_2 - OL_{102,20}_2 == 0.4162456740610242
Dbal_{103,20}_2 : Fbr_{A2,20}_2 - Fbr_{A7,20}_2 - Fbr_{A6,20}_2 - Pch_{103_STORAGE_1,20}_2 + Pdis_{103_STORAGE_1,20}_2 + LOL_{103,20}_2 - OL_{103,20}_2 == 0.7724146497447103
Dbal_{104,20}_2 : Fbr_{A4,20}_2 - Fbr_{A8,20}_2 - Pch_{104_STORAGE_1,20}_2 + Pdis_{104_STORAGE_1,20}_2 + LOL_{104,20}_2 - OL_{104,20}_2 == 0.3175

Dbal_{118,21}_2 : Pth_{118_CC_1,21}_2 + Fbr_{A29,21}_2 - Fbr_{A31-1,21}_2 - Fbr_{A31-2,21}_2 - Pch_{118_STORAGE_1,21}_2 + Pdis_{118_STORAGE_1,21}_2 + LOL_{118,21}_2 - OL_{118,21}_2 == 1.3741810952207871
Dbal_{119,21}_2 : Fbr_{A28,21}_2 - Fbr_{A32-2,21}_2 - Fbr_{A32-1,21}_2 - Pch_{119_STORAGE_1,21}_2 + Pdis_{119_STORAGE_1,21}_2 + LOL_{119,21}_2 - OL_{119,21}_2 == 0.7469272644744569
Dbal_{120,21}_2 : Fbr_{A32-2,21}_2 + Fbr_{A32-1,21}_2 - Fbr_{A33-2,21}_2 - Fbr_{A33-1,21}_2 - Pch_{120_STORAGE_1,21}_2 + Pdis_{120_STORAGE_1,21}_2 + LOL_{120,21}_2 - OL_{120,21}_2 == 0.5282137590164414
Dbal_{121,21}_2 : Pth_{121_NUCLEAR_1,21}_2 + Fbr_{A25-1,21}_2 + Fbr_{A25-2,21}_2 + Fbr_{A31-1,21}_2 + Fbr_{A31-2,21}_2 - Fbr_{A34,21}_2 - Pch_{121_STORAGE_1,21}_2 + Pdis_{121_STORAGE_1,21}_2 + LOL_{121,21}_2 - OL_{121,21}_2 == 0.0
Dbal_{122,21}_2 : Pre_{122_WIND_1,21}_2 + Fbr_{A30,21}_2 + Fbr_{A34,21}_2 - Pch_{122_STORAGE_1,21}_2 + Pdis_{122_STORAGE_1,21}_2 + LOL_{122,21}_2 - OL_{122,21}_2 == 0.0
Dbal_{123,21}_

Dbal_{122,23}_2 : Pre_{122_WIND_1,23}_2 + Fbr_{A30,23}_2 + Fbr_{A34,23}_2 - Pch_{122_STORAGE_1,23}_2 + Pdis_{122_STORAGE_1,23}_2 + LOL_{122,23}_2 - OL_{122,23}_2 == 0.0
Dbal_{123,23}_2 : Pth_{123_STEAM_2,23}_2 + Pth_{123_CT_1,23}_2 + Pth_{123_STEAM_3,23}_2 + Pth_{123_CT_5,23}_2 + Pth_{123_CT_4,23}_2 + Fbr_{A21,23}_2 + Fbr_{A22,23}_2 + Fbr_{A33-2,23}_2 + Fbr_{A33-1,23}_2 - Pch_{123_STORAGE_1,23}_2 + Pdis_{123_STORAGE_1,23}_2 + LOL_{123,23}_2 - OL_{123,23}_2 == 0.0
Dbal_{124,23}_2 : Fbr_{A7,23}_2 + Fbr_{A26,23}_2 - Pch_{124_STORAGE_1,23}_2 + Pdis_{124_STORAGE_1,23}_2 + LOL_{124,23}_2 - OL_{124,23}_2 == 0.0
Dbal_{101,24}_2 : Pth_{101_STEAM_3,24}_2 + Pth_{101_STEAM_4,24}_2 + Pth_{101_CT_1,24}_2 + Pth_{101_CT_2,24}_2 - Fbr_{A3,24}_2 - Fbr_{A1,24}_2 - Fbr_{A2,24}_2 - Pch_{101_STORAGE_1,24}_2 + Pdis_{101_STORAGE_1,24}_2 + LOL_{101,24}_2 - OL_{101,24}_2 == 0.3652548258755793
Dbal_{102,24}_2 : Pth_{102_STEAM_3,24}_2 + Pth_{102_CT_2,24}_2 + Pth_{102_CT_1,24}_2 + Pth_{102_STEAM_4,24}_2 + Fbr_{A1,

In [39]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1152 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [40]:
# add ptdf constraint

for ts in timesteps
    for br in branches
        ### change constraint here
        subcon = get_ptdf_con(submodel, br, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.value
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar == value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        #println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

In [41]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [42]:
# add wind lb constraint

for ts in timesteps
    bus = 122
    ### change constraint here
    subcon = get_wind_lb(submodel, bus, ts)
    ###
    subconname = JuMP.name(subcon)
    exconname = string(subconname, "_$(scen)")

    terms = JuMP.constraint_object(subcon).func.terms
    ### change sense here
    value = JuMP.constraint_object(subcon).set.lower
    ###

    count = 0
    for (subvar, coeff) in terms
        if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
            exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
        else
            exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
        end
        if count == 0
            ### change objective type here
            JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                        base_name = exconname)
            ###
            count += 1
        else
            excon = JuMP.constraint_by_name(exmodel, exconname)
            JuMP.set_normalized_coefficient(excon, exvar, coeff)
        end
    end
    println(JuMP.constraint_by_name(exmodel, exconname))
end

Wlb_{122,1}_2 : Pre_{122_WIND_1,1}_2 >= 0.0
Wlb_{122,2}_2 : Pre_{122_WIND_1,2}_2 >= 0.0
Wlb_{122,3}_2 : Pre_{122_WIND_1,3}_2 >= 0.0
Wlb_{122,4}_2 : Pre_{122_WIND_1,4}_2 >= 0.0
Wlb_{122,5}_2 : Pre_{122_WIND_1,5}_2 >= 0.0
Wlb_{122,6}_2 : Pre_{122_WIND_1,6}_2 >= 0.0
Wlb_{122,7}_2 : Pre_{122_WIND_1,7}_2 >= 0.0
Wlb_{122,8}_2 : Pre_{122_WIND_1,8}_2 >= 0.0
Wlb_{122,9}_2 : Pre_{122_WIND_1,9}_2 >= 0.0
Wlb_{122,10}_2 : Pre_{122_WIND_1,10}_2 >= 0.0
Wlb_{122,11}_2 : Pre_{122_WIND_1,11}_2 >= 0.0
Wlb_{122,12}_2 : Pre_{122_WIND_1,12}_2 >= 0.0
Wlb_{122,13}_2 : Pre_{122_WIND_1,13}_2 >= 0.0
Wlb_{122,14}_2 : Pre_{122_WIND_1,14}_2 >= 0.0
Wlb_{122,15}_2 : Pre_{122_WIND_1,15}_2 >= 0.0
Wlb_{122,16}_2 : Pre_{122_WIND_1,16}_2 >= 0.0
Wlb_{122,17}_2 : Pre_{122_WIND_1,17}_2 >= 0.0
Wlb_{122,18}_2 : Pre_{122_WIND_1,18}_2 >= 0.0
Wlb_{122,19}_2 : Pre_{122_WIND_1,19}_2 >= 0.0
Wlb_{122,20}_2 : Pre_{122_WIND_1,20}_2 >= 0.0
Wlb_{122,21}_2 : Pre_{122_WIND_1,21}_2 >= 0.0
Wlb_{122,22}_2 : Pre_{122_WIND_1,22}_2 >= 0.0
Wlb_{1

In [43]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 72 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [44]:
# add ramping lb constraint

for ts in timesteps
    if ts > 1
        for gen in gens
            ### change constraint here
            subcon = get_ramp_lb(submodel, gen, ts)
            ###
            subconname = JuMP.name(subcon)
            exconname = string(subconname, "_$(scen)")

            terms = JuMP.constraint_object(subcon).func.terms
            ### change sense here
            value = JuMP.constraint_object(subcon).set.lower
            ###

            count = 0
            for (subvar, coeff) in terms
                if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                    exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
                else
                    exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
                end
                if count == 0
                    ### change objective type here
                    JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                                base_name = exconname)
                    ###
                    count += 1
                else
                    excon = JuMP.constraint_by_name(exmodel, exconname)
                    JuMP.set_normalized_coefficient(excon, exvar, coeff)
                end
            end
            println(JuMP.constraint_by_name(exmodel, exconname))
        end
    end
end

Rlb_{101_CT_1,2}_2 : -Pth_{101_CT_1,1}_2 + Pth_{101_CT_1,2}_2 >= -1.7999999999999998
Rlb_{101_CT_2,2}_2 : -Pth_{101_CT_2,1}_2 + Pth_{101_CT_2,2}_2 >= -1.7999999999999998
Rlb_{101_STEAM_3,2}_2 : -Pth_{101_STEAM_3,1}_2 + Pth_{101_STEAM_3,2}_2 >= -1.2
Rlb_{101_STEAM_4,2}_2 : -Pth_{101_STEAM_4,1}_2 + Pth_{101_STEAM_4,2}_2 >= -1.2
Rlb_{102_CT_1,2}_2 : -Pth_{102_CT_1,1}_2 + Pth_{102_CT_1,2}_2 >= -1.7999999999999998
Rlb_{102_CT_2,2}_2 : -Pth_{102_CT_2,1}_2 + Pth_{102_CT_2,2}_2 >= -1.7999999999999998
Rlb_{102_STEAM_3,2}_2 : -Pth_{102_STEAM_3,1}_2 + Pth_{102_STEAM_3,2}_2 >= -1.2
Rlb_{102_STEAM_4,2}_2 : -Pth_{102_STEAM_4,1}_2 + Pth_{102_STEAM_4,2}_2 >= -1.2
Rlb_{107_CC_1,2}_2 : -Pth_{107_CC_1,1}_2 + Pth_{107_CC_1,2}_2 >= -2.484
Rlb_{113_CT_1,2}_2 : -Pth_{113_CT_1,1}_2 + Pth_{113_CT_1,2}_2 >= -2.22
Rlb_{113_CT_2,2}_2 : -Pth_{113_CT_2,1}_2 + Pth_{113_CT_2,2}_2 >= -2.22
Rlb_{113_CT_3,2}_2 : -Pth_{113_CT_3,1}_2 + Pth_{113_CT_3,2}_2 >= -2.22
Rlb_{113_CT_4,2}_2 : -Pth_{113_CT_4,1}_2 + Pth_{113_CT_4,2}

Rlb_{113_CT_1,9}_2 : -Pth_{113_CT_1,8}_2 + Pth_{113_CT_1,9}_2 >= -2.22
Rlb_{113_CT_2,9}_2 : -Pth_{113_CT_2,8}_2 + Pth_{113_CT_2,9}_2 >= -2.22
Rlb_{113_CT_3,9}_2 : -Pth_{113_CT_3,8}_2 + Pth_{113_CT_3,9}_2 >= -2.22
Rlb_{113_CT_4,9}_2 : -Pth_{113_CT_4,8}_2 + Pth_{113_CT_4,9}_2 >= -2.22
Rlb_{115_STEAM_1,9}_2 : -Pth_{115_STEAM_1,8}_2 + Pth_{115_STEAM_1,9}_2 >= -0.6
Rlb_{115_STEAM_2,9}_2 : -Pth_{115_STEAM_2,8}_2 + Pth_{115_STEAM_2,9}_2 >= -0.6
Rlb_{115_STEAM_3,9}_2 : -Pth_{115_STEAM_3,8}_2 + Pth_{115_STEAM_3,9}_2 >= -1.7999999999999998
Rlb_{116_STEAM_1,9}_2 : -Pth_{116_STEAM_1,8}_2 + Pth_{116_STEAM_1,9}_2 >= -1.7999999999999998
Rlb_{118_CC_1,9}_2 : -Pth_{118_CC_1,8}_2 + Pth_{118_CC_1,9}_2 >= -2.484
Rlb_{123_STEAM_2,9}_2 : -Pth_{123_STEAM_2,8}_2 + Pth_{123_STEAM_2,9}_2 >= -1.7999999999999998
Rlb_{123_STEAM_3,9}_2 : -Pth_{123_STEAM_3,8}_2 + Pth_{123_STEAM_3,9}_2 >= -2.4
Rlb_{123_CT_1,9}_2 : -Pth_{123_CT_1,8}_2 + Pth_{123_CT_1,9}_2 >= -2.22
Rlb_{123_CT_4,9}_2 : -Pth_{123_CT_4,8}_2 + Pth_{123_CT

Rlb_{113_CT_3,17}_2 : -Pth_{113_CT_3,16}_2 + Pth_{113_CT_3,17}_2 >= -2.22
Rlb_{113_CT_4,17}_2 : -Pth_{113_CT_4,16}_2 + Pth_{113_CT_4,17}_2 >= -2.22
Rlb_{115_STEAM_1,17}_2 : -Pth_{115_STEAM_1,16}_2 + Pth_{115_STEAM_1,17}_2 >= -0.6
Rlb_{115_STEAM_2,17}_2 : -Pth_{115_STEAM_2,16}_2 + Pth_{115_STEAM_2,17}_2 >= -0.6
Rlb_{115_STEAM_3,17}_2 : -Pth_{115_STEAM_3,16}_2 + Pth_{115_STEAM_3,17}_2 >= -1.7999999999999998
Rlb_{116_STEAM_1,17}_2 : -Pth_{116_STEAM_1,16}_2 + Pth_{116_STEAM_1,17}_2 >= -1.7999999999999998
Rlb_{118_CC_1,17}_2 : -Pth_{118_CC_1,16}_2 + Pth_{118_CC_1,17}_2 >= -2.484
Rlb_{123_STEAM_2,17}_2 : -Pth_{123_STEAM_2,16}_2 + Pth_{123_STEAM_2,17}_2 >= -1.7999999999999998
Rlb_{123_STEAM_3,17}_2 : -Pth_{123_STEAM_3,16}_2 + Pth_{123_STEAM_3,17}_2 >= -2.4
Rlb_{123_CT_1,17}_2 : -Pth_{123_CT_1,16}_2 + Pth_{123_CT_1,17}_2 >= -2.22
Rlb_{123_CT_4,17}_2 : -Pth_{123_CT_4,16}_2 + Pth_{123_CT_4,17}_2 >= -2.22
Rlb_{123_CT_5,17}_2 : -Pth_{123_CT_5,16}_2 + Pth_{123_CT_5,17}_2 >= -2.22
Rlb_{121_NUCLEAR_1

Rlb_{107_CC_1,23}_2 : -Pth_{107_CC_1,22}_2 + Pth_{107_CC_1,23}_2 >= -2.484
Rlb_{113_CT_1,23}_2 : -Pth_{113_CT_1,22}_2 + Pth_{113_CT_1,23}_2 >= -2.22
Rlb_{113_CT_2,23}_2 : -Pth_{113_CT_2,22}_2 + Pth_{113_CT_2,23}_2 >= -2.22
Rlb_{113_CT_3,23}_2 : -Pth_{113_CT_3,22}_2 + Pth_{113_CT_3,23}_2 >= -2.22
Rlb_{113_CT_4,23}_2 : -Pth_{113_CT_4,22}_2 + Pth_{113_CT_4,23}_2 >= -2.22
Rlb_{115_STEAM_1,23}_2 : -Pth_{115_STEAM_1,22}_2 + Pth_{115_STEAM_1,23}_2 >= -0.6
Rlb_{115_STEAM_2,23}_2 : -Pth_{115_STEAM_2,22}_2 + Pth_{115_STEAM_2,23}_2 >= -0.6
Rlb_{115_STEAM_3,23}_2 : -Pth_{115_STEAM_3,22}_2 + Pth_{115_STEAM_3,23}_2 >= -1.7999999999999998
Rlb_{116_STEAM_1,23}_2 : -Pth_{116_STEAM_1,22}_2 + Pth_{116_STEAM_1,23}_2 >= -1.7999999999999998
Rlb_{118_CC_1,23}_2 : -Pth_{118_CC_1,22}_2 + Pth_{118_CC_1,23}_2 >= -2.484
Rlb_{123_STEAM_2,23}_2 : -Pth_{123_STEAM_2,22}_2 + Pth_{123_STEAM_2,23}_2 >= -1.7999999999999998
Rlb_{123_STEAM_3,23}_2 : -Pth_{123_STEAM_3,22}_2 + Pth_{123_STEAM_3,23}_2 >= -2.4
Rlb_{123_CT_1,23}

In [45]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 624 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [46]:
# add charging lb constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_charge_lb(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

Clb_{101,1}_2 : Pch_{101_STORAGE_1,1}_2 >= 0.0
Clb_{102,1}_2 : Pch_{102_STORAGE_1,1}_2 >= 0.0
Clb_{103,1}_2 : Pch_{103_STORAGE_1,1}_2 >= 0.0
Clb_{104,1}_2 : Pch_{104_STORAGE_1,1}_2 >= 0.0
Clb_{105,1}_2 : Pch_{105_STORAGE_1,1}_2 >= 0.0
Clb_{106,1}_2 : Pch_{106_STORAGE_1,1}_2 >= 0.0
Clb_{107,1}_2 : Pch_{107_STORAGE_1,1}_2 >= 0.0
Clb_{108,1}_2 : Pch_{108_STORAGE_1,1}_2 >= 0.0
Clb_{109,1}_2 : Pch_{109_STORAGE_1,1}_2 >= 0.0
Clb_{110,1}_2 : Pch_{110_STORAGE_1,1}_2 >= 0.0
Clb_{111,1}_2 : Pch_{111_STORAGE_1,1}_2 >= 0.0
Clb_{112,1}_2 : Pch_{112_STORAGE_1,1}_2 >= 0.0
Clb_{113,1}_2 : Pch_{113_STORAGE_1,1}_2 >= 0.0
Clb_{114,1}_2 : Pch_{114_STORAGE_1,1}_2 >= 0.0
Clb_{115,1}_2 : Pch_{115_STORAGE_1,1}_2 >= 0.0
Clb_{116,1}_2 : Pch_{116_STORAGE_1,1}_2 >= 0.0
Clb_{117,1}_2 : Pch_{117_STORAGE_1,1}_2 >= 0.0
Clb_{118,1}_2 : Pch_{118_STORAGE_1,1}_2 >= 0.0
Clb_{119,1}_2 : Pch_{119_STORAGE_1,1}_2 >= 0.0
Clb_{120,1}_2 : Pch_{120_STORAGE_1,1}_2 >= 0.0
Clb_{121,1}_2 : Pch_{121_STORAGE_1,1}_2 >= 0.0
Clb_{122,1}_2

Clb_{103,10}_2 : Pch_{103_STORAGE_1,10}_2 >= 0.0
Clb_{104,10}_2 : Pch_{104_STORAGE_1,10}_2 >= 0.0
Clb_{105,10}_2 : Pch_{105_STORAGE_1,10}_2 >= 0.0
Clb_{106,10}_2 : Pch_{106_STORAGE_1,10}_2 >= 0.0
Clb_{107,10}_2 : Pch_{107_STORAGE_1,10}_2 >= 0.0
Clb_{108,10}_2 : Pch_{108_STORAGE_1,10}_2 >= 0.0
Clb_{109,10}_2 : Pch_{109_STORAGE_1,10}_2 >= 0.0
Clb_{110,10}_2 : Pch_{110_STORAGE_1,10}_2 >= 0.0
Clb_{111,10}_2 : Pch_{111_STORAGE_1,10}_2 >= 0.0
Clb_{112,10}_2 : Pch_{112_STORAGE_1,10}_2 >= 0.0
Clb_{113,10}_2 : Pch_{113_STORAGE_1,10}_2 >= 0.0
Clb_{114,10}_2 : Pch_{114_STORAGE_1,10}_2 >= 0.0
Clb_{115,10}_2 : Pch_{115_STORAGE_1,10}_2 >= 0.0
Clb_{116,10}_2 : Pch_{116_STORAGE_1,10}_2 >= 0.0
Clb_{117,10}_2 : Pch_{117_STORAGE_1,10}_2 >= 0.0
Clb_{118,10}_2 : Pch_{118_STORAGE_1,10}_2 >= 0.0
Clb_{119,10}_2 : Pch_{119_STORAGE_1,10}_2 >= 0.0
Clb_{120,10}_2 : Pch_{120_STORAGE_1,10}_2 >= 0.0
Clb_{121,10}_2 : Pch_{121_STORAGE_1,10}_2 >= 0.0
Clb_{122,10}_2 : Pch_{122_STORAGE_1,10}_2 >= 0.0
Clb_{123,10}_2 : Pch

Clb_{101,19}_2 : Pch_{101_STORAGE_1,19}_2 >= 0.0
Clb_{102,19}_2 : Pch_{102_STORAGE_1,19}_2 >= 0.0
Clb_{103,19}_2 : Pch_{103_STORAGE_1,19}_2 >= 0.0
Clb_{104,19}_2 : Pch_{104_STORAGE_1,19}_2 >= 0.0
Clb_{105,19}_2 : Pch_{105_STORAGE_1,19}_2 >= 0.0
Clb_{106,19}_2 : Pch_{106_STORAGE_1,19}_2 >= 0.0
Clb_{107,19}_2 : Pch_{107_STORAGE_1,19}_2 >= 0.0
Clb_{108,19}_2 : Pch_{108_STORAGE_1,19}_2 >= 0.0
Clb_{109,19}_2 : Pch_{109_STORAGE_1,19}_2 >= 0.0
Clb_{110,19}_2 : Pch_{110_STORAGE_1,19}_2 >= 0.0
Clb_{111,19}_2 : Pch_{111_STORAGE_1,19}_2 >= 0.0
Clb_{112,19}_2 : Pch_{112_STORAGE_1,19}_2 >= 0.0
Clb_{113,19}_2 : Pch_{113_STORAGE_1,19}_2 >= 0.0
Clb_{114,19}_2 : Pch_{114_STORAGE_1,19}_2 >= 0.0
Clb_{115,19}_2 : Pch_{115_STORAGE_1,19}_2 >= 0.0
Clb_{116,19}_2 : Pch_{116_STORAGE_1,19}_2 >= 0.0
Clb_{117,19}_2 : Pch_{117_STORAGE_1,19}_2 >= 0.0
Clb_{118,19}_2 : Pch_{118_STORAGE_1,19}_2 >= 0.0
Clb_{119,19}_2 : Pch_{119_STORAGE_1,19}_2 >= 0.0
Clb_{120,19}_2 : Pch_{120_STORAGE_1,19}_2 >= 0.0
Clb_{121,19}_2 : Pch

In [47]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1200 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [48]:
# add discharging lb constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_discharge_lb(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Dlb_{101,1}_2 : Pdis_{101_STORAGE_1,1}_2 >= 0.0
Dlb_{102,1}_2 : Pdis_{102_STORAGE_1,1}_2 >= 0.0
Dlb_{103,1}_2 : Pdis_{103_STORAGE_1,1}_2 >= 0.0
Dlb_{104,1}_2 : Pdis_{104_STORAGE_1,1}_2 >= 0.0
Dlb_{105,1}_2 : Pdis_{105_STORAGE_1,1}_2 >= 0.0
Dlb_{106,1}_2 : Pdis_{106_STORAGE_1,1}_2 >= 0.0
Dlb_{107,1}_2 : Pdis_{107_STORAGE_1,1}_2 >= 0.0
Dlb_{108,1}_2 : Pdis_{108_STORAGE_1,1}_2 >= 0.0
Dlb_{109,1}_2 : Pdis_{109_STORAGE_1,1}_2 >= 0.0
Dlb_{110,1}_2 : Pdis_{110_STORAGE_1,1}_2 >= 0.0
Dlb_{111,1}_2 : Pdis_{111_STORAGE_1,1}_2 >= 0.0
Dlb_{112,1}_2 : Pdis_{112_STORAGE_1,1}_2 >= 0.0
Dlb_{113,1}_2 : Pdis_{113_STORAGE_1,1}_2 >= 0.0
Dlb_{114,1}_2 : Pdis_{114_STORAGE_1,1}_2 >= 0.0
Dlb_{115,1}_2 : Pdis_{115_STORAGE_1,1}_2 >= 0.0
Dlb_{116,1}_2 : Pdis_{116_STORAGE_1,1}_2 >= 0.0
Dlb_{117,1}_2 : Pdis_{117_STORAGE_1,1}_2 >= 0.0
Dlb_{118,1}_2 : Pdis_{118_STORAGE_1,1}_2 >= 0.0
Dlb_{119,1}_2 : Pdis_{119_STORAGE_1,1}_2 >= 0.0
Dlb_{120,1}_2 : Pdis_{120_STORAGE_1,1}_2 >= 0.0
Dlb_{121,1}_2 : Pdis_{121_STORAGE_1,1}_2

Dlb_{119,8}_2 : Pdis_{119_STORAGE_1,8}_2 >= 0.0
Dlb_{120,8}_2 : Pdis_{120_STORAGE_1,8}_2 >= 0.0
Dlb_{121,8}_2 : Pdis_{121_STORAGE_1,8}_2 >= 0.0
Dlb_{122,8}_2 : Pdis_{122_STORAGE_1,8}_2 >= 0.0
Dlb_{123,8}_2 : Pdis_{123_STORAGE_1,8}_2 >= 0.0
Dlb_{124,8}_2 : Pdis_{124_STORAGE_1,8}_2 >= 0.0
Dlb_{101,9}_2 : Pdis_{101_STORAGE_1,9}_2 >= 0.0
Dlb_{102,9}_2 : Pdis_{102_STORAGE_1,9}_2 >= 0.0
Dlb_{103,9}_2 : Pdis_{103_STORAGE_1,9}_2 >= 0.0
Dlb_{104,9}_2 : Pdis_{104_STORAGE_1,9}_2 >= 0.0
Dlb_{105,9}_2 : Pdis_{105_STORAGE_1,9}_2 >= 0.0
Dlb_{106,9}_2 : Pdis_{106_STORAGE_1,9}_2 >= 0.0
Dlb_{107,9}_2 : Pdis_{107_STORAGE_1,9}_2 >= 0.0
Dlb_{108,9}_2 : Pdis_{108_STORAGE_1,9}_2 >= 0.0
Dlb_{109,9}_2 : Pdis_{109_STORAGE_1,9}_2 >= 0.0
Dlb_{110,9}_2 : Pdis_{110_STORAGE_1,9}_2 >= 0.0
Dlb_{111,9}_2 : Pdis_{111_STORAGE_1,9}_2 >= 0.0
Dlb_{112,9}_2 : Pdis_{112_STORAGE_1,9}_2 >= 0.0
Dlb_{113,9}_2 : Pdis_{113_STORAGE_1,9}_2 >= 0.0
Dlb_{114,9}_2 : Pdis_{114_STORAGE_1,9}_2 >= 0.0
Dlb_{115,9}_2 : Pdis_{115_STORAGE_1,9}_2

Dlb_{108,16}_2 : Pdis_{108_STORAGE_1,16}_2 >= 0.0
Dlb_{109,16}_2 : Pdis_{109_STORAGE_1,16}_2 >= 0.0
Dlb_{110,16}_2 : Pdis_{110_STORAGE_1,16}_2 >= 0.0
Dlb_{111,16}_2 : Pdis_{111_STORAGE_1,16}_2 >= 0.0
Dlb_{112,16}_2 : Pdis_{112_STORAGE_1,16}_2 >= 0.0
Dlb_{113,16}_2 : Pdis_{113_STORAGE_1,16}_2 >= 0.0
Dlb_{114,16}_2 : Pdis_{114_STORAGE_1,16}_2 >= 0.0
Dlb_{115,16}_2 : Pdis_{115_STORAGE_1,16}_2 >= 0.0
Dlb_{116,16}_2 : Pdis_{116_STORAGE_1,16}_2 >= 0.0
Dlb_{117,16}_2 : Pdis_{117_STORAGE_1,16}_2 >= 0.0
Dlb_{118,16}_2 : Pdis_{118_STORAGE_1,16}_2 >= 0.0
Dlb_{119,16}_2 : Pdis_{119_STORAGE_1,16}_2 >= 0.0
Dlb_{120,16}_2 : Pdis_{120_STORAGE_1,16}_2 >= 0.0
Dlb_{121,16}_2 : Pdis_{121_STORAGE_1,16}_2 >= 0.0
Dlb_{122,16}_2 : Pdis_{122_STORAGE_1,16}_2 >= 0.0
Dlb_{123,16}_2 : Pdis_{123_STORAGE_1,16}_2 >= 0.0
Dlb_{124,16}_2 : Pdis_{124_STORAGE_1,16}_2 >= 0.0
Dlb_{101,17}_2 : Pdis_{101_STORAGE_1,17}_2 >= 0.0
Dlb_{102,17}_2 : Pdis_{102_STORAGE_1,17}_2 >= 0.0
Dlb_{103,17}_2 : Pdis_{103_STORAGE_1,17}_2 >= 0.0


Dlb_{111,24}_2 : Pdis_{111_STORAGE_1,24}_2 >= 0.0
Dlb_{112,24}_2 : Pdis_{112_STORAGE_1,24}_2 >= 0.0
Dlb_{113,24}_2 : Pdis_{113_STORAGE_1,24}_2 >= 0.0
Dlb_{114,24}_2 : Pdis_{114_STORAGE_1,24}_2 >= 0.0
Dlb_{115,24}_2 : Pdis_{115_STORAGE_1,24}_2 >= 0.0
Dlb_{116,24}_2 : Pdis_{116_STORAGE_1,24}_2 >= 0.0
Dlb_{117,24}_2 : Pdis_{117_STORAGE_1,24}_2 >= 0.0
Dlb_{118,24}_2 : Pdis_{118_STORAGE_1,24}_2 >= 0.0
Dlb_{119,24}_2 : Pdis_{119_STORAGE_1,24}_2 >= 0.0
Dlb_{120,24}_2 : Pdis_{120_STORAGE_1,24}_2 >= 0.0
Dlb_{121,24}_2 : Pdis_{121_STORAGE_1,24}_2 >= 0.0
Dlb_{122,24}_2 : Pdis_{122_STORAGE_1,24}_2 >= 0.0
Dlb_{123,24}_2 : Pdis_{123_STORAGE_1,24}_2 >= 0.0
Dlb_{124,24}_2 : Pdis_{124_STORAGE_1,24}_2 >= 0.0


A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1776 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [49]:
# add storage lb constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_storage_lb(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Elb_{101,1}_2 : Est_{101_STORAGE_1,1}_2 >= 0.0
Elb_{102,1}_2 : Est_{102_STORAGE_1,1}_2 >= 0.0
Elb_{103,1}_2 : Est_{103_STORAGE_1,1}_2 >= 0.0
Elb_{104,1}_2 : Est_{104_STORAGE_1,1}_2 >= 0.0
Elb_{105,1}_2 : Est_{105_STORAGE_1,1}_2 >= 0.0
Elb_{106,1}_2 : Est_{106_STORAGE_1,1}_2 >= 0.0
Elb_{107,1}_2 : Est_{107_STORAGE_1,1}_2 >= 0.0
Elb_{108,1}_2 : Est_{108_STORAGE_1,1}_2 >= 0.0
Elb_{109,1}_2 : Est_{109_STORAGE_1,1}_2 >= 0.0
Elb_{110,1}_2 : Est_{110_STORAGE_1,1}_2 >= 0.0
Elb_{111,1}_2 : Est_{111_STORAGE_1,1}_2 >= 0.0
Elb_{112,1}_2 : Est_{112_STORAGE_1,1}_2 >= 0.0
Elb_{113,1}_2 : Est_{113_STORAGE_1,1}_2 >= 0.0
Elb_{114,1}_2 : Est_{114_STORAGE_1,1}_2 >= 0.0
Elb_{115,1}_2 : Est_{115_STORAGE_1,1}_2 >= 0.0
Elb_{116,1}_2 : Est_{116_STORAGE_1,1}_2 >= 0.0
Elb_{117,1}_2 : Est_{117_STORAGE_1,1}_2 >= 0.0
Elb_{118,1}_2 : Est_{118_STORAGE_1,1}_2 >= 0.0
Elb_{119,1}_2 : Est_{119_STORAGE_1,1}_2 >= 0.0
Elb_{120,1}_2 : Est_{120_STORAGE_1,1}_2 >= 0.0
Elb_{121,1}_2 : Est_{121_STORAGE_1,1}_2 >= 0.0
Elb_{122,1}_2

Elb_{121,9}_2 : Est_{121_STORAGE_1,9}_2 >= 0.0
Elb_{122,9}_2 : Est_{122_STORAGE_1,9}_2 >= 0.0
Elb_{123,9}_2 : Est_{123_STORAGE_1,9}_2 >= 0.0
Elb_{124,9}_2 : Est_{124_STORAGE_1,9}_2 >= 0.0
Elb_{101,10}_2 : Est_{101_STORAGE_1,10}_2 >= 0.0
Elb_{102,10}_2 : Est_{102_STORAGE_1,10}_2 >= 0.0
Elb_{103,10}_2 : Est_{103_STORAGE_1,10}_2 >= 0.0
Elb_{104,10}_2 : Est_{104_STORAGE_1,10}_2 >= 0.0
Elb_{105,10}_2 : Est_{105_STORAGE_1,10}_2 >= 0.0
Elb_{106,10}_2 : Est_{106_STORAGE_1,10}_2 >= 0.0
Elb_{107,10}_2 : Est_{107_STORAGE_1,10}_2 >= 0.0
Elb_{108,10}_2 : Est_{108_STORAGE_1,10}_2 >= 0.0
Elb_{109,10}_2 : Est_{109_STORAGE_1,10}_2 >= 0.0
Elb_{110,10}_2 : Est_{110_STORAGE_1,10}_2 >= 0.0
Elb_{111,10}_2 : Est_{111_STORAGE_1,10}_2 >= 0.0
Elb_{112,10}_2 : Est_{112_STORAGE_1,10}_2 >= 0.0
Elb_{113,10}_2 : Est_{113_STORAGE_1,10}_2 >= 0.0
Elb_{114,10}_2 : Est_{114_STORAGE_1,10}_2 >= 0.0
Elb_{115,10}_2 : Est_{115_STORAGE_1,10}_2 >= 0.0
Elb_{116,10}_2 : Est_{116_STORAGE_1,10}_2 >= 0.0
Elb_{117,10}_2 : Est_{117_ST

Elb_{123,16}_2 : Est_{123_STORAGE_1,16}_2 >= 0.0
Elb_{124,16}_2 : Est_{124_STORAGE_1,16}_2 >= 0.0
Elb_{101,17}_2 : Est_{101_STORAGE_1,17}_2 >= 0.0
Elb_{102,17}_2 : Est_{102_STORAGE_1,17}_2 >= 0.0
Elb_{103,17}_2 : Est_{103_STORAGE_1,17}_2 >= 0.0
Elb_{104,17}_2 : Est_{104_STORAGE_1,17}_2 >= 0.0
Elb_{105,17}_2 : Est_{105_STORAGE_1,17}_2 >= 0.0
Elb_{106,17}_2 : Est_{106_STORAGE_1,17}_2 >= 0.0
Elb_{107,17}_2 : Est_{107_STORAGE_1,17}_2 >= 0.0
Elb_{108,17}_2 : Est_{108_STORAGE_1,17}_2 >= 0.0
Elb_{109,17}_2 : Est_{109_STORAGE_1,17}_2 >= 0.0
Elb_{110,17}_2 : Est_{110_STORAGE_1,17}_2 >= 0.0
Elb_{111,17}_2 : Est_{111_STORAGE_1,17}_2 >= 0.0
Elb_{112,17}_2 : Est_{112_STORAGE_1,17}_2 >= 0.0
Elb_{113,17}_2 : Est_{113_STORAGE_1,17}_2 >= 0.0
Elb_{114,17}_2 : Est_{114_STORAGE_1,17}_2 >= 0.0
Elb_{115,17}_2 : Est_{115_STORAGE_1,17}_2 >= 0.0
Elb_{116,17}_2 : Est_{116_STORAGE_1,17}_2 >= 0.0
Elb_{117,17}_2 : Est_{117_STORAGE_1,17}_2 >= 0.0
Elb_{118,17}_2 : Est_{118_STORAGE_1,17}_2 >= 0.0
Elb_{119,17}_2 : Est

Elb_{101,24}_2 : Est_{101_STORAGE_1,24}_2 >= 0.0
Elb_{102,24}_2 : Est_{102_STORAGE_1,24}_2 >= 0.0
Elb_{103,24}_2 : Est_{103_STORAGE_1,24}_2 >= 0.0
Elb_{104,24}_2 : Est_{104_STORAGE_1,24}_2 >= 0.0
Elb_{105,24}_2 : Est_{105_STORAGE_1,24}_2 >= 0.0
Elb_{106,24}_2 : Est_{106_STORAGE_1,24}_2 >= 0.0
Elb_{107,24}_2 : Est_{107_STORAGE_1,24}_2 >= 0.0
Elb_{108,24}_2 : Est_{108_STORAGE_1,24}_2 >= 0.0
Elb_{109,24}_2 : Est_{109_STORAGE_1,24}_2 >= 0.0
Elb_{110,24}_2 : Est_{110_STORAGE_1,24}_2 >= 0.0
Elb_{111,24}_2 : Est_{111_STORAGE_1,24}_2 >= 0.0
Elb_{112,24}_2 : Est_{112_STORAGE_1,24}_2 >= 0.0
Elb_{113,24}_2 : Est_{113_STORAGE_1,24}_2 >= 0.0
Elb_{114,24}_2 : Est_{114_STORAGE_1,24}_2 >= 0.0
Elb_{115,24}_2 : Est_{115_STORAGE_1,24}_2 >= 0.0
Elb_{116,24}_2 : Est_{116_STORAGE_1,24}_2 >= 0.0
Elb_{117,24}_2 : Est_{117_STORAGE_1,24}_2 >= 0.0
Elb_{118,24}_2 : Est_{118_STORAGE_1,24}_2 >= 0.0
Elb_{119,24}_2 : Est_{119_STORAGE_1,24}_2 >= 0.0
Elb_{120,24}_2 : Est_{120_STORAGE_1,24}_2 >= 0.0
Elb_{121,24}_2 : Est

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2352 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [50]:
# add loss of load lb constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_loss_of_load_lb(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Llb_{101,1}_2 : LOL_{101,1}_2 >= 0.0
Llb_{102,1}_2 : LOL_{102,1}_2 >= 0.0
Llb_{103,1}_2 : LOL_{103,1}_2 >= 0.0
Llb_{104,1}_2 : LOL_{104,1}_2 >= 0.0
Llb_{105,1}_2 : LOL_{105,1}_2 >= 0.0
Llb_{106,1}_2 : LOL_{106,1}_2 >= 0.0
Llb_{107,1}_2 : LOL_{107,1}_2 >= 0.0
Llb_{108,1}_2 : LOL_{108,1}_2 >= 0.0
Llb_{109,1}_2 : LOL_{109,1}_2 >= 0.0
Llb_{110,1}_2 : LOL_{110,1}_2 >= 0.0
Llb_{111,1}_2 : LOL_{111,1}_2 >= 0.0
Llb_{112,1}_2 : LOL_{112,1}_2 >= 0.0
Llb_{113,1}_2 : LOL_{113,1}_2 >= 0.0
Llb_{114,1}_2 : LOL_{114,1}_2 >= 0.0
Llb_{115,1}_2 : LOL_{115,1}_2 >= 0.0
Llb_{116,1}_2 : LOL_{116,1}_2 >= 0.0
Llb_{117,1}_2 : LOL_{117,1}_2 >= 0.0
Llb_{118,1}_2 : LOL_{118,1}_2 >= 0.0
Llb_{119,1}_2 : LOL_{119,1}_2 >= 0.0
Llb_{120,1}_2 : LOL_{120,1}_2 >= 0.0
Llb_{121,1}_2 : LOL_{121,1}_2 >= 0.0
Llb_{122,1}_2 : LOL_{122,1}_2 >= 0.0
Llb_{123,1}_2 : LOL_{123,1}_2 >= 0.0
Llb_{124,1}_2 : LOL_{124,1}_2 >= 0.0
Llb_{101,2}_2 : LOL_{101,2}_2 >= 0.0
Llb_{102,2}_2 : LOL_{102,2}_2 >= 0.0
Llb_{103,2}_2 : LOL_{103,2}_2 >= 0.0
L

Llb_{102,12}_2 : LOL_{102,12}_2 >= 0.0
Llb_{103,12}_2 : LOL_{103,12}_2 >= 0.0
Llb_{104,12}_2 : LOL_{104,12}_2 >= 0.0
Llb_{105,12}_2 : LOL_{105,12}_2 >= 0.0
Llb_{106,12}_2 : LOL_{106,12}_2 >= 0.0
Llb_{107,12}_2 : LOL_{107,12}_2 >= 0.0
Llb_{108,12}_2 : LOL_{108,12}_2 >= 0.0
Llb_{109,12}_2 : LOL_{109,12}_2 >= 0.0
Llb_{110,12}_2 : LOL_{110,12}_2 >= 0.0
Llb_{111,12}_2 : LOL_{111,12}_2 >= 0.0
Llb_{112,12}_2 : LOL_{112,12}_2 >= 0.0
Llb_{113,12}_2 : LOL_{113,12}_2 >= 0.0
Llb_{114,12}_2 : LOL_{114,12}_2 >= 0.0
Llb_{115,12}_2 : LOL_{115,12}_2 >= 0.0
Llb_{116,12}_2 : LOL_{116,12}_2 >= 0.0
Llb_{117,12}_2 : LOL_{117,12}_2 >= 0.0
Llb_{118,12}_2 : LOL_{118,12}_2 >= 0.0
Llb_{119,12}_2 : LOL_{119,12}_2 >= 0.0
Llb_{120,12}_2 : LOL_{120,12}_2 >= 0.0
Llb_{121,12}_2 : LOL_{121,12}_2 >= 0.0
Llb_{122,12}_2 : LOL_{122,12}_2 >= 0.0
Llb_{123,12}_2 : LOL_{123,12}_2 >= 0.0
Llb_{124,12}_2 : LOL_{124,12}_2 >= 0.0
Llb_{101,13}_2 : LOL_{101,13}_2 >= 0.0
Llb_{102,13}_2 : LOL_{102,13}_2 >= 0.0
Llb_{103,13}_2 : LOL_{103

Llb_{103,21}_2 : LOL_{103,21}_2 >= 0.0
Llb_{104,21}_2 : LOL_{104,21}_2 >= 0.0
Llb_{105,21}_2 : LOL_{105,21}_2 >= 0.0
Llb_{106,21}_2 : LOL_{106,21}_2 >= 0.0
Llb_{107,21}_2 : LOL_{107,21}_2 >= 0.0
Llb_{108,21}_2 : LOL_{108,21}_2 >= 0.0
Llb_{109,21}_2 : LOL_{109,21}_2 >= 0.0
Llb_{110,21}_2 : LOL_{110,21}_2 >= 0.0
Llb_{111,21}_2 : LOL_{111,21}_2 >= 0.0
Llb_{112,21}_2 : LOL_{112,21}_2 >= 0.0
Llb_{113,21}_2 : LOL_{113,21}_2 >= 0.0
Llb_{114,21}_2 : LOL_{114,21}_2 >= 0.0
Llb_{115,21}_2 : LOL_{115,21}_2 >= 0.0
Llb_{116,21}_2 : LOL_{116,21}_2 >= 0.0
Llb_{117,21}_2 : LOL_{117,21}_2 >= 0.0
Llb_{118,21}_2 : LOL_{118,21}_2 >= 0.0
Llb_{119,21}_2 : LOL_{119,21}_2 >= 0.0
Llb_{120,21}_2 : LOL_{120,21}_2 >= 0.0
Llb_{121,21}_2 : LOL_{121,21}_2 >= 0.0
Llb_{122,21}_2 : LOL_{122,21}_2 >= 0.0
Llb_{123,21}_2 : LOL_{123,21}_2 >= 0.0
Llb_{124,21}_2 : LOL_{124,21}_2 >= 0.0
Llb_{101,22}_2 : LOL_{101,22}_2 >= 0.0
Llb_{102,22}_2 : LOL_{102,22}_2 >= 0.0
Llb_{103,22}_2 : LOL_{103,22}_2 >= 0.0
Llb_{104,22}_2 : LOL_{104

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2928 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [51]:
# add overload lb constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_overload_lb(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar >= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Olb_{101,1}_2 : OL_{101,1}_2 >= 0.0
Olb_{102,1}_2 : OL_{102,1}_2 >= 0.0
Olb_{103,1}_2 : OL_{103,1}_2 >= 0.0
Olb_{104,1}_2 : OL_{104,1}_2 >= 0.0
Olb_{105,1}_2 : OL_{105,1}_2 >= 0.0
Olb_{106,1}_2 : OL_{106,1}_2 >= 0.0
Olb_{107,1}_2 : OL_{107,1}_2 >= 0.0
Olb_{108,1}_2 : OL_{108,1}_2 >= 0.0
Olb_{109,1}_2 : OL_{109,1}_2 >= 0.0
Olb_{110,1}_2 : OL_{110,1}_2 >= 0.0
Olb_{111,1}_2 : OL_{111,1}_2 >= 0.0
Olb_{112,1}_2 : OL_{112,1}_2 >= 0.0
Olb_{113,1}_2 : OL_{113,1}_2 >= 0.0
Olb_{114,1}_2 : OL_{114,1}_2 >= 0.0
Olb_{115,1}_2 : OL_{115,1}_2 >= 0.0
Olb_{116,1}_2 : OL_{116,1}_2 >= 0.0
Olb_{117,1}_2 : OL_{117,1}_2 >= 0.0
Olb_{118,1}_2 : OL_{118,1}_2 >= 0.0
Olb_{119,1}_2 : OL_{119,1}_2 >= 0.0
Olb_{120,1}_2 : OL_{120,1}_2 >= 0.0
Olb_{121,1}_2 : OL_{121,1}_2 >= 0.0
Olb_{122,1}_2 : OL_{122,1}_2 >= 0.0
Olb_{123,1}_2 : OL_{123,1}_2 >= 0.0
Olb_{124,1}_2 : OL_{124,1}_2 >= 0.0
Olb_{101,2}_2 : OL_{101,2}_2 >= 0.0
Olb_{102,2}_2 : OL_{102,2}_2 >= 0.0
Olb_{103,2}_2 : OL_{103,2}_2 >= 0.0
Olb_{104,2}_2 : OL_{104,2}_2

Olb_{118,10}_2 : OL_{118,10}_2 >= 0.0
Olb_{119,10}_2 : OL_{119,10}_2 >= 0.0
Olb_{120,10}_2 : OL_{120,10}_2 >= 0.0
Olb_{121,10}_2 : OL_{121,10}_2 >= 0.0
Olb_{122,10}_2 : OL_{122,10}_2 >= 0.0
Olb_{123,10}_2 : OL_{123,10}_2 >= 0.0
Olb_{124,10}_2 : OL_{124,10}_2 >= 0.0
Olb_{101,11}_2 : OL_{101,11}_2 >= 0.0
Olb_{102,11}_2 : OL_{102,11}_2 >= 0.0
Olb_{103,11}_2 : OL_{103,11}_2 >= 0.0
Olb_{104,11}_2 : OL_{104,11}_2 >= 0.0
Olb_{105,11}_2 : OL_{105,11}_2 >= 0.0
Olb_{106,11}_2 : OL_{106,11}_2 >= 0.0
Olb_{107,11}_2 : OL_{107,11}_2 >= 0.0
Olb_{108,11}_2 : OL_{108,11}_2 >= 0.0
Olb_{109,11}_2 : OL_{109,11}_2 >= 0.0
Olb_{110,11}_2 : OL_{110,11}_2 >= 0.0
Olb_{111,11}_2 : OL_{111,11}_2 >= 0.0
Olb_{112,11}_2 : OL_{112,11}_2 >= 0.0
Olb_{113,11}_2 : OL_{113,11}_2 >= 0.0
Olb_{114,11}_2 : OL_{114,11}_2 >= 0.0
Olb_{115,11}_2 : OL_{115,11}_2 >= 0.0
Olb_{116,11}_2 : OL_{116,11}_2 >= 0.0
Olb_{117,11}_2 : OL_{117,11}_2 >= 0.0
Olb_{118,11}_2 : OL_{118,11}_2 >= 0.0
Olb_{119,11}_2 : OL_{119,11}_2 >= 0.0
Olb_{120,11}

Olb_{119,19}_2 : OL_{119,19}_2 >= 0.0
Olb_{120,19}_2 : OL_{120,19}_2 >= 0.0
Olb_{121,19}_2 : OL_{121,19}_2 >= 0.0
Olb_{122,19}_2 : OL_{122,19}_2 >= 0.0
Olb_{123,19}_2 : OL_{123,19}_2 >= 0.0
Olb_{124,19}_2 : OL_{124,19}_2 >= 0.0
Olb_{101,20}_2 : OL_{101,20}_2 >= 0.0
Olb_{102,20}_2 : OL_{102,20}_2 >= 0.0
Olb_{103,20}_2 : OL_{103,20}_2 >= 0.0
Olb_{104,20}_2 : OL_{104,20}_2 >= 0.0
Olb_{105,20}_2 : OL_{105,20}_2 >= 0.0
Olb_{106,20}_2 : OL_{106,20}_2 >= 0.0
Olb_{107,20}_2 : OL_{107,20}_2 >= 0.0
Olb_{108,20}_2 : OL_{108,20}_2 >= 0.0
Olb_{109,20}_2 : OL_{109,20}_2 >= 0.0
Olb_{110,20}_2 : OL_{110,20}_2 >= 0.0
Olb_{111,20}_2 : OL_{111,20}_2 >= 0.0
Olb_{112,20}_2 : OL_{112,20}_2 >= 0.0
Olb_{113,20}_2 : OL_{113,20}_2 >= 0.0
Olb_{114,20}_2 : OL_{114,20}_2 >= 0.0
Olb_{115,20}_2 : OL_{115,20}_2 >= 0.0
Olb_{116,20}_2 : OL_{116,20}_2 >= 0.0
Olb_{117,20}_2 : OL_{117,20}_2 >= 0.0
Olb_{118,20}_2 : OL_{118,20}_2 >= 0.0
Olb_{119,20}_2 : OL_{119,20}_2 >= 0.0
Olb_{120,20}_2 : OL_{120,20}_2 >= 0.0
Olb_{121,20}

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [52]:
# add wind ub constraint

for ts in timesteps
    bus = 122
    ### change constraint here
    subcon = get_wind_ub(submodel, bus, ts)
    ###
    subconname = JuMP.name(subcon)
    exconname = string(subconname, "_$(scen)")

    terms = JuMP.constraint_object(subcon).func.terms
    ### change sense here
    value = JuMP.constraint_object(subcon).set.upper
    ###

    count = 0
    for (subvar, coeff) in terms
        if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
            exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
        else
            exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
        end
        if count == 0
            ### change objective type here
            JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                        base_name = exconname)
            ###
            count += 1
        else
            excon = JuMP.constraint_by_name(exmodel, exconname)
            JuMP.set_normalized_coefficient(excon, exvar, coeff)
        end
    end
    println(JuMP.constraint_by_name(exmodel, exconname))
end

Wub_{122,1}_2 : Pre_{122_WIND_1,1}_2 <= 6.3
Wub_{122,2}_2 : Pre_{122_WIND_1,2}_2 <= 6.723
Wub_{122,3}_2 : Pre_{122_WIND_1,3}_2 <= 6.6129999999999995
Wub_{122,4}_2 : Pre_{122_WIND_1,4}_2 <= 6.54
Wub_{122,5}_2 : Pre_{122_WIND_1,5}_2 <= 6.544
Wub_{122,6}_2 : Pre_{122_WIND_1,6}_2 <= 6.664
Wub_{122,7}_2 : Pre_{122_WIND_1,7}_2 <= 6.867000000000001
Wub_{122,8}_2 : Pre_{122_WIND_1,8}_2 <= 6.893
Wub_{122,9}_2 : Pre_{122_WIND_1,9}_2 <= 6.838
Wub_{122,10}_2 : Pre_{122_WIND_1,10}_2 <= 6.986000000000001
Wub_{122,11}_2 : Pre_{122_WIND_1,11}_2 <= 7.135
Wub_{122,12}_2 : Pre_{122_WIND_1,12}_2 <= 7.103
Wub_{122,13}_2 : Pre_{122_WIND_1,13}_2 <= 7.022
Wub_{122,14}_2 : Pre_{122_WIND_1,14}_2 <= 6.992000000000001
Wub_{122,15}_2 : Pre_{122_WIND_1,15}_2 <= 6.869
Wub_{122,16}_2 : Pre_{122_WIND_1,16}_2 <= 6.954
Wub_{122,17}_2 : Pre_{122_WIND_1,17}_2 <= 7.122999999999999
Wub_{122,18}_2 : Pre_{122_WIND_1,18}_2 <= 7.124
Wub_{122,19}_2 : Pre_{122_WIND_1,19}_2 <= 7.122000000000001
Wub_{122,20}_2 : Pre_{122_WIND_1,20}

In [53]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 49 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [54]:
# add ramping ub constraint

for ts in timesteps
    if ts > 1
        for gen in gens
            ### change constraint here
            subcon = get_ramp_ub(submodel, gen, ts)
            ###
            subconname = JuMP.name(subcon)
            exconname = string(subconname, "_$(scen)")

            terms = JuMP.constraint_object(subcon).func.terms
            ### change sense here
            value = JuMP.constraint_object(subcon).set.upper
            ###

            count = 0
            for (subvar, coeff) in terms
                if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                    exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
                else
                    exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
                end
                if count == 0
                    ### change objective type here
                    JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                                base_name = exconname)
                    ###
                    count += 1
                else
                    excon = JuMP.constraint_by_name(exmodel, exconname)
                    JuMP.set_normalized_coefficient(excon, exvar, coeff)
                end
            end
            println(JuMP.constraint_by_name(exmodel, exconname))
        end
    end
end

Rub_{101_CT_1,2}_2 : -Pth_{101_CT_1,1}_2 + Pth_{101_CT_1,2}_2 <= 1.7999999999999998
Rub_{101_CT_2,2}_2 : -Pth_{101_CT_2,1}_2 + Pth_{101_CT_2,2}_2 <= 1.7999999999999998
Rub_{101_STEAM_3,2}_2 : -Pth_{101_STEAM_3,1}_2 + Pth_{101_STEAM_3,2}_2 <= 1.2
Rub_{101_STEAM_4,2}_2 : -Pth_{101_STEAM_4,1}_2 + Pth_{101_STEAM_4,2}_2 <= 1.2
Rub_{102_CT_1,2}_2 : -Pth_{102_CT_1,1}_2 + Pth_{102_CT_1,2}_2 <= 1.7999999999999998
Rub_{102_CT_2,2}_2 : -Pth_{102_CT_2,1}_2 + Pth_{102_CT_2,2}_2 <= 1.7999999999999998
Rub_{102_STEAM_3,2}_2 : -Pth_{102_STEAM_3,1}_2 + Pth_{102_STEAM_3,2}_2 <= 1.2
Rub_{102_STEAM_4,2}_2 : -Pth_{102_STEAM_4,1}_2 + Pth_{102_STEAM_4,2}_2 <= 1.2
Rub_{107_CC_1,2}_2 : -Pth_{107_CC_1,1}_2 + Pth_{107_CC_1,2}_2 <= 2.484
Rub_{113_CT_1,2}_2 : -Pth_{113_CT_1,1}_2 + Pth_{113_CT_1,2}_2 <= 2.22
Rub_{113_CT_2,2}_2 : -Pth_{113_CT_2,1}_2 + Pth_{113_CT_2,2}_2 <= 2.22
Rub_{113_CT_3,2}_2 : -Pth_{113_CT_3,1}_2 + Pth_{113_CT_3,2}_2 <= 2.22
Rub_{113_CT_4,2}_2 : -Pth_{113_CT_4,1}_2 + Pth_{113_CT_4,2}_2 <= 2.22
R

Rub_{101_CT_1,7}_2 : -Pth_{101_CT_1,6}_2 + Pth_{101_CT_1,7}_2 <= 1.7999999999999998
Rub_{101_CT_2,7}_2 : -Pth_{101_CT_2,6}_2 + Pth_{101_CT_2,7}_2 <= 1.7999999999999998
Rub_{101_STEAM_3,7}_2 : -Pth_{101_STEAM_3,6}_2 + Pth_{101_STEAM_3,7}_2 <= 1.2
Rub_{101_STEAM_4,7}_2 : -Pth_{101_STEAM_4,6}_2 + Pth_{101_STEAM_4,7}_2 <= 1.2
Rub_{102_CT_1,7}_2 : -Pth_{102_CT_1,6}_2 + Pth_{102_CT_1,7}_2 <= 1.7999999999999998
Rub_{102_CT_2,7}_2 : -Pth_{102_CT_2,6}_2 + Pth_{102_CT_2,7}_2 <= 1.7999999999999998
Rub_{102_STEAM_3,7}_2 : -Pth_{102_STEAM_3,6}_2 + Pth_{102_STEAM_3,7}_2 <= 1.2
Rub_{102_STEAM_4,7}_2 : -Pth_{102_STEAM_4,6}_2 + Pth_{102_STEAM_4,7}_2 <= 1.2
Rub_{107_CC_1,7}_2 : -Pth_{107_CC_1,6}_2 + Pth_{107_CC_1,7}_2 <= 2.484
Rub_{113_CT_1,7}_2 : -Pth_{113_CT_1,6}_2 + Pth_{113_CT_1,7}_2 <= 2.22
Rub_{113_CT_2,7}_2 : -Pth_{113_CT_2,6}_2 + Pth_{113_CT_2,7}_2 <= 2.22
Rub_{113_CT_3,7}_2 : -Pth_{113_CT_3,6}_2 + Pth_{113_CT_3,7}_2 <= 2.22
Rub_{113_CT_4,7}_2 : -Pth_{113_CT_4,6}_2 + Pth_{113_CT_4,7}_2 <= 2.22
R

Rub_{101_CT_2,13}_2 : -Pth_{101_CT_2,12}_2 + Pth_{101_CT_2,13}_2 <= 1.7999999999999998
Rub_{101_STEAM_3,13}_2 : -Pth_{101_STEAM_3,12}_2 + Pth_{101_STEAM_3,13}_2 <= 1.2
Rub_{101_STEAM_4,13}_2 : -Pth_{101_STEAM_4,12}_2 + Pth_{101_STEAM_4,13}_2 <= 1.2
Rub_{102_CT_1,13}_2 : -Pth_{102_CT_1,12}_2 + Pth_{102_CT_1,13}_2 <= 1.7999999999999998
Rub_{102_CT_2,13}_2 : -Pth_{102_CT_2,12}_2 + Pth_{102_CT_2,13}_2 <= 1.7999999999999998
Rub_{102_STEAM_3,13}_2 : -Pth_{102_STEAM_3,12}_2 + Pth_{102_STEAM_3,13}_2 <= 1.2
Rub_{102_STEAM_4,13}_2 : -Pth_{102_STEAM_4,12}_2 + Pth_{102_STEAM_4,13}_2 <= 1.2
Rub_{107_CC_1,13}_2 : -Pth_{107_CC_1,12}_2 + Pth_{107_CC_1,13}_2 <= 2.484
Rub_{113_CT_1,13}_2 : -Pth_{113_CT_1,12}_2 + Pth_{113_CT_1,13}_2 <= 2.22
Rub_{113_CT_2,13}_2 : -Pth_{113_CT_2,12}_2 + Pth_{113_CT_2,13}_2 <= 2.22
Rub_{113_CT_3,13}_2 : -Pth_{113_CT_3,12}_2 + Pth_{113_CT_3,13}_2 <= 2.22
Rub_{113_CT_4,13}_2 : -Pth_{113_CT_4,12}_2 + Pth_{113_CT_4,13}_2 <= 2.22
Rub_{115_STEAM_1,13}_2 : -Pth_{115_STEAM_1,12}_2 

Rub_{123_STEAM_3,17}_2 : -Pth_{123_STEAM_3,16}_2 + Pth_{123_STEAM_3,17}_2 <= 2.4
Rub_{123_CT_1,17}_2 : -Pth_{123_CT_1,16}_2 + Pth_{123_CT_1,17}_2 <= 2.22
Rub_{123_CT_4,17}_2 : -Pth_{123_CT_4,16}_2 + Pth_{123_CT_4,17}_2 <= 2.22
Rub_{123_CT_5,17}_2 : -Pth_{123_CT_5,16}_2 + Pth_{123_CT_5,17}_2 <= 2.22
Rub_{121_NUCLEAR_1,17}_2 : -Pth_{121_NUCLEAR_1,16}_2 + Pth_{121_NUCLEAR_1,17}_2 <= 12.0
Rub_{101_CT_1,18}_2 : -Pth_{101_CT_1,17}_2 + Pth_{101_CT_1,18}_2 <= 1.7999999999999998
Rub_{101_CT_2,18}_2 : -Pth_{101_CT_2,17}_2 + Pth_{101_CT_2,18}_2 <= 1.7999999999999998
Rub_{101_STEAM_3,18}_2 : -Pth_{101_STEAM_3,17}_2 + Pth_{101_STEAM_3,18}_2 <= 1.2
Rub_{101_STEAM_4,18}_2 : -Pth_{101_STEAM_4,17}_2 + Pth_{101_STEAM_4,18}_2 <= 1.2
Rub_{102_CT_1,18}_2 : -Pth_{102_CT_1,17}_2 + Pth_{102_CT_1,18}_2 <= 1.7999999999999998
Rub_{102_CT_2,18}_2 : -Pth_{102_CT_2,17}_2 + Pth_{102_CT_2,18}_2 <= 1.7999999999999998
Rub_{102_STEAM_3,18}_2 : -Pth_{102_STEAM_3,17}_2 + Pth_{102_STEAM_3,18}_2 <= 1.2
Rub_{102_STEAM_4,18}_

Rub_{123_CT_1,22}_2 : -Pth_{123_CT_1,21}_2 + Pth_{123_CT_1,22}_2 <= 2.22
Rub_{123_CT_4,22}_2 : -Pth_{123_CT_4,21}_2 + Pth_{123_CT_4,22}_2 <= 2.22
Rub_{123_CT_5,22}_2 : -Pth_{123_CT_5,21}_2 + Pth_{123_CT_5,22}_2 <= 2.22
Rub_{121_NUCLEAR_1,22}_2 : -Pth_{121_NUCLEAR_1,21}_2 + Pth_{121_NUCLEAR_1,22}_2 <= 12.0
Rub_{101_CT_1,23}_2 : -Pth_{101_CT_1,22}_2 + Pth_{101_CT_1,23}_2 <= 1.7999999999999998
Rub_{101_CT_2,23}_2 : -Pth_{101_CT_2,22}_2 + Pth_{101_CT_2,23}_2 <= 1.7999999999999998
Rub_{101_STEAM_3,23}_2 : -Pth_{101_STEAM_3,22}_2 + Pth_{101_STEAM_3,23}_2 <= 1.2
Rub_{101_STEAM_4,23}_2 : -Pth_{101_STEAM_4,22}_2 + Pth_{101_STEAM_4,23}_2 <= 1.2
Rub_{102_CT_1,23}_2 : -Pth_{102_CT_1,22}_2 + Pth_{102_CT_1,23}_2 <= 1.7999999999999998
Rub_{102_CT_2,23}_2 : -Pth_{102_CT_2,22}_2 + Pth_{102_CT_2,23}_2 <= 1.7999999999999998
Rub_{102_STEAM_3,23}_2 : -Pth_{102_STEAM_3,22}_2 + Pth_{102_STEAM_3,23}_2 <= 1.2
Rub_{102_STEAM_4,23}_2 : -Pth_{102_STEAM_4,22}_2 + Pth_{102_STEAM_4,23}_2 <= 1.2
Rub_{107_CC_1,23}_2 :

In [55]:
exmodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 601 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [56]:
# add charging ub constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_charge_ub(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.upper
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Cub_{101,1}_2 : Pch_{101_STORAGE_1,1}_2 - PR_{101}_0 <= 0.0
Cub_{102,1}_2 : Pch_{102_STORAGE_1,1}_2 - PR_{102}_0 <= 0.0
Cub_{103,1}_2 : Pch_{103_STORAGE_1,1}_2 - PR_{103}_0 <= 0.0
Cub_{104,1}_2 : Pch_{104_STORAGE_1,1}_2 - PR_{104}_0 <= 0.0
Cub_{105,1}_2 : Pch_{105_STORAGE_1,1}_2 - PR_{105}_0 <= 0.0
Cub_{106,1}_2 : Pch_{106_STORAGE_1,1}_2 - PR_{106}_0 <= 0.0
Cub_{107,1}_2 : Pch_{107_STORAGE_1,1}_2 - PR_{107}_0 <= 0.0
Cub_{108,1}_2 : Pch_{108_STORAGE_1,1}_2 - PR_{108}_0 <= 0.0
Cub_{109,1}_2 : Pch_{109_STORAGE_1,1}_2 - PR_{109}_0 <= 0.0
Cub_{110,1}_2 : Pch_{110_STORAGE_1,1}_2 - PR_{110}_0 <= 0.0
Cub_{111,1}_2 : Pch_{111_STORAGE_1,1}_2 - PR_{111}_0 <= 0.0
Cub_{112,1}_2 : Pch_{112_STORAGE_1,1}_2 - PR_{112}_0 <= 0.0
Cub_{113,1}_2 : Pch_{113_STORAGE_1,1}_2 - PR_{113}_0 <= 0.0
Cub_{114,1}_2 : Pch_{114_STORAGE_1,1}_2 - PR_{114}_0 <= 0.0
Cub_{115,1}_2 : Pch_{115_STORAGE_1,1}_2 - PR_{115}_0 <= 0.0
Cub_{116,1}_2 : Pch_{116_STORAGE_1,1}_2 - PR_{116}_0 <= 0.0
Cub_{117,1}_2 : Pch_{117_STORAGE_1,1}_2 

Cub_{122,6}_2 : Pch_{122_STORAGE_1,6}_2 - PR_{122}_0 <= 0.0
Cub_{123,6}_2 : Pch_{123_STORAGE_1,6}_2 - PR_{123}_0 <= 0.0
Cub_{124,6}_2 : Pch_{124_STORAGE_1,6}_2 - PR_{124}_0 <= 0.0
Cub_{101,7}_2 : Pch_{101_STORAGE_1,7}_2 - PR_{101}_0 <= 0.0
Cub_{102,7}_2 : Pch_{102_STORAGE_1,7}_2 - PR_{102}_0 <= 0.0
Cub_{103,7}_2 : Pch_{103_STORAGE_1,7}_2 - PR_{103}_0 <= 0.0
Cub_{104,7}_2 : Pch_{104_STORAGE_1,7}_2 - PR_{104}_0 <= 0.0
Cub_{105,7}_2 : Pch_{105_STORAGE_1,7}_2 - PR_{105}_0 <= 0.0
Cub_{106,7}_2 : Pch_{106_STORAGE_1,7}_2 - PR_{106}_0 <= 0.0
Cub_{107,7}_2 : Pch_{107_STORAGE_1,7}_2 - PR_{107}_0 <= 0.0
Cub_{108,7}_2 : Pch_{108_STORAGE_1,7}_2 - PR_{108}_0 <= 0.0
Cub_{109,7}_2 : Pch_{109_STORAGE_1,7}_2 - PR_{109}_0 <= 0.0
Cub_{110,7}_2 : Pch_{110_STORAGE_1,7}_2 - PR_{110}_0 <= 0.0
Cub_{111,7}_2 : Pch_{111_STORAGE_1,7}_2 - PR_{111}_0 <= 0.0
Cub_{112,7}_2 : Pch_{112_STORAGE_1,7}_2 - PR_{112}_0 <= 0.0
Cub_{113,7}_2 : Pch_{113_STORAGE_1,7}_2 - PR_{113}_0 <= 0.0
Cub_{114,7}_2 : Pch_{114_STORAGE_1,7}_2 

Cub_{123,12}_2 : Pch_{123_STORAGE_1,12}_2 - PR_{123}_0 <= 0.0
Cub_{124,12}_2 : Pch_{124_STORAGE_1,12}_2 - PR_{124}_0 <= 0.0
Cub_{101,13}_2 : Pch_{101_STORAGE_1,13}_2 - PR_{101}_0 <= 0.0
Cub_{102,13}_2 : Pch_{102_STORAGE_1,13}_2 - PR_{102}_0 <= 0.0
Cub_{103,13}_2 : Pch_{103_STORAGE_1,13}_2 - PR_{103}_0 <= 0.0
Cub_{104,13}_2 : Pch_{104_STORAGE_1,13}_2 - PR_{104}_0 <= 0.0
Cub_{105,13}_2 : Pch_{105_STORAGE_1,13}_2 - PR_{105}_0 <= 0.0
Cub_{106,13}_2 : Pch_{106_STORAGE_1,13}_2 - PR_{106}_0 <= 0.0
Cub_{107,13}_2 : Pch_{107_STORAGE_1,13}_2 - PR_{107}_0 <= 0.0
Cub_{108,13}_2 : Pch_{108_STORAGE_1,13}_2 - PR_{108}_0 <= 0.0
Cub_{109,13}_2 : Pch_{109_STORAGE_1,13}_2 - PR_{109}_0 <= 0.0
Cub_{110,13}_2 : Pch_{110_STORAGE_1,13}_2 - PR_{110}_0 <= 0.0
Cub_{111,13}_2 : Pch_{111_STORAGE_1,13}_2 - PR_{111}_0 <= 0.0
Cub_{112,13}_2 : Pch_{112_STORAGE_1,13}_2 - PR_{112}_0 <= 0.0
Cub_{113,13}_2 : Pch_{113_STORAGE_1,13}_2 - PR_{113}_0 <= 0.0
Cub_{114,13}_2 : Pch_{114_STORAGE_1,13}_2 - PR_{114}_0 <= 0.0
Cub_{115

Cub_{124,18}_2 : Pch_{124_STORAGE_1,18}_2 - PR_{124}_0 <= 0.0
Cub_{101,19}_2 : Pch_{101_STORAGE_1,19}_2 - PR_{101}_0 <= 0.0
Cub_{102,19}_2 : Pch_{102_STORAGE_1,19}_2 - PR_{102}_0 <= 0.0
Cub_{103,19}_2 : Pch_{103_STORAGE_1,19}_2 - PR_{103}_0 <= 0.0
Cub_{104,19}_2 : Pch_{104_STORAGE_1,19}_2 - PR_{104}_0 <= 0.0
Cub_{105,19}_2 : Pch_{105_STORAGE_1,19}_2 - PR_{105}_0 <= 0.0
Cub_{106,19}_2 : Pch_{106_STORAGE_1,19}_2 - PR_{106}_0 <= 0.0
Cub_{107,19}_2 : Pch_{107_STORAGE_1,19}_2 - PR_{107}_0 <= 0.0
Cub_{108,19}_2 : Pch_{108_STORAGE_1,19}_2 - PR_{108}_0 <= 0.0
Cub_{109,19}_2 : Pch_{109_STORAGE_1,19}_2 - PR_{109}_0 <= 0.0
Cub_{110,19}_2 : Pch_{110_STORAGE_1,19}_2 - PR_{110}_0 <= 0.0
Cub_{111,19}_2 : Pch_{111_STORAGE_1,19}_2 - PR_{111}_0 <= 0.0
Cub_{112,19}_2 : Pch_{112_STORAGE_1,19}_2 - PR_{112}_0 <= 0.0
Cub_{113,19}_2 : Pch_{113_STORAGE_1,19}_2 - PR_{113}_0 <= 0.0
Cub_{114,19}_2 : Pch_{114_STORAGE_1,19}_2 - PR_{114}_0 <= 0.0
Cub_{115,19}_2 : Pch_{115_STORAGE_1,19}_2 - PR_{115}_0 <= 0.0
Cub_{116

Cub_{114,24}_2 : Pch_{114_STORAGE_1,24}_2 - PR_{114}_0 <= 0.0
Cub_{115,24}_2 : Pch_{115_STORAGE_1,24}_2 - PR_{115}_0 <= 0.0
Cub_{116,24}_2 : Pch_{116_STORAGE_1,24}_2 - PR_{116}_0 <= 0.0
Cub_{117,24}_2 : Pch_{117_STORAGE_1,24}_2 - PR_{117}_0 <= 0.0
Cub_{118,24}_2 : Pch_{118_STORAGE_1,24}_2 - PR_{118}_0 <= 0.0
Cub_{119,24}_2 : Pch_{119_STORAGE_1,24}_2 - PR_{119}_0 <= 0.0
Cub_{120,24}_2 : Pch_{120_STORAGE_1,24}_2 - PR_{120}_0 <= 0.0
Cub_{121,24}_2 : Pch_{121_STORAGE_1,24}_2 - PR_{121}_0 <= 0.0
Cub_{122,24}_2 : Pch_{122_STORAGE_1,24}_2 - PR_{122}_0 <= 0.0
Cub_{123,24}_2 : Pch_{123_STORAGE_1,24}_2 - PR_{123}_0 <= 0.0
Cub_{124,24}_2 : Pch_{124_STORAGE_1,24}_2 - PR_{124}_0 <= 0.0


A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1177 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [57]:
# add discharging ub constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_discharge_ub(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.upper
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Dub_{101,1}_2 : Pdis_{101_STORAGE_1,1}_2 - PR_{101}_0 <= 0.0
Dub_{102,1}_2 : Pdis_{102_STORAGE_1,1}_2 - PR_{102}_0 <= 0.0
Dub_{103,1}_2 : Pdis_{103_STORAGE_1,1}_2 - PR_{103}_0 <= 0.0
Dub_{104,1}_2 : Pdis_{104_STORAGE_1,1}_2 - PR_{104}_0 <= 0.0
Dub_{105,1}_2 : Pdis_{105_STORAGE_1,1}_2 - PR_{105}_0 <= 0.0
Dub_{106,1}_2 : Pdis_{106_STORAGE_1,1}_2 - PR_{106}_0 <= 0.0
Dub_{107,1}_2 : Pdis_{107_STORAGE_1,1}_2 - PR_{107}_0 <= 0.0
Dub_{108,1}_2 : Pdis_{108_STORAGE_1,1}_2 - PR_{108}_0 <= 0.0
Dub_{109,1}_2 : Pdis_{109_STORAGE_1,1}_2 - PR_{109}_0 <= 0.0
Dub_{110,1}_2 : Pdis_{110_STORAGE_1,1}_2 - PR_{110}_0 <= 0.0
Dub_{111,1}_2 : Pdis_{111_STORAGE_1,1}_2 - PR_{111}_0 <= 0.0
Dub_{112,1}_2 : Pdis_{112_STORAGE_1,1}_2 - PR_{112}_0 <= 0.0
Dub_{113,1}_2 : Pdis_{113_STORAGE_1,1}_2 - PR_{113}_0 <= 0.0
Dub_{114,1}_2 : Pdis_{114_STORAGE_1,1}_2 - PR_{114}_0 <= 0.0
Dub_{115,1}_2 : Pdis_{115_STORAGE_1,1}_2 - PR_{115}_0 <= 0.0
Dub_{116,1}_2 : Pdis_{116_STORAGE_1,1}_2 - PR_{116}_0 <= 0.0
Dub_{117,1}_2 : Pdis_{11

Dub_{111,7}_2 : Pdis_{111_STORAGE_1,7}_2 - PR_{111}_0 <= 0.0
Dub_{112,7}_2 : Pdis_{112_STORAGE_1,7}_2 - PR_{112}_0 <= 0.0
Dub_{113,7}_2 : Pdis_{113_STORAGE_1,7}_2 - PR_{113}_0 <= 0.0
Dub_{114,7}_2 : Pdis_{114_STORAGE_1,7}_2 - PR_{114}_0 <= 0.0
Dub_{115,7}_2 : Pdis_{115_STORAGE_1,7}_2 - PR_{115}_0 <= 0.0
Dub_{116,7}_2 : Pdis_{116_STORAGE_1,7}_2 - PR_{116}_0 <= 0.0
Dub_{117,7}_2 : Pdis_{117_STORAGE_1,7}_2 - PR_{117}_0 <= 0.0
Dub_{118,7}_2 : Pdis_{118_STORAGE_1,7}_2 - PR_{118}_0 <= 0.0
Dub_{119,7}_2 : Pdis_{119_STORAGE_1,7}_2 - PR_{119}_0 <= 0.0
Dub_{120,7}_2 : Pdis_{120_STORAGE_1,7}_2 - PR_{120}_0 <= 0.0
Dub_{121,7}_2 : Pdis_{121_STORAGE_1,7}_2 - PR_{121}_0 <= 0.0
Dub_{122,7}_2 : Pdis_{122_STORAGE_1,7}_2 - PR_{122}_0 <= 0.0
Dub_{123,7}_2 : Pdis_{123_STORAGE_1,7}_2 - PR_{123}_0 <= 0.0
Dub_{124,7}_2 : Pdis_{124_STORAGE_1,7}_2 - PR_{124}_0 <= 0.0
Dub_{101,8}_2 : Pdis_{101_STORAGE_1,8}_2 - PR_{101}_0 <= 0.0
Dub_{102,8}_2 : Pdis_{102_STORAGE_1,8}_2 - PR_{102}_0 <= 0.0
Dub_{103,8}_2 : Pdis_{10

Dub_{106,13}_2 : Pdis_{106_STORAGE_1,13}_2 - PR_{106}_0 <= 0.0
Dub_{107,13}_2 : Pdis_{107_STORAGE_1,13}_2 - PR_{107}_0 <= 0.0
Dub_{108,13}_2 : Pdis_{108_STORAGE_1,13}_2 - PR_{108}_0 <= 0.0
Dub_{109,13}_2 : Pdis_{109_STORAGE_1,13}_2 - PR_{109}_0 <= 0.0
Dub_{110,13}_2 : Pdis_{110_STORAGE_1,13}_2 - PR_{110}_0 <= 0.0
Dub_{111,13}_2 : Pdis_{111_STORAGE_1,13}_2 - PR_{111}_0 <= 0.0
Dub_{112,13}_2 : Pdis_{112_STORAGE_1,13}_2 - PR_{112}_0 <= 0.0
Dub_{113,13}_2 : Pdis_{113_STORAGE_1,13}_2 - PR_{113}_0 <= 0.0
Dub_{114,13}_2 : Pdis_{114_STORAGE_1,13}_2 - PR_{114}_0 <= 0.0
Dub_{115,13}_2 : Pdis_{115_STORAGE_1,13}_2 - PR_{115}_0 <= 0.0
Dub_{116,13}_2 : Pdis_{116_STORAGE_1,13}_2 - PR_{116}_0 <= 0.0
Dub_{117,13}_2 : Pdis_{117_STORAGE_1,13}_2 - PR_{117}_0 <= 0.0
Dub_{118,13}_2 : Pdis_{118_STORAGE_1,13}_2 - PR_{118}_0 <= 0.0
Dub_{119,13}_2 : Pdis_{119_STORAGE_1,13}_2 - PR_{119}_0 <= 0.0
Dub_{120,13}_2 : Pdis_{120_STORAGE_1,13}_2 - PR_{120}_0 <= 0.0
Dub_{121,13}_2 : Pdis_{121_STORAGE_1,13}_2 - PR_{121}_0

Dub_{105,19}_2 : Pdis_{105_STORAGE_1,19}_2 - PR_{105}_0 <= 0.0
Dub_{106,19}_2 : Pdis_{106_STORAGE_1,19}_2 - PR_{106}_0 <= 0.0
Dub_{107,19}_2 : Pdis_{107_STORAGE_1,19}_2 - PR_{107}_0 <= 0.0
Dub_{108,19}_2 : Pdis_{108_STORAGE_1,19}_2 - PR_{108}_0 <= 0.0
Dub_{109,19}_2 : Pdis_{109_STORAGE_1,19}_2 - PR_{109}_0 <= 0.0
Dub_{110,19}_2 : Pdis_{110_STORAGE_1,19}_2 - PR_{110}_0 <= 0.0
Dub_{111,19}_2 : Pdis_{111_STORAGE_1,19}_2 - PR_{111}_0 <= 0.0
Dub_{112,19}_2 : Pdis_{112_STORAGE_1,19}_2 - PR_{112}_0 <= 0.0
Dub_{113,19}_2 : Pdis_{113_STORAGE_1,19}_2 - PR_{113}_0 <= 0.0
Dub_{114,19}_2 : Pdis_{114_STORAGE_1,19}_2 - PR_{114}_0 <= 0.0
Dub_{115,19}_2 : Pdis_{115_STORAGE_1,19}_2 - PR_{115}_0 <= 0.0
Dub_{116,19}_2 : Pdis_{116_STORAGE_1,19}_2 - PR_{116}_0 <= 0.0
Dub_{117,19}_2 : Pdis_{117_STORAGE_1,19}_2 - PR_{117}_0 <= 0.0
Dub_{118,19}_2 : Pdis_{118_STORAGE_1,19}_2 - PR_{118}_0 <= 0.0
Dub_{119,19}_2 : Pdis_{119_STORAGE_1,19}_2 - PR_{119}_0 <= 0.0
Dub_{120,19}_2 : Pdis_{120_STORAGE_1,19}_2 - PR_{120}_0

Dub_{119,24}_2 : Pdis_{119_STORAGE_1,24}_2 - PR_{119}_0 <= 0.0
Dub_{120,24}_2 : Pdis_{120_STORAGE_1,24}_2 - PR_{120}_0 <= 0.0
Dub_{121,24}_2 : Pdis_{121_STORAGE_1,24}_2 - PR_{121}_0 <= 0.0
Dub_{122,24}_2 : Pdis_{122_STORAGE_1,24}_2 - PR_{122}_0 <= 0.0
Dub_{123,24}_2 : Pdis_{123_STORAGE_1,24}_2 - PR_{123}_0 <= 0.0
Dub_{124,24}_2 : Pdis_{124_STORAGE_1,24}_2 - PR_{124}_0 <= 0.0


A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1753 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [58]:
# add storage ub constraint

for ts in timesteps
    for bus in buses
        ### change constraint here
        subcon = get_storage_ub(submodel, bus, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        value = JuMP.constraint_object(subcon).set.upper
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, coeff*exvar <= value, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Eub_{101,1}_2 : Est_{101_STORAGE_1,1}_2 - ER_{101}_0 <= 0.0
Eub_{102,1}_2 : Est_{102_STORAGE_1,1}_2 - ER_{102}_0 <= 0.0
Eub_{103,1}_2 : Est_{103_STORAGE_1,1}_2 - ER_{103}_0 <= 0.0
Eub_{104,1}_2 : Est_{104_STORAGE_1,1}_2 - ER_{104}_0 <= 0.0
Eub_{105,1}_2 : Est_{105_STORAGE_1,1}_2 - ER_{105}_0 <= 0.0
Eub_{106,1}_2 : Est_{106_STORAGE_1,1}_2 - ER_{106}_0 <= 0.0
Eub_{107,1}_2 : Est_{107_STORAGE_1,1}_2 - ER_{107}_0 <= 0.0
Eub_{108,1}_2 : Est_{108_STORAGE_1,1}_2 - ER_{108}_0 <= 0.0
Eub_{109,1}_2 : Est_{109_STORAGE_1,1}_2 - ER_{109}_0 <= 0.0
Eub_{110,1}_2 : Est_{110_STORAGE_1,1}_2 - ER_{110}_0 <= 0.0
Eub_{111,1}_2 : Est_{111_STORAGE_1,1}_2 - ER_{111}_0 <= 0.0
Eub_{112,1}_2 : Est_{112_STORAGE_1,1}_2 - ER_{112}_0 <= 0.0
Eub_{113,1}_2 : Est_{113_STORAGE_1,1}_2 - ER_{113}_0 <= 0.0
Eub_{114,1}_2 : Est_{114_STORAGE_1,1}_2 - ER_{114}_0 <= 0.0
Eub_{115,1}_2 : Est_{115_STORAGE_1,1}_2 - ER_{115}_0 <= 0.0
Eub_{116,1}_2 : Est_{116_STORAGE_1,1}_2 - ER_{116}_0 <= 0.0
Eub_{117,1}_2 : Est_{117_STORAGE_1,1}_2 

Eub_{106,7}_2 : Est_{106_STORAGE_1,7}_2 - ER_{106}_0 <= 0.0
Eub_{107,7}_2 : Est_{107_STORAGE_1,7}_2 - ER_{107}_0 <= 0.0
Eub_{108,7}_2 : Est_{108_STORAGE_1,7}_2 - ER_{108}_0 <= 0.0
Eub_{109,7}_2 : Est_{109_STORAGE_1,7}_2 - ER_{109}_0 <= 0.0
Eub_{110,7}_2 : Est_{110_STORAGE_1,7}_2 - ER_{110}_0 <= 0.0
Eub_{111,7}_2 : Est_{111_STORAGE_1,7}_2 - ER_{111}_0 <= 0.0
Eub_{112,7}_2 : Est_{112_STORAGE_1,7}_2 - ER_{112}_0 <= 0.0
Eub_{113,7}_2 : Est_{113_STORAGE_1,7}_2 - ER_{113}_0 <= 0.0
Eub_{114,7}_2 : Est_{114_STORAGE_1,7}_2 - ER_{114}_0 <= 0.0
Eub_{115,7}_2 : Est_{115_STORAGE_1,7}_2 - ER_{115}_0 <= 0.0
Eub_{116,7}_2 : Est_{116_STORAGE_1,7}_2 - ER_{116}_0 <= 0.0
Eub_{117,7}_2 : Est_{117_STORAGE_1,7}_2 - ER_{117}_0 <= 0.0
Eub_{118,7}_2 : Est_{118_STORAGE_1,7}_2 - ER_{118}_0 <= 0.0
Eub_{119,7}_2 : Est_{119_STORAGE_1,7}_2 - ER_{119}_0 <= 0.0
Eub_{120,7}_2 : Est_{120_STORAGE_1,7}_2 - ER_{120}_0 <= 0.0
Eub_{121,7}_2 : Est_{121_STORAGE_1,7}_2 - ER_{121}_0 <= 0.0
Eub_{122,7}_2 : Est_{122_STORAGE_1,7}_2 

Eub_{105,13}_2 : Est_{105_STORAGE_1,13}_2 - ER_{105}_0 <= 0.0
Eub_{106,13}_2 : Est_{106_STORAGE_1,13}_2 - ER_{106}_0 <= 0.0
Eub_{107,13}_2 : Est_{107_STORAGE_1,13}_2 - ER_{107}_0 <= 0.0
Eub_{108,13}_2 : Est_{108_STORAGE_1,13}_2 - ER_{108}_0 <= 0.0
Eub_{109,13}_2 : Est_{109_STORAGE_1,13}_2 - ER_{109}_0 <= 0.0
Eub_{110,13}_2 : Est_{110_STORAGE_1,13}_2 - ER_{110}_0 <= 0.0
Eub_{111,13}_2 : Est_{111_STORAGE_1,13}_2 - ER_{111}_0 <= 0.0
Eub_{112,13}_2 : Est_{112_STORAGE_1,13}_2 - ER_{112}_0 <= 0.0
Eub_{113,13}_2 : Est_{113_STORAGE_1,13}_2 - ER_{113}_0 <= 0.0
Eub_{114,13}_2 : Est_{114_STORAGE_1,13}_2 - ER_{114}_0 <= 0.0
Eub_{115,13}_2 : Est_{115_STORAGE_1,13}_2 - ER_{115}_0 <= 0.0
Eub_{116,13}_2 : Est_{116_STORAGE_1,13}_2 - ER_{116}_0 <= 0.0
Eub_{117,13}_2 : Est_{117_STORAGE_1,13}_2 - ER_{117}_0 <= 0.0
Eub_{118,13}_2 : Est_{118_STORAGE_1,13}_2 - ER_{118}_0 <= 0.0
Eub_{119,13}_2 : Est_{119_STORAGE_1,13}_2 - ER_{119}_0 <= 0.0
Eub_{120,13}_2 : Est_{120_STORAGE_1,13}_2 - ER_{120}_0 <= 0.0
Eub_{121

Eub_{114,19}_2 : Est_{114_STORAGE_1,19}_2 - ER_{114}_0 <= 0.0
Eub_{115,19}_2 : Est_{115_STORAGE_1,19}_2 - ER_{115}_0 <= 0.0
Eub_{116,19}_2 : Est_{116_STORAGE_1,19}_2 - ER_{116}_0 <= 0.0
Eub_{117,19}_2 : Est_{117_STORAGE_1,19}_2 - ER_{117}_0 <= 0.0
Eub_{118,19}_2 : Est_{118_STORAGE_1,19}_2 - ER_{118}_0 <= 0.0
Eub_{119,19}_2 : Est_{119_STORAGE_1,19}_2 - ER_{119}_0 <= 0.0
Eub_{120,19}_2 : Est_{120_STORAGE_1,19}_2 - ER_{120}_0 <= 0.0
Eub_{121,19}_2 : Est_{121_STORAGE_1,19}_2 - ER_{121}_0 <= 0.0
Eub_{122,19}_2 : Est_{122_STORAGE_1,19}_2 - ER_{122}_0 <= 0.0
Eub_{123,19}_2 : Est_{123_STORAGE_1,19}_2 - ER_{123}_0 <= 0.0
Eub_{124,19}_2 : Est_{124_STORAGE_1,19}_2 - ER_{124}_0 <= 0.0
Eub_{101,20}_2 : Est_{101_STORAGE_1,20}_2 - ER_{101}_0 <= 0.0
Eub_{102,20}_2 : Est_{102_STORAGE_1,20}_2 - ER_{102}_0 <= 0.0
Eub_{103,20}_2 : Est_{103_STORAGE_1,20}_2 - ER_{103}_0 <= 0.0
Eub_{104,20}_2 : Est_{104_STORAGE_1,20}_2 - ER_{104}_0 <= 0.0
Eub_{105,20}_2 : Est_{105_STORAGE_1,20}_2 - ER_{105}_0 <= 0.0
Eub_{106

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2329 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [59]:
# add thermal interval constraint

for ts in timesteps
    for gen in gens
        ### change constraint here
        subcon = get_thermal_interval(submodel, gen, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        upper = JuMP.constraint_object(subcon).set.upper
        lower = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, lower <= coeff*exvar <= upper, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Tint_{101_CT_1,1}_2 : Pth_{101_CT_1,1}_2 in [0.0, 0.2]
Tint_{101_CT_2,1}_2 : Pth_{101_CT_2,1}_2 in [0.0, 0.2]
Tint_{101_STEAM_3,1}_2 : Pth_{101_STEAM_3,1}_2 in [0.0, 0.76]
Tint_{101_STEAM_4,1}_2 : Pth_{101_STEAM_4,1}_2 in [0.0, 0.76]
Tint_{102_CT_1,1}_2 : Pth_{102_CT_1,1}_2 in [0.0, 0.2]
Tint_{102_CT_2,1}_2 : Pth_{102_CT_2,1}_2 in [0.0, 0.2]
Tint_{102_STEAM_3,1}_2 : Pth_{102_STEAM_3,1}_2 in [0.0, 0.76]
Tint_{102_STEAM_4,1}_2 : Pth_{102_STEAM_4,1}_2 in [0.0, 0.76]
Tint_{107_CC_1,1}_2 : Pth_{107_CC_1,1}_2 in [0.0, 3.55]
Tint_{113_CT_1,1}_2 : Pth_{113_CT_1,1}_2 in [0.0, 0.55]
Tint_{113_CT_2,1}_2 : Pth_{113_CT_2,1}_2 in [0.0, 0.55]
Tint_{113_CT_3,1}_2 : Pth_{113_CT_3,1}_2 in [0.0, 0.55]
Tint_{113_CT_4,1}_2 : Pth_{113_CT_4,1}_2 in [0.0, 0.55]
Tint_{115_STEAM_1,1}_2 : Pth_{115_STEAM_1,1}_2 in [0.0, 0.12]
Tint_{115_STEAM_2,1}_2 : Pth_{115_STEAM_2,1}_2 in [0.0, 0.12]
Tint_{115_STEAM_3,1}_2 : Pth_{115_STEAM_3,1}_2 in [0.0, 1.55]
Tint_{116_STEAM_1,1}_2 : Pth_{116_STEAM_1,1}_2 in [0.0, 1.55]
Tint

Tint_{102_CT_2,7}_2 : Pth_{102_CT_2,7}_2 in [0.0, 0.2]
Tint_{102_STEAM_3,7}_2 : Pth_{102_STEAM_3,7}_2 in [0.0, 0.76]
Tint_{102_STEAM_4,7}_2 : Pth_{102_STEAM_4,7}_2 in [0.0, 0.76]
Tint_{107_CC_1,7}_2 : Pth_{107_CC_1,7}_2 in [0.0, 3.55]
Tint_{113_CT_1,7}_2 : Pth_{113_CT_1,7}_2 in [0.0, 0.55]
Tint_{113_CT_2,7}_2 : Pth_{113_CT_2,7}_2 in [0.0, 0.55]
Tint_{113_CT_3,7}_2 : Pth_{113_CT_3,7}_2 in [0.0, 0.55]
Tint_{113_CT_4,7}_2 : Pth_{113_CT_4,7}_2 in [0.0, 0.55]
Tint_{115_STEAM_1,7}_2 : Pth_{115_STEAM_1,7}_2 in [0.0, 0.12]
Tint_{115_STEAM_2,7}_2 : Pth_{115_STEAM_2,7}_2 in [0.0, 0.12]
Tint_{115_STEAM_3,7}_2 : Pth_{115_STEAM_3,7}_2 in [0.0, 1.55]
Tint_{116_STEAM_1,7}_2 : Pth_{116_STEAM_1,7}_2 in [0.0, 1.55]
Tint_{118_CC_1,7}_2 : Pth_{118_CC_1,7}_2 in [0.0, 3.55]
Tint_{123_STEAM_2,7}_2 : Pth_{123_STEAM_2,7}_2 in [0.0, 1.55]
Tint_{123_STEAM_3,7}_2 : Pth_{123_STEAM_3,7}_2 in [0.0, 3.5]
Tint_{123_CT_1,7}_2 : Pth_{123_CT_1,7}_2 in [0.0, 0.55]
Tint_{123_CT_4,7}_2 : Pth_{123_CT_4,7}_2 in [0.0, 0.55]
Ti

Tint_{121_NUCLEAR_1,12}_2 : Pth_{121_NUCLEAR_1,12}_2 in [0.0, 4.0]
Tint_{101_CT_1,13}_2 : Pth_{101_CT_1,13}_2 in [0.0, 0.2]
Tint_{101_CT_2,13}_2 : Pth_{101_CT_2,13}_2 in [0.0, 0.2]
Tint_{101_STEAM_3,13}_2 : Pth_{101_STEAM_3,13}_2 in [0.0, 0.76]
Tint_{101_STEAM_4,13}_2 : Pth_{101_STEAM_4,13}_2 in [0.0, 0.76]
Tint_{102_CT_1,13}_2 : Pth_{102_CT_1,13}_2 in [0.0, 0.2]
Tint_{102_CT_2,13}_2 : Pth_{102_CT_2,13}_2 in [0.0, 0.2]
Tint_{102_STEAM_3,13}_2 : Pth_{102_STEAM_3,13}_2 in [0.0, 0.76]
Tint_{102_STEAM_4,13}_2 : Pth_{102_STEAM_4,13}_2 in [0.0, 0.76]
Tint_{107_CC_1,13}_2 : Pth_{107_CC_1,13}_2 in [0.0, 3.55]
Tint_{113_CT_1,13}_2 : Pth_{113_CT_1,13}_2 in [0.0, 0.55]
Tint_{113_CT_2,13}_2 : Pth_{113_CT_2,13}_2 in [0.0, 0.55]
Tint_{113_CT_3,13}_2 : Pth_{113_CT_3,13}_2 in [0.0, 0.55]
Tint_{113_CT_4,13}_2 : Pth_{113_CT_4,13}_2 in [0.0, 0.55]
Tint_{115_STEAM_1,13}_2 : Pth_{115_STEAM_1,13}_2 in [0.0, 0.12]
Tint_{115_STEAM_2,13}_2 : Pth_{115_STEAM_2,13}_2 in [0.0, 0.12]
Tint_{115_STEAM_3,13}_2 : Pth_{

Tint_{118_CC_1,18}_2 : Pth_{118_CC_1,18}_2 in [0.0, 3.55]
Tint_{123_STEAM_2,18}_2 : Pth_{123_STEAM_2,18}_2 in [0.0, 1.55]
Tint_{123_STEAM_3,18}_2 : Pth_{123_STEAM_3,18}_2 in [0.0, 3.5]
Tint_{123_CT_1,18}_2 : Pth_{123_CT_1,18}_2 in [0.0, 0.55]
Tint_{123_CT_4,18}_2 : Pth_{123_CT_4,18}_2 in [0.0, 0.55]
Tint_{123_CT_5,18}_2 : Pth_{123_CT_5,18}_2 in [0.0, 0.55]
Tint_{121_NUCLEAR_1,18}_2 : Pth_{121_NUCLEAR_1,18}_2 in [0.0, 4.0]
Tint_{101_CT_1,19}_2 : Pth_{101_CT_1,19}_2 in [0.0, 0.2]
Tint_{101_CT_2,19}_2 : Pth_{101_CT_2,19}_2 in [0.0, 0.2]
Tint_{101_STEAM_3,19}_2 : Pth_{101_STEAM_3,19}_2 in [0.0, 0.76]
Tint_{101_STEAM_4,19}_2 : Pth_{101_STEAM_4,19}_2 in [0.0, 0.76]
Tint_{102_CT_1,19}_2 : Pth_{102_CT_1,19}_2 in [0.0, 0.2]
Tint_{102_CT_2,19}_2 : Pth_{102_CT_2,19}_2 in [0.0, 0.2]
Tint_{102_STEAM_3,19}_2 : Pth_{102_STEAM_3,19}_2 in [0.0, 0.76]
Tint_{102_STEAM_4,19}_2 : Pth_{102_STEAM_4,19}_2 in [0.0, 0.76]
Tint_{107_CC_1,19}_2 : Pth_{107_CC_1,19}_2 in [0.0, 3.55]
Tint_{113_CT_1,19}_2 : Pth_{113_

Tint_{116_STEAM_1,24}_2 : Pth_{116_STEAM_1,24}_2 in [0.0, 1.55]
Tint_{118_CC_1,24}_2 : Pth_{118_CC_1,24}_2 in [0.0, 3.55]
Tint_{123_STEAM_2,24}_2 : Pth_{123_STEAM_2,24}_2 in [0.0, 1.55]
Tint_{123_STEAM_3,24}_2 : Pth_{123_STEAM_3,24}_2 in [0.0, 3.5]
Tint_{123_CT_1,24}_2 : Pth_{123_CT_1,24}_2 in [0.0, 0.55]
Tint_{123_CT_4,24}_2 : Pth_{123_CT_4,24}_2 in [0.0, 0.55]
Tint_{123_CT_5,24}_2 : Pth_{123_CT_5,24}_2 in [0.0, 0.55]
Tint_{121_NUCLEAR_1,24}_2 : Pth_{121_NUCLEAR_1,24}_2 in [0.0, 4.0]


A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2329 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 576 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [60]:
# add branch interval constraint

for ts in timesteps
    for br in branches
        ### change constraint here
        subcon = get_branch_interval(submodel, br, ts)
        ###
        subconname = JuMP.name(subcon)
        exconname = string(subconname, "_$(scen)")

        terms = JuMP.constraint_object(subcon).func.terms
        ### change sense here
        upper = JuMP.constraint_object(subcon).set.upper
        lower = JuMP.constraint_object(subcon).set.lower
        ###

        count = 0
        for (subvar, coeff) in terms
            if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
            else
                exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
            end
            if count == 0
                ### change objective type here
                JuMP.@constraint(exmodel, lower <= coeff*exvar <= upper, 
                                            base_name = exconname)
                ###
                count += 1
            else
                excon = JuMP.constraint_by_name(exmodel, exconname)
                JuMP.set_normalized_coefficient(excon, exvar, coeff)
            end
        end
        println(JuMP.constraint_by_name(exmodel, exconname))
    end
end

exmodel

Bint_{A1,1}_2 : Fbr_{A1,1}_2 in [-1.75, 1.75]
Bint_{A2,1}_2 : Fbr_{A2,1}_2 in [-1.75, 1.75]
Bint_{A3,1}_2 : Fbr_{A3,1}_2 in [-1.75, 1.75]
Bint_{A4,1}_2 : Fbr_{A4,1}_2 in [-1.75, 1.75]
Bint_{A5,1}_2 : Fbr_{A5,1}_2 in [-1.75, 1.75]
Bint_{A6,1}_2 : Fbr_{A6,1}_2 in [-1.75, 1.75]
Bint_{A7,1}_2 : Fbr_{A7,1}_2 in [-4.0, 4.0]
Bint_{A8,1}_2 : Fbr_{A8,1}_2 in [-1.75, 1.75]
Bint_{A9,1}_2 : Fbr_{A9,1}_2 in [-1.75, 1.75]
Bint_{A10,1}_2 : Fbr_{A10,1}_2 in [-1.75, 1.75]
Bint_{A11,1}_2 : Fbr_{A11,1}_2 in [-1.75, 1.75]
Bint_{A12-1,1}_2 : Fbr_{A12-1,1}_2 in [-1.75, 1.75]
Bint_{A13-2,1}_2 : Fbr_{A13-2,1}_2 in [-1.75, 1.75]
Bint_{A14,1}_2 : Fbr_{A14,1}_2 in [-4.0, 4.0]
Bint_{A15,1}_2 : Fbr_{A15,1}_2 in [-4.0, 4.0]
Bint_{A16,1}_2 : Fbr_{A16,1}_2 in [-4.0, 4.0]
Bint_{A17,1}_2 : Fbr_{A17,1}_2 in [-4.0, 4.0]
Bint_{A18,1}_2 : Fbr_{A18,1}_2 in [-5.0, 5.0]
Bint_{A19,1}_2 : Fbr_{A19,1}_2 in [-5.0, 5.0]
Bint_{A20,1}_2 : Fbr_{A20,1}_2 in [-5.0, 5.0]
Bint_{A21,1}_2 : Fbr_{A21,1}_2 in [-5.0, 5.0]
Bint_{A22,1}_2 : Fbr

Bint_{A4,6}_2 : Fbr_{A4,6}_2 in [-1.75, 1.75]
Bint_{A5,6}_2 : Fbr_{A5,6}_2 in [-1.75, 1.75]
Bint_{A6,6}_2 : Fbr_{A6,6}_2 in [-1.75, 1.75]
Bint_{A7,6}_2 : Fbr_{A7,6}_2 in [-4.0, 4.0]
Bint_{A8,6}_2 : Fbr_{A8,6}_2 in [-1.75, 1.75]
Bint_{A9,6}_2 : Fbr_{A9,6}_2 in [-1.75, 1.75]
Bint_{A10,6}_2 : Fbr_{A10,6}_2 in [-1.75, 1.75]
Bint_{A11,6}_2 : Fbr_{A11,6}_2 in [-1.75, 1.75]
Bint_{A12-1,6}_2 : Fbr_{A12-1,6}_2 in [-1.75, 1.75]
Bint_{A13-2,6}_2 : Fbr_{A13-2,6}_2 in [-1.75, 1.75]
Bint_{A14,6}_2 : Fbr_{A14,6}_2 in [-4.0, 4.0]
Bint_{A15,6}_2 : Fbr_{A15,6}_2 in [-4.0, 4.0]
Bint_{A16,6}_2 : Fbr_{A16,6}_2 in [-4.0, 4.0]
Bint_{A17,6}_2 : Fbr_{A17,6}_2 in [-4.0, 4.0]
Bint_{A18,6}_2 : Fbr_{A18,6}_2 in [-5.0, 5.0]
Bint_{A19,6}_2 : Fbr_{A19,6}_2 in [-5.0, 5.0]
Bint_{A20,6}_2 : Fbr_{A20,6}_2 in [-5.0, 5.0]
Bint_{A21,6}_2 : Fbr_{A21,6}_2 in [-5.0, 5.0]
Bint_{A22,6}_2 : Fbr_{A22,6}_2 in [-5.0, 5.0]
Bint_{A23,6}_2 : Fbr_{A23,6}_2 in [-5.0, 5.0]
Bint_{A24,6}_2 : Fbr_{A24,6}_2 in [-5.0, 5.0]
Bint_{A25-1,6}_2 : F

Bint_{A15,11}_2 : Fbr_{A15,11}_2 in [-4.0, 4.0]
Bint_{A16,11}_2 : Fbr_{A16,11}_2 in [-4.0, 4.0]
Bint_{A17,11}_2 : Fbr_{A17,11}_2 in [-4.0, 4.0]
Bint_{A18,11}_2 : Fbr_{A18,11}_2 in [-5.0, 5.0]
Bint_{A19,11}_2 : Fbr_{A19,11}_2 in [-5.0, 5.0]
Bint_{A20,11}_2 : Fbr_{A20,11}_2 in [-5.0, 5.0]
Bint_{A21,11}_2 : Fbr_{A21,11}_2 in [-5.0, 5.0]
Bint_{A22,11}_2 : Fbr_{A22,11}_2 in [-5.0, 5.0]
Bint_{A23,11}_2 : Fbr_{A23,11}_2 in [-5.0, 5.0]
Bint_{A24,11}_2 : Fbr_{A24,11}_2 in [-5.0, 5.0]
Bint_{A25-1,11}_2 : Fbr_{A25-1,11}_2 in [-5.0, 5.0]
Bint_{A25-2,11}_2 : Fbr_{A25-2,11}_2 in [-5.0, 5.0]
Bint_{A26,11}_2 : Fbr_{A26,11}_2 in [-5.0, 5.0]
Bint_{A27,11}_2 : Fbr_{A27,11}_2 in [-5.0, 5.0]
Bint_{A28,11}_2 : Fbr_{A28,11}_2 in [-5.0, 5.0]
Bint_{A29,11}_2 : Fbr_{A29,11}_2 in [-5.0, 5.0]
Bint_{A30,11}_2 : Fbr_{A30,11}_2 in [-5.0, 5.0]
Bint_{A31-1,11}_2 : Fbr_{A31-1,11}_2 in [-5.0, 5.0]
Bint_{A31-2,11}_2 : Fbr_{A31-2,11}_2 in [-5.0, 5.0]
Bint_{A32-1,11}_2 : Fbr_{A32-1,11}_2 in [-5.0, 5.0]
Bint_{A32-2,11}_2 : 

Bint_{A5,16}_2 : Fbr_{A5,16}_2 in [-1.75, 1.75]
Bint_{A6,16}_2 : Fbr_{A6,16}_2 in [-1.75, 1.75]
Bint_{A7,16}_2 : Fbr_{A7,16}_2 in [-4.0, 4.0]
Bint_{A8,16}_2 : Fbr_{A8,16}_2 in [-1.75, 1.75]
Bint_{A9,16}_2 : Fbr_{A9,16}_2 in [-1.75, 1.75]
Bint_{A10,16}_2 : Fbr_{A10,16}_2 in [-1.75, 1.75]
Bint_{A11,16}_2 : Fbr_{A11,16}_2 in [-1.75, 1.75]
Bint_{A12-1,16}_2 : Fbr_{A12-1,16}_2 in [-1.75, 1.75]
Bint_{A13-2,16}_2 : Fbr_{A13-2,16}_2 in [-1.75, 1.75]
Bint_{A14,16}_2 : Fbr_{A14,16}_2 in [-4.0, 4.0]
Bint_{A15,16}_2 : Fbr_{A15,16}_2 in [-4.0, 4.0]
Bint_{A16,16}_2 : Fbr_{A16,16}_2 in [-4.0, 4.0]
Bint_{A17,16}_2 : Fbr_{A17,16}_2 in [-4.0, 4.0]
Bint_{A18,16}_2 : Fbr_{A18,16}_2 in [-5.0, 5.0]
Bint_{A19,16}_2 : Fbr_{A19,16}_2 in [-5.0, 5.0]
Bint_{A20,16}_2 : Fbr_{A20,16}_2 in [-5.0, 5.0]
Bint_{A21,16}_2 : Fbr_{A21,16}_2 in [-5.0, 5.0]
Bint_{A22,16}_2 : Fbr_{A22,16}_2 in [-5.0, 5.0]
Bint_{A23,16}_2 : Fbr_{A23,16}_2 in [-5.0, 5.0]
Bint_{A24,16}_2 : Fbr_{A24,16}_2 in [-5.0, 5.0]
Bint_{A25-1,16}_2 : Fbr_{A

Bint_{A5,21}_2 : Fbr_{A5,21}_2 in [-1.75, 1.75]
Bint_{A6,21}_2 : Fbr_{A6,21}_2 in [-1.75, 1.75]
Bint_{A7,21}_2 : Fbr_{A7,21}_2 in [-4.0, 4.0]
Bint_{A8,21}_2 : Fbr_{A8,21}_2 in [-1.75, 1.75]
Bint_{A9,21}_2 : Fbr_{A9,21}_2 in [-1.75, 1.75]
Bint_{A10,21}_2 : Fbr_{A10,21}_2 in [-1.75, 1.75]
Bint_{A11,21}_2 : Fbr_{A11,21}_2 in [-1.75, 1.75]
Bint_{A12-1,21}_2 : Fbr_{A12-1,21}_2 in [-1.75, 1.75]
Bint_{A13-2,21}_2 : Fbr_{A13-2,21}_2 in [-1.75, 1.75]
Bint_{A14,21}_2 : Fbr_{A14,21}_2 in [-4.0, 4.0]
Bint_{A15,21}_2 : Fbr_{A15,21}_2 in [-4.0, 4.0]
Bint_{A16,21}_2 : Fbr_{A16,21}_2 in [-4.0, 4.0]
Bint_{A17,21}_2 : Fbr_{A17,21}_2 in [-4.0, 4.0]
Bint_{A18,21}_2 : Fbr_{A18,21}_2 in [-5.0, 5.0]
Bint_{A19,21}_2 : Fbr_{A19,21}_2 in [-5.0, 5.0]
Bint_{A20,21}_2 : Fbr_{A20,21}_2 in [-5.0, 5.0]
Bint_{A21,21}_2 : Fbr_{A21,21}_2 in [-5.0, 5.0]
Bint_{A22,21}_2 : Fbr_{A22,21}_2 in [-5.0, 5.0]
Bint_{A23,21}_2 : Fbr_{A23,21}_2 in [-5.0, 5.0]
Bint_{A24,21}_2 : Fbr_{A24,21}_2 in [-5.0, 5.0]
Bint_{A25-1,21}_2 : Fbr_{A

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2329 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [61]:
submodel

A JuMP Model
Minimization problem with:
Variables: 4440
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3504 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2329 constraints
`AffExpr`-in-`MathOptInterface.Interval{Float64}`: 1488 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [62]:
# change objective function for extensive form

exobjfunc = JuMP.objective_function(exmodel)

for (subvar, coeff) in JuMP.objective_function(submodel).terms
    
    if JuMP.name(subvar)[1:2] == "PR" || JuMP.name(subvar)[1:2] == "ER"
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_0"))
    else
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(subvar),"_$(scen)"))
    end
    
    JuMP.add_to_expression!(exobjfunc, coeff, exvar)
    
end

In [63]:
JuMP.@objective(exmodel, Min, exobjfunc);

# end scenario loop

In [64]:
# set extensive form objective to 1/nscen times itself

exobjfunc = JuMP.objective_function(exmodel)

JuMP.@objective(exmodel, Min, 1/nscen*exobjfunc)

println(JuMP.objective_function(exmodel))

0.45025113294843755 Pth_{123_STEAM_2,1}_2 + 0.35286562530749993 Pth_{101_STEAM_3,1}_2 + 0.6237745039929375 Pth_{123_CT_1,1}_2 + 0.6286728385051875 Pth_{113_CT_3,1}_2 + 0.3941458845919687 Pth_{102_STEAM_3,1}_2 + 0.6286728385051875 Pth_{113_CT_2,1}_2 + 0.35286562530749993 Pth_{101_STEAM_4,1}_2 + 0.6286728385051875 Pth_{113_CT_1,1}_2 + 0.6286728385051875 Pth_{113_CT_4,1}_2 + 0.50680014612975 Pth_{118_CC_1,1}_2 + 0.4298088628608749 Pth_{115_STEAM_3,1}_2 + 2.00336822254125 Pth_{101_CT_1,1}_2 + 2.56887392770125 Pth_{115_STEAM_2,1}_2 + 2.00336822254125 Pth_{101_CT_2,1}_2 + 0.4139221883735624 Pth_{123_STEAM_3,1}_2 + 0.6237745039929375 Pth_{123_CT_5,1}_2 + 0.42469346949618747 Pth_{116_STEAM_1,1}_2 + 2.56887392770125 Pth_{115_STEAM_1,1}_2 + 0.14799016875 Pth_{121_NUCLEAR_1,1}_2 + 1.984798391146875 Pth_{102_CT_2,1}_2 + 0.6237745039929375 Pth_{123_CT_4,1}_2 + 1.984798391146875 Pth_{102_CT_1,1}_2 + 0.3941458845919687 Pth_{102_STEAM_4,1}_2 + 0.49583213624362504 Pth_{107_CC_1,1}_2 + 0.450251132948437

In [140]:
#checking model solutions

JuMP.set_optimizer(exmodel, Gurobi.Optimizer)

JuMP.set_optimizer(submodel, Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2022-08-19
Academic license - for non-commercial use only - expires 2022-08-19


In [141]:
JuMP.optimize!(exmodel)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 18697 rows, 11808 columns and 240120 nonzeros
Model fingerprint: 0x7649a6de
Coefficient statistics:
  Matrix range     [2e-04, 5e+00]
  Objective range  [2e-02, 5e+01]
  Bounds range     [1e-01, 5e+00]
  RHS range        [9e-03, 2e+02]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 11358 rows and 3458 columns
Presolve time: 0.13s
Presolved: 7339 rows, 8488 columns, 169938 nonzeros

Ordering time: 0.07s

Barrier performed 0 iterations in 0.22 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 1237 iterations and 0.22 seconds
Optimal objective  6.798147305e+01

User-callback calls 87, time in user-callback 0.00 sec


In [142]:
JuMP.optimize!(submodel)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9385 rows, 5928 columns and 120144 nonzeros
Model fingerprint: 0xe3f888d8
Coefficient statistics:
  Matrix range     [2e-04, 5e+00]
  Objective range  [4e-02, 7e+01]
  Bounds range     [1e-01, 5e+00]
  RHS range        [2e-02, 2e+02]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 5691 rows and 1728 columns
Presolve time: 0.06s
Presolved: 3694 rows, 4269 columns, 85071 nonzeros

Ordering time: 0.05s

Barrier performed 0 iterations in 0.13 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 668 iterations and 0.13 seconds
Optimal objective  3.582349172e+01

User-callback calls 67, time in user-callback 0.00 sec


In [144]:


vars = JuMP.all_variables(submodel)

for i = 1:length(vars)
    
    subval = JuMP.value(vars[i])
    
    if JuMP.name(vars[i])[1:2] == "PR" || JuMP.name(vars[i])[1:2] == "ER"
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(vars[i]),"_0"))
    else
        exvar = JuMP.variable_by_name(exmodel, string(JuMP.name(vars[i]),"_$(scen)"))
    end
    
    exval = JuMP.value(exvar)
    
    println("$(vars[i]), $(subval), $(exval)")
    
end



Pth_{123_STEAM_2,1}, 0.0, 0.0
Pth_{101_STEAM_3,1}, 0.0, 0.0
Pth_{123_CT_1,1}, 0.0, 0.0
Pth_{113_CT_3,1}, 0.0, 0.0
Pth_{102_STEAM_3,1}, 0.0, 0.0
Pth_{113_CT_2,1}, 0.0, 0.0
Pth_{101_STEAM_4,1}, 0.0, 0.0
Pth_{113_CT_1,1}, 0.0, 0.0
Pth_{113_CT_4,1}, 0.0, 0.0
Pth_{118_CC_1,1}, 0.0, 0.0
Pth_{115_STEAM_3,1}, 0.0, 0.0
Pth_{101_CT_1,1}, 0.0, 0.0
Pth_{115_STEAM_2,1}, 0.0, 0.0
Pth_{101_CT_2,1}, 0.0, 0.0
Pth_{123_STEAM_3,1}, 0.0, 0.0
Pth_{123_CT_5,1}, 0.0, 0.0
Pth_{116_STEAM_1,1}, 0.0, 0.0
Pth_{115_STEAM_1,1}, 0.0, 0.0
Pth_{121_NUCLEAR_1,1}, 3.409178617703408, 3.409178614048944
Pth_{102_CT_2,1}, 0.0, 0.0
Pth_{123_CT_4,1}, 0.0, 0.0
Pth_{102_CT_1,1}, 0.0, 0.0
Pth_{102_STEAM_4,1}, 0.0, 0.0
Pth_{107_CC_1,1}, 0.0, 0.0
Pth_{123_STEAM_2,2}, 0.0, 0.0
Pth_{101_STEAM_3,2}, 0.0, 0.0
Pth_{123_CT_1,2}, 0.0, 0.0
Pth_{113_CT_3,2}, 0.0, 0.0
Pth_{102_STEAM_3,2}, 0.0, 0.0
Pth_{113_CT_2,2}, 0.0, 0.0
Pth_{101_STEAM_4,2}, 0.0, 0.0
Pth_{113_CT_1,2}, 0.0, 0.0
Pth_{113_CT_4,2}, 0.0, 0.0
Pth_{118_CC_1,2}, 0.0, 0.0
Pth_{11

Pth_{101_STEAM_3,20}, 0.3578985628922915, 0.35789856226577776
Pth_{123_CT_1,20}, 0.0, 0.0
Pth_{113_CT_3,20}, 0.0, 0.0
Pth_{102_STEAM_3,20}, 0.0, 0.0
Pth_{113_CT_2,20}, 0.0, 0.0
Pth_{101_STEAM_4,20}, 0.76, 0.76
Pth_{113_CT_1,20}, 0.0, 0.0
Pth_{113_CT_4,20}, 0.0, 0.0
Pth_{118_CC_1,20}, 0.0, 0.0
Pth_{115_STEAM_3,20}, 0.0, 0.0
Pth_{101_CT_1,20}, 0.0, 0.0
Pth_{115_STEAM_2,20}, 0.0, 0.0
Pth_{101_CT_2,20}, 0.0, 0.0
Pth_{123_STEAM_3,20}, 0.0, 0.0
Pth_{123_CT_5,20}, 0.0, 0.0
Pth_{116_STEAM_1,20}, 0.0, 0.0
Pth_{115_STEAM_1,20}, 0.0, 0.0
Pth_{121_NUCLEAR_1,20}, 4.0, 4.0
Pth_{102_CT_2,20}, 0.0, 0.0
Pth_{123_CT_4,20}, 0.0, 0.0
Pth_{102_CT_1,20}, 0.0, 0.0
Pth_{102_STEAM_4,20}, 0.0, 0.0
Pth_{107_CC_1,20}, 0.0, 0.0
Pth_{123_STEAM_2,21}, 0.0, 0.0
Pth_{101_STEAM_3,21}, 0.6420093928851273, 0.0
Pth_{123_CT_1,21}, 0.0, 0.0
Pth_{113_CT_3,21}, 0.0, 0.0
Pth_{102_STEAM_3,21}, 0.0, 0.0
Pth_{113_CT_2,21}, 0.0, 0.0
Pth_{101_STEAM_4,21}, 0.0, 0.6420093922826332
Pth_{113_CT_1,21}, 0.0, 0.0
Pth_{113_CT_4,21}, 0.0, 0

Fbr_{A3,5}, 0.11622011141552038, 0.11622011123480394
Fbr_{A1,5}, 0.4118476792787049, 0.4118476759654641
Fbr_{A2,5}, -0.5097372584507733, -0.5097372585870197
Fbr_{A5,5}, 0.08786251389496962, 0.08786251378497825
Fbr_{A4,5}, -0.049664024309570065, -0.04966402444501651
Fbr_{A7,5}, -1.809851727286341, -1.8098517273723664
Fbr_{A6,5}, 0.6067448434575534, 0.6067448434073326
Fbr_{A8,5}, -0.33471597568406763, -0.33471597581951407
Fbr_{A9,5}, -0.1572756859990759, -0.15727568617979235
Fbr_{A10,5}, -0.4360167510577157, -0.4360167511677071
Fbr_{A11,5}, -0.48150667847634265, -0.4815066784763426
Fbr_{A12-1,5}, -0.6652262508212512, -0.6652262508317368
Fbr_{A13-2,5}, -0.4749815656550915, -0.47498156564460575
Fbr_{A14,5}, -0.6401737812705103, -0.6401737813641835
Fbr_{A15,5}, -0.42713294700719273, -0.4271329547390209
Fbr_{A17,5}, -0.8016176046687127, -0.801617604813292
Fbr_{A16,5}, -1.0178068241241474, -1.0178068242597902
Fbr_{A18,5}, 0.18404295120918132, 0.18404295096913045
Fbr_{A19,5}, -1.84202354872995

Fbr_{A9,9}, -0.03869909166283321, -0.03869909185620072
Fbr_{A10,9}, -0.387988128563179, -0.38798812868087035
Fbr_{A11,9}, -0.5152152384696781, -0.5152152384696781
Fbr_{A12-1,9}, -0.6997704595331404, -0.6997704595443602
Fbr_{A13-2,9}, -0.5202592271365378, -0.520259227125318
Fbr_{A14,9}, -0.6259662415312807, -0.6259662416315115
Fbr_{A15,9}, -0.4145383499169897, -0.41453835847012155
Fbr_{A17,9}, -0.768064894037564, -0.7680648941922645
Fbr_{A16,9}, -0.9826173335316677, -0.9826173336768061
Fbr_{A18,9}, 0.21545045844977073, 0.21545045819291508
Fbr_{A19,9}, -1.824034025695879, -1.8240340371962258
Fbr_{A21,9}, -1.0169549388052201, -1.016954947874953
Fbr_{A20,9}, -0.16564830514933349, -0.16564830478743303
Fbr_{A22,9}, -1.0424541536649092, -1.0424541535598646
Fbr_{A23,9}, -2.6236480728875864, -2.6236480843879333
Fbr_{A25-1,9}, -2.600143251924061, -2.6001432480788624
Fbr_{A26,9}, 1.7900393116443858, 1.790039308226918
Fbr_{A25-2,9}, -2.6001432480702507, -2.6001432480788624
Fbr_{A24,9}, 2.103661340

Fbr_{A25-1,14}, -2.6594693361908974, -2.6594693346894074
Fbr_{A26,14}, 1.9502468341941082, 1.9502468307404512
Fbr_{A25-2,14}, -2.6594693346813307, -2.6594693346894074
Fbr_{A24,14}, 2.1432774395337573, 2.1432774394635814
Fbr_{A28,14}, 3.407405726519624, 3.407405717030225
Fbr_{A27,14}, -4.3857963618064035, -4.385796358281189
Fbr_{A29,14}, -1.3327616286096704, -1.332761628595698
Fbr_{A30,14}, -3.053034733196733, -3.053034736808944
Fbr_{A31-1,14}, -1.3100132969131073, -1.3100132969061211
Fbr_{A31-2,14}, -1.3100132969131073, -1.3100132969061211
Fbr_{A32-2,14}, 1.3538605770417635, 1.3538605770039853
Fbr_{A32-1,14}, 1.3538605864556053, 1.3538605770039853
Fbr_{A33-2,14}, 1.106458299244472, 1.1064582992066938
Fbr_{A33-1,14}, 1.106458299244472, 1.1064582992066934
Fbr_{A34,14}, -3.938965266803268, -3.938965263191057
Fbr_{A3,15}, 0.025312879556770695, 0.02531287786387726
Fbr_{A1,15}, 0.28474470700226023, 0.28474470675650737
Fbr_{A2,15}, -0.6031595129309365, -0.6031595130673577
Fbr_{A5,15}, 0.03277

...

Pdis_{106_STORAGE_1,17}, 0.0, 0.0
Pdis_{107_STORAGE_1,17}, 0.0, 0.0
Pdis_{108_STORAGE_1,17}, 0.0, 0.0
Pdis_{109_STORAGE_1,17}, 0.0, 0.0
Pdis_{110_STORAGE_1,17}, 0.0, 0.0
Pdis_{111_STORAGE_1,17}, 0.0, 0.0
Pdis_{112_STORAGE_1,17}, 0.0, 0.0
Pdis_{113_STORAGE_1,17}, 0.0, 0.0
Pdis_{114_STORAGE_1,17}, 0.0, 0.0
Pdis_{115_STORAGE_1,17}, 0.0, 0.0
Pdis_{116_STORAGE_1,17}, 0.0, 0.0
Pdis_{117_STORAGE_1,17}, 0.0, 0.0
Pdis_{118_STORAGE_1,17}, 0.0, 0.0
Pdis_{119_STORAGE_1,17}, 0.0, 0.0
Pdis_{120_STORAGE_1,17}, 0.0, 0.0
Pdis_{121_STORAGE_1,17}, 0.0, 0.0
Pdis_{122_STORAGE_1,17}, 0.0, 0.0
Pdis_{123_STORAGE_1,17}, 0.0, 0.0
Pdis_{124_STORAGE_1,17}, 0.0, 0.0
Est_{101_STORAGE_1,17}, 0.0, 0.0
Est_{102_STORAGE_1,17}, 0.0, 0.0
Est_{103_STORAGE_1,17}, 0.0, 0.0
Est_{104_STORAGE_1,17}, 0.0, 0.0
Est_{105_STORAGE_1,17}, 0.0, 0.0
Est_{106_STORAGE_1,17}, 0.0, 0.0
Est_{107_STORAGE_1,17}, 0.0, 0.0
Est_{108_STORAGE_1,17}, 0.0, 0.0
Est_{109_STORAGE_1,17}, 0.0, 0.0
Est_{110_STORAGE_1,17}, 0.0, 0.0
Est_{111_STORAGE_1,17}, 

Pch_{120_STORAGE_1,23}, 0.0, 0.0
Pch_{121_STORAGE_1,23}, 0.0, 0.0
Pch_{122_STORAGE_1,23}, 0.0, 0.0
Pch_{123_STORAGE_1,23}, 0.0, 0.0
Pch_{124_STORAGE_1,23}, 0.0, 0.0
Pdis_{101_STORAGE_1,23}, 0.0, 0.0
Pdis_{102_STORAGE_1,23}, 0.0, 0.0
Pdis_{103_STORAGE_1,23}, 0.0, 0.0
Pdis_{104_STORAGE_1,23}, 0.0, 0.0
Pdis_{105_STORAGE_1,23}, 0.0, 0.0
Pdis_{106_STORAGE_1,23}, 0.0, 0.0
Pdis_{107_STORAGE_1,23}, 0.0, 0.0
Pdis_{108_STORAGE_1,23}, 0.0, 0.0
Pdis_{109_STORAGE_1,23}, 0.0, 0.0
Pdis_{110_STORAGE_1,23}, 0.0, 0.0
Pdis_{111_STORAGE_1,23}, 0.0, 0.0
Pdis_{112_STORAGE_1,23}, 0.0, 0.0
Pdis_{113_STORAGE_1,23}, 0.0, 0.0
Pdis_{114_STORAGE_1,23}, 0.0, 0.0
Pdis_{115_STORAGE_1,23}, 0.0, 0.0
Pdis_{116_STORAGE_1,23}, 0.0, 0.0
Pdis_{117_STORAGE_1,23}, 0.0, 0.0
Pdis_{118_STORAGE_1,23}, 0.0, 0.0
Pdis_{119_STORAGE_1,23}, 0.0, 0.0
Pdis_{120_STORAGE_1,23}, 0.0, 0.0
Pdis_{121_STORAGE_1,23}, 0.0, 0.0
Pdis_{122_STORAGE_1,23}, 0.0, 0.0
Pdis_{123_STORAGE_1,23}, 0.0, 0.0
Pdis_{124_STORAGE_1,23}, 0.0, 0.0
Est_{101_STORAGE_1,

LOL_{102,6}, 0.0, 0.0
OL_{102,6}, 0.0, 0.0
LOL_{103,6}, 0.0, 0.0
OL_{103,6}, 0.0, 0.0
LOL_{104,6}, 0.0, 0.0
OL_{104,6}, 0.0, 0.0
LOL_{105,6}, 0.0, 0.0
OL_{105,6}, 0.0, 0.0
LOL_{106,6}, 0.0, 0.0
OL_{106,6}, 0.0, 0.0
LOL_{107,6}, 0.0, 0.0
OL_{107,6}, 0.0, 0.0
LOL_{108,6}, 0.0, 0.0
OL_{108,6}, 0.0, 0.0
LOL_{109,6}, 0.0, 0.0
OL_{109,6}, 0.0, 0.0
LOL_{110,6}, 0.0, 0.0
OL_{110,6}, 0.0, 0.0
LOL_{111,6}, 0.0, 0.0
OL_{111,6}, 0.0, 0.0
LOL_{112,6}, 0.0, 0.0
OL_{112,6}, 0.0, 0.0
LOL_{113,6}, 0.0, 0.0
OL_{113,6}, 0.0, 0.0
LOL_{114,6}, 0.0, 0.0
OL_{114,6}, 0.0, 0.0
LOL_{115,6}, 0.0, 0.0
OL_{115,6}, 0.0, 0.0
LOL_{116,6}, 0.0, 0.0
OL_{116,6}, 0.0, 0.0
LOL_{117,6}, 0.0, 0.0
OL_{117,6}, 0.0, 0.0
LOL_{118,6}, 0.0, 0.0
OL_{118,6}, 0.0, 0.0
LOL_{119,6}, 0.0, 0.0
OL_{119,6}, 0.0, 0.0
LOL_{120,6}, 0.0, 0.0
OL_{120,6}, 0.0, 0.0
LOL_{121,6}, 0.0, 0.0
OL_{121,6}, 0.0, 0.0
LOL_{122,6}, 0.0, 0.0
OL_{122,6}, 0.0, 0.0
LOL_{123,6}, 0.0, 0.0
OL_{123,6}, 0.0, 0.0
LOL_{124,6}, 0.0, 0.0
OL_{124,6}, 0.0, 0.0
LOL_{101,7}

LOL_{117,14}, 0.0, 0.0
OL_{117,14}, 0.0, 0.0
LOL_{118,14}, 0.0, 0.0
OL_{118,14}, 0.0, 0.0
LOL_{119,14}, 0.0, 0.0
OL_{119,14}, 0.0, 0.0
LOL_{120,14}, 0.0, 0.0
OL_{120,14}, 0.0, 0.0
LOL_{121,14}, 0.0, 0.0
OL_{121,14}, 0.0, 0.0
LOL_{122,14}, 0.0, 0.0
OL_{122,14}, 0.0, 0.0
LOL_{123,14}, 0.0, 0.0
OL_{123,14}, 0.0, 0.0
LOL_{124,14}, 0.0, 0.0
OL_{124,14}, 0.0, 0.0
LOL_{101,15}, 0.0, 0.0
OL_{101,15}, 0.0, 0.0
LOL_{102,15}, 0.0, 0.0
OL_{102,15}, 0.0, 0.0
LOL_{103,15}, 0.0, 0.0
OL_{103,15}, 0.0, 0.0
LOL_{104,15}, 0.0, 0.0
OL_{104,15}, 0.0, 0.0
LOL_{105,15}, 0.0, 0.0
OL_{105,15}, 0.0, 0.0
LOL_{106,15}, 0.0, 0.0
OL_{106,15}, 0.0, 0.0
LOL_{107,15}, 0.0, 0.0
OL_{107,15}, 0.0, 0.0
LOL_{108,15}, 0.0, 0.0
OL_{108,15}, 0.0, 0.0
LOL_{109,15}, 0.0, 0.0
OL_{109,15}, 0.0, 0.0
LOL_{110,15}, 0.0, 0.0
OL_{110,15}, 0.0, 0.0
LOL_{111,15}, 0.0, 0.0
OL_{111,15}, 0.0, 0.0
LOL_{112,15}, 0.0, 0.0
OL_{112,15}, 0.0, 0.0
LOL_{113,15}, 0.0, 0.0
OL_{113,15}, 0.0, 0.0
LOL_{114,15}, 0.0, 0.0
OL_{114,15}, 0.0, 0.0
LOL_{115,1

OL_{123,22}, 0.0, 0.0
LOL_{124,22}, 0.0, 0.0
OL_{124,22}, 0.0, 0.0
LOL_{101,23}, 0.0, 0.0
OL_{101,23}, 0.0, 0.0
LOL_{102,23}, 0.0, 0.0
OL_{102,23}, 0.0, 0.0
LOL_{103,23}, 0.0, 0.0
OL_{103,23}, 0.0, 0.0
LOL_{104,23}, 0.0, 0.0
OL_{104,23}, 0.0, 0.0
LOL_{105,23}, 0.0, 0.0
OL_{105,23}, 0.0, 0.0
LOL_{106,23}, 0.0, 0.0
OL_{106,23}, 0.0, 0.0
LOL_{107,23}, 0.0, 0.0
OL_{107,23}, 0.0, 0.0
LOL_{108,23}, 0.0, 0.0
OL_{108,23}, 0.0, 0.0
LOL_{109,23}, 0.0, 0.0
OL_{109,23}, 0.0, 0.0
LOL_{110,23}, 0.0, 0.0
OL_{110,23}, 0.0, 0.0
LOL_{111,23}, 0.0, 0.0
OL_{111,23}, 0.0, 0.0
LOL_{112,23}, 0.0, 0.0
OL_{112,23}, 0.0, 0.0
LOL_{113,23}, 0.0, 0.0
OL_{113,23}, 0.0, 0.0
LOL_{114,23}, 0.0, 0.0
OL_{114,23}, 0.0, 0.0
LOL_{115,23}, 0.0, 0.0
OL_{115,23}, 0.0, 0.0
LOL_{116,23}, 0.0, 0.0
OL_{116,23}, 0.0, 0.0
LOL_{117,23}, 0.0, 0.0
OL_{117,23}, 0.0, 0.0
LOL_{118,23}, 0.0, 0.0
OL_{118,23}, 0.0, 0.0
LOL_{119,23}, 0.0, 0.0
OL_{119,23}, 0.0, 0.0
LOL_{120,23}, 0.0, 0.0
OL_{120,23}, 0.0, 0.0
LOL_{121,23}, 0.0, 0.0
OL_{121,23

In [145]:
exvars = JuMP.all_variables(exmodel)

for i = 1:length(exvars)
    
    println(i, " ", exvars[i], " ", JuMP.value(exvars[i]))
end
    

1 PR_{101}_0 0.0
2 ER_{101}_0 0.0
3 PR_{102}_0 0.0
4 ER_{102}_0 0.0
5 PR_{103}_0 0.0
6 ER_{103}_0 0.0
7 PR_{104}_0 0.0
8 ER_{104}_0 0.0
9 PR_{105}_0 0.0
10 ER_{105}_0 0.0
11 PR_{106}_0 0.0
12 ER_{106}_0 0.0
13 PR_{107}_0 0.0
14 ER_{107}_0 0.0
15 PR_{108}_0 0.0
16 ER_{108}_0 0.0
17 PR_{109}_0 0.0
18 ER_{109}_0 0.0
19 PR_{110}_0 0.0
20 ER_{110}_0 0.0
21 PR_{111}_0 0.0
22 ER_{111}_0 0.0
23 PR_{112}_0 0.0
24 ER_{112}_0 0.0
25 PR_{113}_0 0.0
26 ER_{113}_0 0.0
27 PR_{114}_0 0.0
28 ER_{114}_0 0.0
29 PR_{115}_0 0.0
30 ER_{115}_0 0.0
31 PR_{116}_0 0.0
32 ER_{116}_0 0.0
33 PR_{117}_0 0.0
34 ER_{117}_0 0.0
35 PR_{118}_0 0.0
36 ER_{118}_0 0.0
37 PR_{119}_0 0.0
38 ER_{119}_0 0.0
39 PR_{120}_0 0.0
40 ER_{120}_0 0.0
41 PR_{121}_0 0.0
42 ER_{121}_0 0.0
43 PR_{122}_0 0.0
44 ER_{122}_0 0.0
45 PR_{123}_0 0.0
46 ER_{123}_0 0.0
47 PR_{124}_0 0.0
48 ER_{124}_0 0.0
49 Pth_{101_CT_1,1}_1 0.0
50 Pth_{101_CT_2,1}_1 0.0
51 Pth_{101_STEAM_3,1}_1 0.0
52 Pth_{101_STEAM_4,1}_1 0.0
53 Pth_{102_CT_1,1}_1 0.0
54 Pth_{1

306 Pth_{118_CC_1,11}_1 0.0
307 Pth_{123_STEAM_2,11}_1 0.0
308 Pth_{123_STEAM_3,11}_1 3.5
309 Pth_{123_CT_1,11}_1 0.0
310 Pth_{123_CT_4,11}_1 0.0
311 Pth_{123_CT_5,11}_1 0.0
312 Pth_{121_NUCLEAR_1,11}_1 4.0
313 Pth_{101_CT_1,12}_1 0.0
314 Pth_{101_CT_2,12}_1 0.0
315 Pth_{101_STEAM_3,12}_1 0.76
316 Pth_{101_STEAM_4,12}_1 0.76
317 Pth_{102_CT_1,12}_1 0.0
318 Pth_{102_CT_2,12}_1 0.0
319 Pth_{102_STEAM_3,12}_1 0.76
320 Pth_{102_STEAM_4,12}_1 0.76
321 Pth_{107_CC_1,12}_1 0.0
322 Pth_{113_CT_1,12}_1 0.0
323 Pth_{113_CT_2,12}_1 0.0
324 Pth_{113_CT_3,12}_1 0.0
325 Pth_{113_CT_4,12}_1 0.0
326 Pth_{115_STEAM_1,12}_1 0.0
327 Pth_{115_STEAM_2,12}_1 0.0
328 Pth_{115_STEAM_3,12}_1 0.0
329 Pth_{116_STEAM_1,12}_1 0.6891202351733681
330 Pth_{118_CC_1,12}_1 0.0
331 Pth_{123_STEAM_2,12}_1 0.0
332 Pth_{123_STEAM_3,12}_1 3.5
333 Pth_{123_CT_1,12}_1 0.0
334 Pth_{123_CT_4,12}_1 0.0
335 Pth_{123_CT_5,12}_1 0.0
336 Pth_{121_NUCLEAR_1,12}_1 4.0
337 Pth_{101_CT_1,13}_1 0.0
338 Pth_{101_CT_2,13}_1 0.0
339 Pth_{10

655 Fbr_{A30,1}_1 -3.042389529195617
656 Fbr_{A31-1,1}_1 -1.064263660214912
657 Fbr_{A31-2,1}_1 -1.064263660214912
658 Fbr_{A32-1,1}_1 1.2154811526202989
659 Fbr_{A32-2,1}_1 1.2154811618951487
660 Fbr_{A33-1,1}_1 0.9942837241310589
661 Fbr_{A33-2,1}_1 0.9942837241310589
662 Fbr_{A34,1}_1 -4.0896104708043834
663 Fbr_{A1,2}_1 0.20925300628716398
664 Fbr_{A2,2}_1 -0.572340772490102
665 Fbr_{A3,2}_1 -0.010450078354014708
666 Fbr_{A4,2}_1 -0.13551196634992185
667 Fbr_{A5,2}_1 0.009272639330138825
668 Fbr_{A6,2}_1 0.5527071451293124
669 Fbr_{A7,2}_1 -1.7476110078134273
670 Fbr_{A8,2}_1 -0.39145456626297476
671 Fbr_{A9,2}_1 -0.25601663251389595
672 Fbr_{A10,2}_1 -0.46110835776878245
673 Fbr_{A11,2}_1 -0.4323354762894606
674 Fbr_{A12-1,2}_1 -0.6075640867781013
675 Fbr_{A13-2,2}_1 -0.4162063227313594
676 Fbr_{A14,2}_1 -0.628857802081915
677 Fbr_{A15,2}_1 -0.4227233765741873
678 Fbr_{A16,2}_1 -1.008477707332711
679 Fbr_{A17,2}_1 -0.799296955568749
680 Fbr_{A18,2}_1 0.1486058896748954
681 Fbr_{A1

925 Fbr_{A32-2,8}_1 0.7915097320821418
926 Fbr_{A33-1,8}_1 0.4983479291529079
927 Fbr_{A33-2,8}_1 0.4983479291529079
928 Fbr_{A34,8}_1 -2.8687517522903243
929 Fbr_{A1,9}_1 0.05720053015208182
930 Fbr_{A2,9}_1 0.19097881572763054
931 Fbr_{A3,9}_1 0.8047647392900108
932 Fbr_{A4,9}_1 0.5755797330061099
933 Fbr_{A5,9}_1 0.5821353947138903
934 Fbr_{A6,9}_1 0.5057238467905858
935 Fbr_{A7,9}_1 -1.0931715557800836
936 Fbr_{A8,9}_1 0.2555599443043798
937 Fbr_{A9,9}_1 0.497718718810599
938 Fbr_{A10,9}_1 -0.006009085150123494
939 Fbr_{A11,9}_1 -0.5405739717320068
940 Fbr_{A12-1,9}_1 -0.6755746771970276
941 Fbr_{A13-2,9}_1 -0.6045044899349793
942 Fbr_{A14,9}_1 -0.3651530453054573
943 Fbr_{A15,9}_1 -0.3059414045401406
944 Fbr_{A16,9}_1 -0.5080884769470858
945 Fbr_{A17,9}_1 -0.4480017823066371
946 Fbr_{A18,9}_1 0.1609156724428106
947 Fbr_{A19,9}_1 -1.0341571946953538
948 Fbr_{A20,9}_1 0.05418668427462431
949 Fbr_{A21,9}_1 -0.808129871121402
950 Fbr_{A22,9}_1 -0.9309144648334304
951 Fbr_{A23,9}_1 -1.

1152 Fbr_{A32-1,14}_1 -0.41435795329417324
1153 Fbr_{A32-2,14}_1 -0.41435794701476414
1154 Fbr_{A33-1,14}_1 -0.6600185189114778
1155 Fbr_{A33-2,14}_1 -0.6600185189114778
1156 Fbr_{A34,14}_1 0.17813533638246337
1157 Fbr_{A1,15}_1 -0.016801378190641647
1158 Fbr_{A2,15}_1 0.3654187489741951
1159 Fbr_{A3,15}_1 0.7592351757517971
1160 Fbr_{A4,15}_1 0.5820471705821489
1161 Fbr_{A5,15}_1 0.5509819772661627
1162 Fbr_{A6,15}_1 0.23315125923740027
1163 Fbr_{A7,15}_1 -0.5546449385662756
1164 Fbr_{A8,15}_1 0.2996498455859702
1165 Fbr_{A9,15}_1 0.48828638478523695
1166 Fbr_{A10,15}_1 0.03198148514450626
1167 Fbr_{A11,15}_1 -0.47702251226082826
1168 Fbr_{A12-1,15}_1 -0.5728426367604207
1169 Fbr_{A13-2,15}_1 -0.5567466692714442
1170 Fbr_{A14,15}_1 -0.2662243962186347
1171 Fbr_{A15,15}_1 -0.44164865723402047
1172 Fbr_{A16,15}_1 -0.301308595325213
1173 Fbr_{A17,15}_1 -0.47932532887383306
1174 Fbr_{A18,15}_1 -0.23420489153876753
1175 Fbr_{A19,15}_1 -0.3333281000050801
1176 Fbr_{A20,15}_1 0.0819973293012

...

7323 Est_{103_STORAGE_1,10}_2 0.0
7324 Est_{104_STORAGE_1,10}_2 0.0
7325 Est_{105_STORAGE_1,10}_2 0.0
7326 Est_{106_STORAGE_1,10}_2 0.0
7327 Est_{107_STORAGE_1,10}_2 0.0
7328 Est_{108_STORAGE_1,10}_2 0.0
7329 Est_{109_STORAGE_1,10}_2 0.0
7330 Est_{110_STORAGE_1,10}_2 0.0
7331 Est_{111_STORAGE_1,10}_2 0.0
7332 Est_{112_STORAGE_1,10}_2 0.0
7333 Est_{113_STORAGE_1,10}_2 0.0
7334 Est_{114_STORAGE_1,10}_2 0.0
7335 Est_{115_STORAGE_1,10}_2 0.0
7336 Est_{116_STORAGE_1,10}_2 0.0
7337 Est_{117_STORAGE_1,10}_2 0.0
7338 Est_{118_STORAGE_1,10}_2 0.0
7339 Est_{119_STORAGE_1,10}_2 0.0
7340 Est_{120_STORAGE_1,10}_2 0.0
7341 Est_{121_STORAGE_1,10}_2 0.0
7342 Est_{122_STORAGE_1,10}_2 0.0
7343 Est_{123_STORAGE_1,10}_2 0.0
7344 Est_{124_STORAGE_1,10}_2 0.0
7345 Est_{101_STORAGE_1,11}_2 0.0
7346 Est_{102_STORAGE_1,11}_2 0.0
7347 Est_{103_STORAGE_1,11}_2 0.0
7348 Est_{104_STORAGE_1,11}_2 0.0
7349 Est_{105_STORAGE_1,11}_2 0.0
7350 Est_{106_STORAGE_1,11}_2 0.0
7351 Est_{107_STORAGE_1,11}_2 0.0
7352 Est_{108_

7569 Est_{109_STORAGE_1,20}_2 0.0
7570 Est_{110_STORAGE_1,20}_2 0.0
7571 Est_{111_STORAGE_1,20}_2 0.0
7572 Est_{112_STORAGE_1,20}_2 0.0
7573 Est_{113_STORAGE_1,20}_2 0.0
7574 Est_{114_STORAGE_1,20}_2 0.0
7575 Est_{115_STORAGE_1,20}_2 0.0
7576 Est_{116_STORAGE_1,20}_2 0.0
7577 Est_{117_STORAGE_1,20}_2 0.0
7578 Est_{118_STORAGE_1,20}_2 0.0
7579 Est_{119_STORAGE_1,20}_2 0.0
7580 Est_{120_STORAGE_1,20}_2 0.0
7581 Est_{121_STORAGE_1,20}_2 0.0
7582 Est_{122_STORAGE_1,20}_2 0.0
7583 Est_{123_STORAGE_1,20}_2 0.0
7584 Est_{124_STORAGE_1,20}_2 0.0
7585 Est_{101_STORAGE_1,21}_2 0.0
7586 Est_{102_STORAGE_1,21}_2 0.0
7587 Est_{103_STORAGE_1,21}_2 0.0
7588 Est_{104_STORAGE_1,21}_2 0.0
7589 Est_{105_STORAGE_1,21}_2 0.0
7590 Est_{106_STORAGE_1,21}_2 0.0
7591 Est_{107_STORAGE_1,21}_2 0.0
7592 Est_{108_STORAGE_1,21}_2 0.0
7593 Est_{109_STORAGE_1,21}_2 0.0
7594 Est_{110_STORAGE_1,21}_2 0.0
7595 Est_{111_STORAGE_1,21}_2 0.0
7596 Est_{112_STORAGE_1,21}_2 0.0
7597 Est_{113_STORAGE_1,21}_2 0.0
7598 Est_{114_

7911 LOL_{115,10}_2 0.0
7912 LOL_{116,10}_2 0.0
7913 LOL_{117,10}_2 0.0
7914 LOL_{118,10}_2 0.0
7915 LOL_{119,10}_2 0.0
7916 LOL_{120,10}_2 0.0
7917 LOL_{121,10}_2 0.0
7918 LOL_{122,10}_2 0.0
7919 LOL_{123,10}_2 0.0
7920 LOL_{124,10}_2 0.0
7921 LOL_{101,11}_2 0.0
7922 LOL_{102,11}_2 0.0
7923 LOL_{103,11}_2 0.0
7924 LOL_{104,11}_2 0.0
7925 LOL_{105,11}_2 0.0
7926 LOL_{106,11}_2 0.0
7927 LOL_{107,11}_2 0.0
7928 LOL_{108,11}_2 0.0
7929 LOL_{109,11}_2 0.0
7930 LOL_{110,11}_2 0.0
7931 LOL_{111,11}_2 0.0
7932 LOL_{112,11}_2 0.0
7933 LOL_{113,11}_2 0.0
7934 LOL_{114,11}_2 0.0
7935 LOL_{115,11}_2 0.0
7936 LOL_{116,11}_2 0.0
7937 LOL_{117,11}_2 0.0
7938 LOL_{118,11}_2 0.0
7939 LOL_{119,11}_2 0.0
7940 LOL_{120,11}_2 0.0
7941 LOL_{121,11}_2 0.0
7942 LOL_{122,11}_2 0.0
7943 LOL_{123,11}_2 0.0
7944 LOL_{124,11}_2 0.0
7945 LOL_{101,12}_2 0.0
7946 LOL_{102,12}_2 0.0
7947 LOL_{103,12}_2 0.0
7948 LOL_{104,12}_2 0.0
7949 LOL_{105,12}_2 0.0
7950 LOL_{106,12}_2 0.0
7951 LOL_{107,12}_2 0.0
7952 LOL_{108,12

8280 OL_{124,1}_2 0.0
8281 OL_{101,2}_2 0.0
8282 OL_{102,2}_2 0.0
8283 OL_{103,2}_2 0.0
8284 OL_{104,2}_2 0.0
8285 OL_{105,2}_2 0.0
8286 OL_{106,2}_2 0.0
8287 OL_{107,2}_2 0.0
8288 OL_{108,2}_2 0.0
8289 OL_{109,2}_2 0.0
8290 OL_{110,2}_2 0.0
8291 OL_{111,2}_2 0.0
8292 OL_{112,2}_2 0.0
8293 OL_{113,2}_2 0.0
8294 OL_{114,2}_2 0.0
8295 OL_{115,2}_2 0.0
8296 OL_{116,2}_2 0.0
8297 OL_{117,2}_2 0.0
8298 OL_{118,2}_2 0.0
8299 OL_{119,2}_2 0.0
8300 OL_{120,2}_2 0.0
8301 OL_{121,2}_2 0.0
8302 OL_{122,2}_2 0.0
8303 OL_{123,2}_2 0.0
8304 OL_{124,2}_2 0.0
8305 OL_{101,3}_2 0.0
8306 OL_{102,3}_2 0.0
8307 OL_{103,3}_2 0.0
8308 OL_{104,3}_2 0.0
8309 OL_{105,3}_2 0.0
8310 OL_{106,3}_2 0.0
8311 OL_{107,3}_2 0.0
8312 OL_{108,3}_2 0.0
8313 OL_{109,3}_2 0.0
8314 OL_{110,3}_2 0.0
8315 OL_{111,3}_2 0.0
8316 OL_{112,3}_2 0.0
8317 OL_{113,3}_2 0.0
8318 OL_{114,3}_2 0.0
8319 OL_{115,3}_2 0.0
8320 OL_{116,3}_2 0.0
8321 OL_{117,3}_2 0.0
8322 OL_{118,3}_2 0.0
8323 OL_{119,3}_2 0.0
8324 OL_{120,3}_2 0.0
8325 OL_{1

8666 OL_{102,18}_2 0.0
8667 OL_{103,18}_2 0.0
8668 OL_{104,18}_2 0.0
8669 OL_{105,18}_2 0.0
8670 OL_{106,18}_2 0.0
8671 OL_{107,18}_2 0.0
8672 OL_{108,18}_2 0.0
8673 OL_{109,18}_2 0.0
8674 OL_{110,18}_2 0.0
8675 OL_{111,18}_2 0.0
8676 OL_{112,18}_2 0.0
8677 OL_{113,18}_2 0.0
8678 OL_{114,18}_2 0.0
8679 OL_{115,18}_2 0.0
8680 OL_{116,18}_2 0.0
8681 OL_{117,18}_2 0.0
8682 OL_{118,18}_2 0.0
8683 OL_{119,18}_2 0.0
8684 OL_{120,18}_2 0.0
8685 OL_{121,18}_2 0.0
8686 OL_{122,18}_2 0.0
8687 OL_{123,18}_2 0.0
8688 OL_{124,18}_2 0.0
8689 OL_{101,19}_2 0.0
8690 OL_{102,19}_2 0.0
8691 OL_{103,19}_2 0.0
8692 OL_{104,19}_2 0.0
8693 OL_{105,19}_2 0.0
8694 OL_{106,19}_2 0.0
8695 OL_{107,19}_2 0.0
8696 OL_{108,19}_2 0.0
8697 OL_{109,19}_2 0.0
8698 OL_{110,19}_2 0.0
8699 OL_{111,19}_2 0.0
8700 OL_{112,19}_2 0.0
8701 OL_{113,19}_2 0.0
8702 OL_{114,19}_2 0.0
8703 OL_{115,19}_2 0.0
8704 OL_{116,19}_2 0.0
8705 OL_{117,19}_2 0.0
8706 OL_{118,19}_2 0.0
8707 OL_{119,19}_2 0.0
8708 OL_{120,19}_2 0.0
8709 OL_{12

In [147]:
0.5*(1.001394544e2 + 3.582349172e1) - 6.798147305e+01

9.999993721976352e-9

In [148]:
JuMP.write_to_file(submodel, "./basic_test_scen2.mps")

In [149]:
JuMP.write_to_file(exmodel, "./basic_extensive_form.mps")